<a href="https://colab.research.google.com/github/Amirstad/AML-Project/blob/main/AML_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AML Final Project

In this project I've chosen this article [Cyclical learning rates for training neural networks](https://arxiv.org/pdf/1506.01186.pdf).

The article describes a new method for dynamically changing a CNN’s learning rate to improve its training.

There are a few options to run the code, either run it on an online service such as Google Colab / DataLore / Jupyter, or you can download it and run it in python.

Both options will take a long time to finish unless you have a serious GPU, so better take my word here.

## Imports

In [38]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.optimizers import Adam
from keras.models import Sequential
from scipy.stats import friedmanchisquare
from sklearn.model_selection import KFold
from tensorflow.keras.datasets import cifar10
from sklearn.preprocessing import label_binarize
from keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from keras.callbacks import LearningRateScheduler, EarlyStopping
from sklearn.metrics import roc_auc_score, confusion_matrix, average_precision_score
from keras.layers import Conv2D, Dense, AvgPool2D, BatchNormalization, Dropout, Flatten

!pip install scikit-posthocs
from scikit_posthocs import posthoc_nemenyi_friedman


     |████████████████████████████████| 51kB 4.4MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for scikit-posthocs: filename=scikit_posthocs-0.6.7-cp37-none-any.whl size=37903 sha256=189c96eef43873b1f66032e8a381f27b620bc67b011add976765e29d2a712cf5
  Stored in directory: /root/.cache/pip/wheels/00/0f/56/897c22820a48a74f50f1a85faaa4b133bb8865e8b094b596f5
Successfully built scikit-posthocs


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Datasets

In [2]:
(c10_X_train, c10_y_train), (c10_X_test, c10_y_test) = cifar10.load_data()
c10_classes = 10

170500096/170498071 [==============================] - 4s 0us/step


In [4]:
BATCH_SIZE = 100
cifar10_X = np.concatenate((c10_X_train, c10_X_test))
cifar10_y = np.concatenate((c10_y_train, c10_y_test))

## Model

In [5]:
def sequential_shallow(n_classes):
  model = Sequential(name="Basic_Model")

  # first block
  model.add(Conv2D(8, kernel_size=(3, 3), activation='relu', input_shape=(32,32,3), name="1st_Conv"))
  model.add(AvgPool2D(pool_size=(2, 2),name="1st_Avg_Pool"))
  model.add(BatchNormalization(name="1st_Normalization"))
  model.add(Dropout(0.1,name="1st_Dropout"))

  # # second block
  # model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', name="2nd_Conv"))
  # model.add(AvgPool2D(pool_size=(2, 2),name="2nd_Avg_Pool"))
  # model.add(BatchNormalization(name="2nd_Normalization"))
  # model.add(Dropout(0.2,name="2nd_Dropout"))
# 
  # # third block
  # model.add(Conv2D(64, kernel_size=(3, 3), activation='relu',name="3rd_Conv"))
  # model.add(AvgPool2D(pool_size=(2, 2), name="3rd_Avg_Pool"))
  # model.add(BatchNormalization(name="3rd_Normalization"))
  # model.add(Dropout(0.3,name="3rd_Dropout"))

  # final dense & output layer
  model.add(Flatten())
  model.add(Dense(n_classes, activation='softmax'))

  return model


def make_model(n_classes, lr=0.001):
  model = sequential_shallow(n_classes)

  model.compile(optimizer=Adam(learning_rate=lr),
                loss=SparseCategoricalCrossentropy(),
                metrics=[SparseCategoricalAccuracy(name='accuracy')])

  return model

## Algorithm

In [6]:
def CLR_callback(base_lr, max_lr, stepsize):
  def scheduler(epoch, lr):
    cycle = np.floor(1 + epoch /(2 * stepsize))
    x = abs(epoch/stepsize - 2*cycle + 1)
    lr = base_lr + (max_lr - base_lr) * max(0, (1-x))
    return lr

  return LearningRateScheduler(scheduler)

## Improvement

In [7]:
def find_boundaries(accuracy, n_classes, base_lr, step, verbose):
  def index_to_lr(idx):
    return base_lr + idx*step


  l_index = 0
  size = len(accuracy)
  end = size - 1
  max_accuracy = np.max(accuracy)
  while l_index < end-1 and (accuracy[l_index] + max_accuracy / size > accuracy[l_index+1] or accuracy[l_index] < 1.1 / n_classes):
    l_index += 1

  r_index = l_index + 1
  while r_index < end and accuracy[r_index] / accuracy[r_index+1] < 1.1 and accuracy[r_index-1] < accuracy[r_index+1]:
    r_index += 1

  min_lr = index_to_lr(l_index)
  max_lr = index_to_lr(r_index)
  if verbose >= 0:
    print((min_lr, max_lr))

  return min_lr, max_lr


# Here we use the CLR callback to train a model with incrementing LR 
def train_incrementing(X, y, n_classes, base_lr, max_lr, step, verbose):
  stepsize = round((max_lr - base_lr)//step) + 1
  clr = CLR_callback(base_lr=base_lr, max_lr=max_lr, stepsize=stepsize)
  model = make_model(n_classes)
  history = model.fit(X,
                      y,
                      batch_size=BATCH_SIZE,
                      epochs=stepsize,
                      callbacks=[clr],
                      verbose=max(0, verbose)
                     )
  accuracy = history.history['accuracy']
  del(model)
  del(history)

  if verbose >= 0:
    plt.plot(accuracy)
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.show()

  return accuracy


def extract_boundaries(X, y, n_classes, base_lr, max_lr, step, verbose=-1):
  accuracy = train_incrementing(X, y, n_classes, base_lr, max_lr, step, verbose)
  return find_boundaries(accuracy, n_classes, base_lr, step, verbose)

## Metrics

In [8]:
# adapted from here
# https://stackoverflow.com/a/43331484
def compute_rates(y_true, y_pred, n_classes):
  cm = confusion_matrix(y_true, y_pred, [i+1 for i in range(n_classes)])

  tp = np.diag(cm)
  fp = cm.sum(axis=0) - tp
  fn = cm.sum(axis=1) - tp
  tn = cm.sum() - (fp + fn + tp)

  # Sensitivity, hit rate, recall, or true positive rate
  tpr = tp/(tp+fn)
  mean_tpr = np.mean(tpr[~np.isnan(tpr)])
  # Precision or positive predictive value
  precision = tp/(tp+fp)
  mean_precision = np.mean(precision[~np.isnan(precision)])
  # Fall out or false positive rate
  fpr = fp/(fp+tn)
  mean_fpr = np.mean(fpr[~np.isnan(fpr)])

  return mean_tpr, mean_fpr, mean_precision

## Training

In [9]:
PATIENCE = 4

In [10]:
def optimize_lr(X, y, n_classes, base_lr, max_lr, lr_step):
  print("Optimizing Learning Rate...")
  lr_scores = []
  for lr in np.arange(base_lr, max_lr, lr_step):
    acc_scores = []
    kfold = KFold(n_splits=3)
    for train_index, val_index in kfold.split(X):
      X_train, X_val = X[train_index], X[val_index]
      y_train, y_val = y[train_index], y[val_index]

      early_stopping = EarlyStopping(monitor='val_loss', patience=PATIENCE, restore_best_weights=True)

      model = make_model(n_classes, lr)
      model.fit(X_train, 
                y_train, 
                batch_size=BATCH_SIZE, 
                epochs=1000, 
                validation_data=(X_val, y_val), 
                callbacks=[early_stopping], 
                verbose=0
               )
      scores = model.evaluate(X_val, 
                              y_val, 
                              batch_size=BATCH_SIZE, 
                              verbose=0
                             )
      del(model)
      acc_scores.append(scores[1])

    avg_acc = np.mean(acc_scores)
    lr_scores.append(avg_acc)

  lr_idx = np.argmax(lr_scores)
  lr = base_lr + lr_step*lr_idx

  return lr


In [11]:
def build_train_model(name, X, y, n_classes, size,
                      base_lr, max_lr, 
                      clr=False, improved=False,
                      lr_step=None):
  
  models_rows = []
  auc_scores = []
  kfold = KFold(n_splits=10)
  fold = 1
  alg_name = "Baseline" if not clr else "Ipmroved" if improved else "Cyclic"
  for train_index, val_index in kfold.split(X):
    row = {
        "dataset name": name,
        "alg name": alg_name,
        "fold": fold,
        "lr" : None,
        "accuracy" : None,
        "tpr" : None,
        "fpr" : None,
        "precision" : None,
        "auc" : None,
        "pr_curve" : None,
        "train_time" : None,
        "infer_time" : None,
    }
    print("{0} Fold {1} {0}".format("-"*20, fold))
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    callbacks = []
    lr = 0.001
    if clr:
      min_boundary = base_lr
      max_boundary = max_lr
      if improved:
        assert lr_step is not None

        min_boundary, max_boundary = extract_boundaries(X_train, y_train, n_classes, base_lr, max_lr, lr_step)
        print("Base LR: {}, Max LR: {}".format(min_boundary, max_boundary))

      stepsize = 2*size/BATCH_SIZE
      clr = CLR_callback(base_lr=min_boundary, max_lr=max_boundary, stepsize=stepsize)
      callbacks.append(clr)
      row["lr"] = "{}-{}".format(min_boundary, max_boundary)
    else:
      assert lr_step is not None
      lr = optimize_lr(X_train, y_train, n_classes, base_lr, max_lr, lr_step)
      print("Learning Rate set to {}".format(lr))
      row["lr"] = lr
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=PATIENCE, restore_best_weights=True)
    callbacks.append(early_stopping)

    model = make_model(n_classes, lr)
    
    start = time.time()
    model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=1000, validation_data=(X_val, y_val), callbacks=callbacks)
    row["train_time"] = round(time.time() - start, 4)

    print("Evaluating...".format("-"*20, fold))
     
    start = time.time()
    probas = model.predict(X_val, batch_size=BATCH_SIZE)
    row["infer_time"] = round((time.time() - start) / (size/1000), 4)
    preds = np.argmax(probas, axis=1)
    row["accuracy"] = round(model.evaluate(X_val, y_val, batch_size=BATCH_SIZE)[1], 4)
    tpr, fpr, precision = compute_rates(y_val, preds, n_classes)
    row["tpr"] = round(tpr, 4)
    row["fpr"] = round(fpr, 4)
    row["precision"] = round(precision, 4)
    row["auc"] = round(roc_auc_score(y_val, probas, multi_class='ovr'), 4)
    auc_scores.append(row["auc"])
    Y_val = label_binarize(y_val.ravel(), classes=[i for i in range(n_classes)])
    row["pr_curve"] = round(average_precision_score(Y_val, probas), 4)
    print(row)
    del(model)
    models_rows.append(row)
    fold += 1

  mean_auc = np.mean(auc_scores)
  return models_rows, mean_auc


## Evaluation

In [12]:
make_model(10).summary()

Model: "Basic_Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1st_Conv (Conv2D)            (None, 30, 30, 8)         224       
_________________________________________________________________
1st_Avg_Pool (AveragePooling (None, 15, 15, 8)         0         
_________________________________________________________________
1st_Normalization (BatchNorm (None, 15, 15, 8)         32        
_________________________________________________________________
1st_Dropout (Dropout)        (None, 15, 15, 8)         0         
_________________________________________________________________
flatten (Flatten)            (None, 1800)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                18010     
Total params: 18,266
Trainable params: 18,250
Non-trainable params: 16
__________________________________________________

In [13]:
DATASET_NAME = "cifar10"

In [14]:
def triple_evaluate(name, X, y, n_classes, base_lr=0, max_lr=0.05, lr_step=0.001, size=1000, **_):
  # Baseline Model
  base_rows, base_auc = build_train_model(name, X, y, n_classes, base_lr=0, max_lr=0.05, lr_step=0.001, size=size)
  # Implementation Model
  alg_rows, alg_auc = build_train_model(name, X, y, n_classes, base_lr=base_lr, max_lr=max_lr, clr=True, lr_step=lr_step, size=size)
  # Improved Model
  improved_rows, improved_auc = build_train_model(name, X, y, n_classes, base_lr=0, max_lr=0.05, clr=True, improved=True, lr_step=0.001, size=size)

  return base_rows + alg_rows + improved_rows, [base_auc, alg_auc, improved_auc]

def evaluate_all(datasets):
  rows = []
  aucs = []
  for dataset in datasets:
    dataset_rows, dataset_aucs = triple_evaluate(**dataset)
    print(dataset_rows)
    rows.extend(dataset_rows)
    aucs.append(dataset_aucs)

  return rows, aucs


In [15]:
# test_limit = 1000
# datasets = [{
#     "name": "{}_{}:{}".format(DATASET_NAME, 50000 + i*test_limit, 50000 + (i+1)*test_limit),
#     "X": c10_X_test[i*test_limit:(i+1)*test_limit],
#     "y": c10_y_test[i*test_limit:(i+1)*test_limit],
#     "n_classes": c10_classes,
#     "base_lr": 0.001,
#     "max_lr": 0.02,
#     "lr_step": 0.001,
#     "size": test_limit,
#   } for i in range(10)]

# test_rows, test_aucs = evaluate_all(datasets)
# print(test_rows)
# print(test_aucs)

In [16]:
d1_scores = [{'dataset name': 'cifar10_50000:51000', 'alg name': 'Baseline', 'fold': 1, 'lr': 0.006, 'accuracy': 0.6513, 'tpr': 0.2464, 'fpr': 0.0835, 'precision': 0.2091, 'auc': 0.7621, 'pr_curve': 0.2901, 'train_time': 1.3373, 'infer_time': 0.598}, {'dataset name': 'cifar10_50000:51000', 'alg name': 'Baseline', 'fold': 2, 'lr': 0.004, 'accuracy': 0.7639, 'tpr': 0.2934, 'fpr': 0.0793, 'precision': 0.2922, 'auc': 0.7622, 'pr_curve': 0.3241, 'train_time': 1.8057, 'infer_time': 0.2655}, {'dataset name': 'cifar10_50000:51000', 'alg name': 'Baseline', 'fold': 3, 'lr': 0.013000000000000001, 'accuracy': 0.6385, 'tpr': 0.32, 'fpr': 0.0788, 'precision': 0.3444, 'auc': 0.7365, 'pr_curve': 0.3314, 'train_time': 1.3621, 'infer_time': 0.1619}, {'dataset name': 'cifar10_50000:51000', 'alg name': 'Baseline', 'fold': 4, 'lr': 0.007, 'accuracy': 0.7263, 'tpr': 0.3231, 'fpr': 0.0763, 'precision': 0.3104, 'auc': 0.7631, 'pr_curve': 0.3375, 'train_time': 1.5945, 'infer_time': 0.5436}, {'dataset name': 'cifar10_50000:51000', 'alg name': 'Baseline', 'fold': 5, 'lr': 0.003, 'accuracy': 0.5871, 'tpr': 0.3333, 'fpr': 0.0777, 'precision': 0.3406, 'auc': 0.7827, 'pr_curve': 0.3612, 'train_time': 1.437, 'infer_time': 0.1814}, {'dataset name': 'cifar10_50000:51000', 'alg name': 'Baseline', 'fold': 6, 'lr': 0.011, 'accuracy': 0.6375, 'tpr': 0.2222, 'fpr': 0.09, 'precision': 0.2519, 'auc': 0.7472, 'pr_curve': 0.331, 'train_time': 1.4119, 'infer_time': 0.1873}, {'dataset name': 'cifar10_50000:51000', 'alg name': 'Baseline', 'fold': 7, 'lr': 0.005, 'accuracy': 0.7582, 'tpr': 0.3245, 'fpr': 0.0768, 'precision': 0.355, 'auc': 0.7292, 'pr_curve': 0.3164, 'train_time': 2.0956, 'infer_time': 1.295}, {'dataset name': 'cifar10_50000:51000', 'alg name': 'Baseline', 'fold': 8, 'lr': 0.007, 'accuracy': 0.704, 'tpr': 0.3289, 'fpr': 0.0749, 'precision': 0.3075, 'auc': 0.7586, 'pr_curve': 0.3499, 'train_time': 1.5916, 'infer_time': 0.5209}, {'dataset name': 'cifar10_50000:51000', 'alg name': 'Baseline', 'fold': 9, 'lr': 0.003, 'accuracy': 0.6931, 'tpr': 0.4104, 'fpr': 0.0664, 'precision': 0.3792, 'auc': 0.8039, 'pr_curve': 0.3788, 'train_time': 1.4577, 'infer_time': 0.1936}, {'dataset name': 'cifar10_50000:51000', 'alg name': 'Baseline', 'fold': 10, 'lr': 0.006, 'accuracy': 0.7304, 'tpr': 0.41, 'fpr': 0.0606, 'precision': 0.4065, 'auc': 0.8265, 'pr_curve': 0.4417, 'train_time': 1.7403, 'infer_time': 0.1867}, {'dataset name': 'cifar10_50000:51000', 'alg name': 'Cyclic', 'fold': 1, 'lr': '0.001-0.02', 'accuracy': 0.5589, 'tpr': 0.392, 'fpr': 0.0676, 'precision': 0.4678, 'auc': 0.7816, 'pr_curve': 0.356, 'train_time': 1.3541, 'infer_time': 0.1698}, {'dataset name': 'cifar10_50000:51000', 'alg name': 'Cyclic', 'fold': 2, 'lr': '0.001-0.02', 'accuracy': 0.6548, 'tpr': 0.354, 'fpr': 0.0729, 'precision': 0.4112, 'auc': 0.7555, 'pr_curve': 0.3291, 'train_time': 1.4352, 'infer_time': 0.1741}, {'dataset name': 'cifar10_50000:51000', 'alg name': 'Cyclic', 'fold': 3, 'lr': '0.001-0.02', 'accuracy': 0.6519, 'tpr': 0.2872, 'fpr': 0.0767, 'precision': 0.3019, 'auc': 0.7063, 'pr_curve': 0.2823, 'train_time': 1.4036, 'infer_time': 0.6857}, {'dataset name': 'cifar10_50000:51000', 'alg name': 'Cyclic', 'fold': 4, 'lr': '0.001-0.02', 'accuracy': 0.6088, 'tpr': 0.285, 'fpr': 0.0772, 'precision': 0.2575, 'auc': 0.7605, 'pr_curve': 0.3196, 'train_time': 1.5792, 'infer_time': 0.1801}, {'dataset name': 'cifar10_50000:51000', 'alg name': 'Cyclic', 'fold': 5, 'lr': '0.001-0.02', 'accuracy': 0.5826, 'tpr': 0.2784, 'fpr': 0.0855, 'precision': 0.3259, 'auc': 0.7171, 'pr_curve': 0.2908, 'train_time': 1.3703, 'infer_time': 0.7251}, {'dataset name': 'cifar10_50000:51000', 'alg name': 'Cyclic', 'fold': 6, 'lr': '0.001-0.02', 'accuracy': 0.6077, 'tpr': 0.3306, 'fpr': 0.0735, 'precision': 0.3166, 'auc': 0.7689, 'pr_curve': 0.3431, 'train_time': 1.3647, 'infer_time': 0.7407}, {'dataset name': 'cifar10_50000:51000', 'alg name': 'Cyclic', 'fold': 7, 'lr': '0.001-0.02', 'accuracy': 0.5598, 'tpr': 0.3696, 'fpr': 0.0708, 'precision': 0.3829, 'auc': 0.7634, 'pr_curve': 0.3548, 'train_time': 1.3655, 'infer_time': 0.1622}, {'dataset name': 'cifar10_50000:51000', 'alg name': 'Cyclic', 'fold': 8, 'lr': '0.001-0.02', 'accuracy': 0.5615, 'tpr': 0.426, 'fpr': 0.0679, 'precision': 0.4447, 'auc': 0.7757, 'pr_curve': 0.3913, 'train_time': 1.3526, 'infer_time': 1.0729}, {'dataset name': 'cifar10_50000:51000', 'alg name': 'Cyclic', 'fold': 9, 'lr': '0.001-0.02', 'accuracy': 0.5774, 'tpr': 0.3164, 'fpr': 0.0702, 'precision': 0.3297, 'auc': 0.7828, 'pr_curve': 0.3392, 'train_time': 1.483, 'infer_time': 0.1836}, {'dataset name': 'cifar10_50000:51000', 'alg name': 'Cyclic', 'fold': 10, 'lr': '0.001-0.02', 'accuracy': 0.6603, 'tpr': 0.4138, 'fpr': 0.0621, 'precision': 0.3967, 'auc': 0.8425, 'pr_curve': 0.5017, 'train_time': 1.6186, 'infer_time': 0.1837}, {'dataset name': 'cifar10_50000:51000', 'alg name': 'Ipmroved', 'fold': 1, 'lr': '0.001-0.05', 'accuracy': 0.6247, 'tpr': 0.281, 'fpr': 0.0822, 'precision': 0.2677, 'auc': 0.7302, 'pr_curve': 0.2836, 'train_time': 1.3455, 'infer_time': 0.1765}, {'dataset name': 'cifar10_50000:51000', 'alg name': 'Ipmroved', 'fold': 2, 'lr': '0.001-0.05', 'accuracy': 0.5824, 'tpr': 0.3345, 'fpr': 0.0734, 'precision': 0.4247, 'auc': 0.7215, 'pr_curve': 0.3206, 'train_time': 1.3624, 'infer_time': 0.7357}, {'dataset name': 'cifar10_50000:51000', 'alg name': 'Ipmroved', 'fold': 3, 'lr': '0.002-0.05', 'accuracy': 0.6163, 'tpr': 0.2423, 'fpr': 0.0839, 'precision': 0.2516, 'auc': 0.6885, 'pr_curve': 0.3072, 'train_time': 1.3623, 'infer_time': 0.7786}, {'dataset name': 'cifar10_50000:51000', 'alg name': 'Ipmroved', 'fold': 4, 'lr': '0.0-0.05', 'accuracy': 0.5489, 'tpr': 0.2494, 'fpr': 0.0854, 'precision': 0.3476, 'auc': 0.7127, 'pr_curve': 0.2982, 'train_time': 1.6633, 'infer_time': 0.4103}, {'dataset name': 'cifar10_50000:51000', 'alg name': 'Ipmroved', 'fold': 5, 'lr': '0.002-0.05', 'accuracy': 0.6157, 'tpr': 0.3074, 'fpr': 0.0802, 'precision': 0.3154, 'auc': 0.7292, 'pr_curve': 0.3065, 'train_time': 1.3394, 'infer_time': 0.1749}, {'dataset name': 'cifar10_50000:51000', 'alg name': 'Ipmroved', 'fold': 6, 'lr': '0.0-0.05', 'accuracy': 0.518, 'tpr': 0.275, 'fpr': 0.0793, 'precision': 0.2711, 'auc': 0.7465, 'pr_curve': 0.3202, 'train_time': 1.2877, 'infer_time': 0.8015}, {'dataset name': 'cifar10_50000:51000', 'alg name': 'Ipmroved', 'fold': 7, 'lr': '0.001-0.05', 'accuracy': 0.4897, 'tpr': 0.2058, 'fpr': 0.0899, 'precision': 0.1763, 'auc': 0.7327, 'pr_curve': 0.2765, 'train_time': 1.2377, 'infer_time': 0.1789}, {'dataset name': 'cifar10_50000:51000', 'alg name': 'Ipmroved', 'fold': 8, 'lr': '0.002-0.05', 'accuracy': 0.5252, 'tpr': 0.3385, 'fpr': 0.075, 'precision': 0.2861, 'auc': 0.7419, 'pr_curve': 0.3528, 'train_time': 1.4241, 'infer_time': 0.6665}, {'dataset name': 'cifar10_50000:51000', 'alg name': 'Ipmroved', 'fold': 9, 'lr': '0.001-0.05', 'accuracy': 0.632, 'tpr': 0.2328, 'fpr': 0.0869, 'precision': 0.2519, 'auc': 0.7341, 'pr_curve': 0.2892, 'train_time': 1.8117, 'infer_time': 0.1948}, {'dataset name': 'cifar10_50000:51000', 'alg name': 'Ipmroved', 'fold': 10, 'lr': '0.001-0.05', 'accuracy': 0.5054, 'tpr': 0.2964, 'fpr': 0.0794, 'precision': 0.3826, 'auc': 0.7631, 'pr_curve': 0.359, 'train_time': 1.2807, 'infer_time': 0.1859}]
d2_scores = [{'dataset name': 'cifar10_51000:52000', 'alg name': 'Baseline', 'fold': 1, 'lr': 0.004, 'accuracy': 0.6602, 'tpr': 0.4017, 'fpr': 0.0703, 'precision': 0.5238, 'auc': 0.7815, 'pr_curve': 0.4011, 'train_time': 1.5021, 'infer_time': 0.6054}, {'dataset name': 'cifar10_51000:52000', 'alg name': 'Baseline', 'fold': 2, 'lr': 0.011, 'accuracy': 0.6489, 'tpr': 0.373, 'fpr': 0.0703, 'precision': 0.3264, 'auc': 0.736, 'pr_curve': 0.3225, 'train_time': 1.3962, 'infer_time': 0.5365}, {'dataset name': 'cifar10_51000:52000', 'alg name': 'Baseline', 'fold': 3, 'lr': 0.004, 'accuracy': 0.7589, 'tpr': 0.3518, 'fpr': 0.0733, 'precision': 0.3575, 'auc': 0.7458, 'pr_curve': 0.321, 'train_time': 1.6439, 'infer_time': 0.4562}, {'dataset name': 'cifar10_51000:52000', 'alg name': 'Baseline', 'fold': 4, 'lr': 0.003, 'accuracy': 0.6449, 'tpr': 0.3742, 'fpr': 0.0713, 'precision': 0.396, 'auc': 0.7421, 'pr_curve': 0.3224, 'train_time': 1.3549, 'infer_time': 0.1814}, {'dataset name': 'cifar10_51000:52000', 'alg name': 'Baseline', 'fold': 5, 'lr': 0.007, 'accuracy': 0.7537, 'tpr': 0.3439, 'fpr': 0.0701, 'precision': 0.3789, 'auc': 0.7645, 'pr_curve': 0.3513, 'train_time': 1.6659, 'infer_time': 0.5133}, {'dataset name': 'cifar10_51000:52000', 'alg name': 'Baseline', 'fold': 6, 'lr': 0.005, 'accuracy': 0.6546, 'tpr': 0.2949, 'fpr': 0.0773, 'precision': 0.2713, 'auc': 0.7246, 'pr_curve': 0.3241, 'train_time': 1.3395, 'infer_time': 0.1679}, {'dataset name': 'cifar10_51000:52000', 'alg name': 'Baseline', 'fold': 7, 'lr': 0.006, 'accuracy': 0.6661, 'tpr': 0.2504, 'fpr': 0.0832, 'precision': 0.2755, 'auc': 0.749, 'pr_curve': 0.351, 'train_time': 1.4466, 'infer_time': 0.6306}, {'dataset name': 'cifar10_51000:52000', 'alg name': 'Baseline', 'fold': 8, 'lr': 0.013000000000000001, 'accuracy': 0.6697, 'tpr': 0.2324, 'fpr': 0.0875, 'precision': 0.227, 'auc': 0.7213, 'pr_curve': 0.3413, 'train_time': 1.4545, 'infer_time': 0.1871}, {'dataset name': 'cifar10_51000:52000', 'alg name': 'Baseline', 'fold': 9, 'lr': 0.005, 'accuracy': 0.6843, 'tpr': 0.2624, 'fpr': 0.0844, 'precision': 0.2638, 'auc': 0.71, 'pr_curve': 0.2536, 'train_time': 1.4631, 'infer_time': 0.1938}, {'dataset name': 'cifar10_51000:52000', 'alg name': 'Baseline', 'fold': 10, 'lr': 0.004, 'accuracy': 0.6943, 'tpr': 0.2669, 'fpr': 0.0821, 'precision': 0.3431, 'auc': 0.7084, 'pr_curve': 0.2728, 'train_time': 1.5215, 'infer_time': 0.1647}, {'dataset name': 'cifar10_51000:52000', 'alg name': 'Cyclic', 'fold': 1, 'lr': '0.001-0.02', 'accuracy': 0.5952, 'tpr': 0.2874, 'fpr': 0.0802, 'precision': 0.3692, 'auc': 0.7636, 'pr_curve': 0.3639, 'train_time': 1.4162, 'infer_time': 0.1596}, {'dataset name': 'cifar10_51000:52000', 'alg name': 'Cyclic', 'fold': 2, 'lr': '0.001-0.02', 'accuracy': 0.6016, 'tpr': 0.4001, 'fpr': 0.0673, 'precision': 0.3681, 'auc': 0.7737, 'pr_curve': 0.3789, 'train_time': 1.329, 'infer_time': 0.1645}, {'dataset name': 'cifar10_51000:52000', 'alg name': 'Cyclic', 'fold': 3, 'lr': '0.001-0.02', 'accuracy': 0.6432, 'tpr': 0.3521, 'fpr': 0.0699, 'precision': 0.3704, 'auc': 0.7354, 'pr_curve': 0.3009, 'train_time': 1.5024, 'infer_time': 0.9271}, {'dataset name': 'cifar10_51000:52000', 'alg name': 'Cyclic', 'fold': 4, 'lr': '0.001-0.02', 'accuracy': 0.611, 'tpr': 0.3071, 'fpr': 0.0778, 'precision': 0.3105, 'auc': 0.7494, 'pr_curve': 0.3601, 'train_time': 1.4732, 'infer_time': 0.1897}, {'dataset name': 'cifar10_51000:52000', 'alg name': 'Cyclic', 'fold': 5, 'lr': '0.001-0.02', 'accuracy': 0.6157, 'tpr': 0.2275, 'fpr': 0.0809, 'precision': 0.3344, 'auc': 0.7715, 'pr_curve': 0.3245, 'train_time': 1.388, 'infer_time': 0.1746}, {'dataset name': 'cifar10_51000:52000', 'alg name': 'Cyclic', 'fold': 6, 'lr': '0.001-0.02', 'accuracy': 0.611, 'tpr': 0.3027, 'fpr': 0.0772, 'precision': 0.4756, 'auc': 0.7167, 'pr_curve': 0.3487, 'train_time': 1.3802, 'infer_time': 0.1754}, {'dataset name': 'cifar10_51000:52000', 'alg name': 'Cyclic', 'fold': 7, 'lr': '0.001-0.02', 'accuracy': 0.612, 'tpr': 0.332, 'fpr': 0.0774, 'precision': 0.3302, 'auc': 0.7516, 'pr_curve': 0.3911, 'train_time': 1.4538, 'infer_time': 0.1624}, {'dataset name': 'cifar10_51000:52000', 'alg name': 'Cyclic', 'fold': 8, 'lr': '0.001-0.02', 'accuracy': 0.5823, 'tpr': 0.4123, 'fpr': 0.0683, 'precision': 0.4644, 'auc': 0.7834, 'pr_curve': 0.4267, 'train_time': 1.3884, 'infer_time': 0.1682}, {'dataset name': 'cifar10_51000:52000', 'alg name': 'Cyclic', 'fold': 9, 'lr': '0.001-0.02', 'accuracy': 0.6132, 'tpr': 0.2571, 'fpr': 0.0797, 'precision': 0.298, 'auc': 0.7397, 'pr_curve': 0.3167, 'train_time': 1.3753, 'infer_time': 0.6989}, {'dataset name': 'cifar10_51000:52000', 'alg name': 'Cyclic', 'fold': 10, 'lr': '0.001-0.02', 'accuracy': 0.6035, 'tpr': 0.2738, 'fpr': 0.0798, 'precision': 0.2707, 'auc': 0.7074, 'pr_curve': 0.2842, 'train_time': 1.4392, 'infer_time': 0.6766}, {'dataset name': 'cifar10_51000:52000', 'alg name': 'Ipmroved', 'fold': 1, 'lr': '0.001-0.05', 'accuracy': 0.5954, 'tpr': 0.3499, 'fpr': 0.0773, 'precision': 0.3761, 'auc': 0.7999, 'pr_curve': 0.4053, 'train_time': 1.5854, 'infer_time': 0.2032}, {'dataset name': 'cifar10_51000:52000', 'alg name': 'Ipmroved', 'fold': 2, 'lr': '0.001-0.05', 'accuracy': 0.6046, 'tpr': 0.4429, 'fpr': 0.0677, 'precision': 0.4405, 'auc': 0.8107, 'pr_curve': 0.406, 'train_time': 1.3095, 'infer_time': 0.7755}, {'dataset name': 'cifar10_51000:52000', 'alg name': 'Ipmroved', 'fold': 3, 'lr': '0.002-0.05', 'accuracy': 0.6051, 'tpr': 0.2628, 'fpr': 0.0793, 'precision': 0.3181, 'auc': 0.7094, 'pr_curve': 0.2823, 'train_time': 1.3255, 'infer_time': 0.7981}, {'dataset name': 'cifar10_51000:52000', 'alg name': 'Ipmroved', 'fold': 4, 'lr': '0.001-0.05', 'accuracy': 0.5716, 'tpr': 0.3244, 'fpr': 0.0778, 'precision': 0.3454, 'auc': 0.7284, 'pr_curve': 0.297, 'train_time': 1.3137, 'infer_time': 0.1687}, {'dataset name': 'cifar10_51000:52000', 'alg name': 'Ipmroved', 'fold': 5, 'lr': '0.001-0.05', 'accuracy': 0.6042, 'tpr': 0.2679, 'fpr': 0.0791, 'precision': 0.3185, 'auc': 0.7599, 'pr_curve': 0.3267, 'train_time': 1.4291, 'infer_time': 0.1951}, {'dataset name': 'cifar10_51000:52000', 'alg name': 'Ipmroved', 'fold': 6, 'lr': '0.001-0.05', 'accuracy': 0.5423, 'tpr': 0.2413, 'fpr': 0.0843, 'precision': 0.3299, 'auc': 0.7429, 'pr_curve': 0.323, 'train_time': 1.5194, 'infer_time': 0.2482}, {'dataset name': 'cifar10_51000:52000', 'alg name': 'Ipmroved', 'fold': 7, 'lr': '0.001-0.05', 'accuracy': 0.6151, 'tpr': 0.3398, 'fpr': 0.0724, 'precision': 0.3583, 'auc': 0.7893, 'pr_curve': 0.4152, 'train_time': 1.2978, 'infer_time': 0.7715}, {'dataset name': 'cifar10_51000:52000', 'alg name': 'Ipmroved', 'fold': 8, 'lr': '0.0-0.05', 'accuracy': 0.5678, 'tpr': 0.3265, 'fpr': 0.0752, 'precision': 0.3333, 'auc': 0.7743, 'pr_curve': 0.343, 'train_time': 1.4735, 'infer_time': 0.6169}, {'dataset name': 'cifar10_51000:52000', 'alg name': 'Ipmroved', 'fold': 9, 'lr': '0.002-0.05', 'accuracy': 0.64, 'tpr': 0.2265, 'fpr': 0.0849, 'precision': 0.2648, 'auc': 0.7397, 'pr_curve': 0.3229, 'train_time': 1.3658, 'infer_time': 0.1659}, {'dataset name': 'cifar10_51000:52000', 'alg name': 'Ipmroved', 'fold': 10, 'lr': '0.001-0.05', 'accuracy': 0.5421, 'tpr': 0.2655, 'fpr': 0.0839, 'precision': 0.2587, 'auc': 0.6948, 'pr_curve': 0.2635, 'train_time': 1.3289, 'infer_time': 0.7518}]
d3_scores = [{'dataset name': 'cifar10_52000:53000', 'alg name': 'Baseline', 'fold': 1, 'lr': 0.006, 'accuracy': 0.6852, 'tpr': 0.3042, 'fpr': 0.0808, 'precision': 0.3019, 'auc': 0.7518, 'pr_curve': 0.3038, 'train_time': 4.6379, 'infer_time': 1.7301}, {'dataset name': 'cifar10_52000:53000', 'alg name': 'Baseline', 'fold': 2, 'lr': 0.004, 'accuracy': 0.7859, 'tpr': 0.2722, 'fpr': 0.0807, 'precision': 0.2783, 'auc': 0.7236, 'pr_curve': 0.2874, 'train_time': 6.7145, 'infer_time': 4.6125}, {'dataset name': 'cifar10_52000:53000', 'alg name': 'Baseline', 'fold': 3, 'lr': 0.004, 'accuracy': 0.6767, 'tpr': 0.4359, 'fpr': 0.0633, 'precision': 0.4414, 'auc': 0.7824, 'pr_curve': 0.3875, 'train_time': 5.4944, 'infer_time': 0.2273}, {'dataset name': 'cifar10_52000:53000', 'alg name': 'Baseline', 'fold': 4, 'lr': 0.004, 'accuracy': 0.7491, 'tpr': 0.3221, 'fpr': 0.0764, 'precision': 0.3329, 'auc': 0.7475, 'pr_curve': 0.3427, 'train_time': 5.238, 'infer_time': 0.973}, {'dataset name': 'cifar10_52000:53000', 'alg name': 'Baseline', 'fold': 5, 'lr': 0.008, 'accuracy': 0.7209, 'tpr': 0.3135, 'fpr': 0.0762, 'precision': 0.3366, 'auc': 0.7372, 'pr_curve': 0.3136, 'train_time': 6.4323, 'infer_time': 0.2315}, {'dataset name': 'cifar10_52000:53000', 'alg name': 'Baseline', 'fold': 6, 'lr': 0.006, 'accuracy': 0.635, 'tpr': 0.3308, 'fpr': 0.0793, 'precision': 0.3121, 'auc': 0.7427, 'pr_curve': 0.3546, 'train_time': 4.4651, 'infer_time': 0.2235}, {'dataset name': 'cifar10_52000:53000', 'alg name': 'Baseline', 'fold': 7, 'lr': 0.003, 'accuracy': 0.6667, 'tpr': 0.3366, 'fpr': 0.0743, 'precision': 0.3397, 'auc': 0.7676, 'pr_curve': 0.3526, 'train_time': 5.8248, 'infer_time': 0.2365}, {'dataset name': 'cifar10_52000:53000', 'alg name': 'Baseline', 'fold': 8, 'lr': 0.005, 'accuracy': 0.6653, 'tpr': 0.3081, 'fpr': 0.0788, 'precision': 0.337, 'auc': 0.7549, 'pr_curve': 0.368, 'train_time': 4.8282, 'infer_time': 0.2358}, {'dataset name': 'cifar10_52000:53000', 'alg name': 'Baseline', 'fold': 9, 'lr': 0.005, 'accuracy': 0.7476, 'tpr': 0.3383, 'fpr': 0.0773, 'precision': 0.3623, 'auc': 0.7673, 'pr_curve': 0.3808, 'train_time': 5.4199, 'infer_time': 0.2399}, {'dataset name': 'cifar10_52000:53000', 'alg name': 'Baseline', 'fold': 10, 'lr': 0.004, 'accuracy': 0.6497, 'tpr': 0.3207, 'fpr': 0.0767, 'precision': 0.3026, 'auc': 0.7579, 'pr_curve': 0.3252, 'train_time': 5.0846, 'infer_time': 0.2323}, {'dataset name': 'cifar10_52000:53000', 'alg name': 'Cyclic', 'fold': 1, 'lr': '0.001-0.02', 'accuracy': 0.5831, 'tpr': 0.4214, 'fpr': 0.0658, 'precision': 0.4717, 'auc': 0.8147, 'pr_curve': 0.3789, 'train_time': 5.1207, 'infer_time': 0.2261}, {'dataset name': 'cifar10_52000:53000', 'alg name': 'Cyclic', 'fold': 2, 'lr': '0.001-0.02', 'accuracy': 0.61, 'tpr': 0.3332, 'fpr': 0.0728, 'precision': 0.3678, 'auc': 0.7564, 'pr_curve': 0.3877, 'train_time': 5.1094, 'infer_time': 0.2337}, {'dataset name': 'cifar10_52000:53000', 'alg name': 'Cyclic', 'fold': 3, 'lr': '0.001-0.02', 'accuracy': 0.6187, 'tpr': 0.4044, 'fpr': 0.0686, 'precision': 0.5334, 'auc': 0.795, 'pr_curve': 0.3614, 'train_time': 5.2226, 'infer_time': 1.4139}, {'dataset name': 'cifar10_52000:53000', 'alg name': 'Cyclic', 'fold': 4, 'lr': '0.001-0.02', 'accuracy': 0.5744, 'tpr': 0.3254, 'fpr': 0.073, 'precision': 0.2878, 'auc': 0.7291, 'pr_curve': 0.3791, 'train_time': 4.436, 'infer_time': 0.239}, {'dataset name': 'cifar10_52000:53000', 'alg name': 'Cyclic', 'fold': 5, 'lr': '0.001-0.02', 'accuracy': 0.6122, 'tpr': 0.3085, 'fpr': 0.0793, 'precision': 0.2635, 'auc': 0.7632, 'pr_curve': 0.3468, 'train_time': 4.8521, 'infer_time': 1.409}, {'dataset name': 'cifar10_52000:53000', 'alg name': 'Cyclic', 'fold': 6, 'lr': '0.001-0.02', 'accuracy': 0.6915, 'tpr': 0.2714, 'fpr': 0.083, 'precision': 0.2725, 'auc': 0.7103, 'pr_curve': 0.3081, 'train_time': 5.8775, 'infer_time': 0.2307}, {'dataset name': 'cifar10_52000:53000', 'alg name': 'Cyclic', 'fold': 7, 'lr': '0.001-0.02', 'accuracy': 0.5991, 'tpr': 0.4043, 'fpr': 0.0704, 'precision': 0.4177, 'auc': 0.7857, 'pr_curve': 0.397, 'train_time': 4.4407, 'infer_time': 1.765}, {'dataset name': 'cifar10_52000:53000', 'alg name': 'Cyclic', 'fold': 8, 'lr': '0.001-0.02', 'accuracy': 0.6661, 'tpr': 0.3531, 'fpr': 0.0753, 'precision': 0.3589, 'auc': 0.7441, 'pr_curve': 0.3451, 'train_time': 5.4928, 'infer_time': 0.236}, {'dataset name': 'cifar10_52000:53000', 'alg name': 'Cyclic', 'fold': 9, 'lr': '0.001-0.02', 'accuracy': 0.6593, 'tpr': 0.3701, 'fpr': 0.075, 'precision': 0.4334, 'auc': 0.7635, 'pr_curve': 0.3985, 'train_time': 5.1139, 'infer_time': 1.0881}, {'dataset name': 'cifar10_52000:53000', 'alg name': 'Cyclic', 'fold': 10, 'lr': '0.001-0.02', 'accuracy': 0.6369, 'tpr': 0.3146, 'fpr': 0.0778, 'precision': 0.3005, 'auc': 0.7337, 'pr_curve': 0.3163, 'train_time': 4.7408, 'infer_time': 1.4354}, {'dataset name': 'cifar10_52000:53000', 'alg name': 'Ipmroved', 'fold': 1, 'lr': '0.001-0.05', 'accuracy': 0.5364, 'tpr': 0.3217, 'fpr': 0.0785, 'precision': 0.4257, 'auc': 0.7733, 'pr_curve': 0.3476, 'train_time': 4.4817, 'infer_time': 2.1108}, {'dataset name': 'cifar10_52000:53000', 'alg name': 'Ipmroved', 'fold': 2, 'lr': '0.001-0.05', 'accuracy': 0.5784, 'tpr': 0.254, 'fpr': 0.0831, 'precision': 0.4001, 'auc': 0.7398, 'pr_curve': 0.3441, 'train_time': 4.4148, 'infer_time': 1.7592}, {'dataset name': 'cifar10_52000:53000', 'alg name': 'Ipmroved', 'fold': 3, 'lr': '0.001-0.05', 'accuracy': 0.5293, 'tpr': 0.3516, 'fpr': 0.0736, 'precision': 0.3797, 'auc': 0.7846, 'pr_curve': 0.3753, 'train_time': 4.0153, 'infer_time': 0.2296}, {'dataset name': 'cifar10_52000:53000', 'alg name': 'Ipmroved', 'fold': 4, 'lr': '0.001-0.021', 'accuracy': 0.6601, 'tpr': 0.2836, 'fpr': 0.0775, 'precision': 0.3887, 'auc': 0.7607, 'pr_curve': 0.3807, 'train_time': 4.7843, 'infer_time': 1.4189}, {'dataset name': 'cifar10_52000:53000', 'alg name': 'Ipmroved', 'fold': 5, 'lr': '0.001-0.05', 'accuracy': 0.5562, 'tpr': 0.3327, 'fpr': 0.0737, 'precision': 0.3987, 'auc': 0.7464, 'pr_curve': 0.3327, 'train_time': 3.7051, 'infer_time': 2.4684}, {'dataset name': 'cifar10_52000:53000', 'alg name': 'Ipmroved', 'fold': 6, 'lr': '0.0-0.037', 'accuracy': 0.5211, 'tpr': 0.2693, 'fpr': 0.0833, 'precision': 0.3486, 'auc': 0.664, 'pr_curve': 0.2738, 'train_time': 4.445, 'infer_time': 1.7396}, {'dataset name': 'cifar10_52000:53000', 'alg name': 'Ipmroved', 'fold': 7, 'lr': '0.0-0.05', 'accuracy': 0.5992, 'tpr': 0.2994, 'fpr': 0.0807, 'precision': 0.3938, 'auc': 0.7525, 'pr_curve': 0.3656, 'train_time': 4.7383, 'infer_time': 0.2391}, {'dataset name': 'cifar10_52000:53000', 'alg name': 'Ipmroved', 'fold': 8, 'lr': '0.001-0.05', 'accuracy': 0.6608, 'tpr': 0.2951, 'fpr': 0.0799, 'precision': 0.2759, 'auc': 0.794, 'pr_curve': 0.4185, 'train_time': 4.4987, 'infer_time': 1.6952}, {'dataset name': 'cifar10_52000:53000', 'alg name': 'Ipmroved', 'fold': 9, 'lr': '0.002-0.021', 'accuracy': 0.5982, 'tpr': 0.3259, 'fpr': 0.08, 'precision': 0.4061, 'auc': 0.7517, 'pr_curve': 0.3608, 'train_time': 4.4215, 'infer_time': 0.2187}, {'dataset name': 'cifar10_52000:53000', 'alg name': 'Ipmroved', 'fold': 10, 'lr': '0.002-0.041', 'accuracy': 0.6449, 'tpr': 0.3589, 'fpr': 0.0674, 'precision': 0.3824, 'auc': 0.7569, 'pr_curve': 0.319, 'train_time': 4.4065, 'infer_time': 1.7662}]
d4_scores = [{'dataset name': 'cifar10_53000:54000', 'alg name': 'Baseline', 'fold': 1, 'lr': 0.004, 'accuracy': 0.7295, 'tpr': 0.3602, 'fpr': 0.0709, 'precision': 0.362, 'auc': 0.7634, 'pr_curve': 0.3536, 'train_time': 5.416, 'infer_time': 0.2497}, {'dataset name': 'cifar10_53000:54000', 'alg name': 'Baseline', 'fold': 2, 'lr': 0.004, 'accuracy': 0.7012, 'tpr': 0.3463, 'fpr': 0.064, 'precision': 0.3926, 'auc': 0.805, 'pr_curve': 0.3858, 'train_time': 5.1611, 'infer_time': 1.0159}, {'dataset name': 'cifar10_53000:54000', 'alg name': 'Baseline', 'fold': 3, 'lr': 0.004, 'accuracy': 0.6472, 'tpr': 0.3276, 'fpr': 0.0783, 'precision': 0.354, 'auc': 0.7593, 'pr_curve': 0.3428, 'train_time': 4.8118, 'infer_time': 0.2229}, {'dataset name': 'cifar10_53000:54000', 'alg name': 'Baseline', 'fold': 4, 'lr': 0.003, 'accuracy': 0.7079, 'tpr': 0.3392, 'fpr': 0.0741, 'precision': 0.2975, 'auc': 0.7695, 'pr_curve': 0.3674, 'train_time': 6.2576, 'infer_time': 5.0409}, {'dataset name': 'cifar10_53000:54000', 'alg name': 'Baseline', 'fold': 5, 'lr': 0.006, 'accuracy': 0.6392, 'tpr': 0.3438, 'fpr': 0.0778, 'precision': 0.3199, 'auc': 0.7381, 'pr_curve': 0.3602, 'train_time': 5.1735, 'infer_time': 1.0379}, {'dataset name': 'cifar10_53000:54000', 'alg name': 'Baseline', 'fold': 6, 'lr': 0.006, 'accuracy': 0.7464, 'tpr': 0.3446, 'fpr': 0.0728, 'precision': 0.3353, 'auc': 0.7709, 'pr_curve': 0.3661, 'train_time': 6.6709, 'infer_time': 4.6678}, {'dataset name': 'cifar10_53000:54000', 'alg name': 'Baseline', 'fold': 7, 'lr': 0.003, 'accuracy': 0.6845, 'tpr': 0.3192, 'fpr': 0.0785, 'precision': 0.3351, 'auc': 0.7426, 'pr_curve': 0.3276, 'train_time': 5.619, 'infer_time': 0.5664}, {'dataset name': 'cifar10_53000:54000', 'alg name': 'Baseline', 'fold': 8, 'lr': 0.007, 'accuracy': 0.6945, 'tpr': 0.2724, 'fpr': 0.0803, 'precision': 0.3017, 'auc': 0.7388, 'pr_curve': 0.323, 'train_time': 4.4857, 'infer_time': 0.2224}, {'dataset name': 'cifar10_53000:54000', 'alg name': 'Baseline', 'fold': 9, 'lr': 0.002, 'accuracy': 0.5973, 'tpr': 0.2246, 'fpr': 0.0817, 'precision': 0.213, 'auc': 0.7457, 'pr_curve': 0.3283, 'train_time': 6.041, 'infer_time': 0.2749}, {'dataset name': 'cifar10_53000:54000', 'alg name': 'Baseline', 'fold': 10, 'lr': 0.004, 'accuracy': 0.7076, 'tpr': 0.3272, 'fpr': 0.0723, 'precision': 0.4216, 'auc': 0.791, 'pr_curve': 0.4049, 'train_time': 5.2459, 'infer_time': 0.9431}, {'dataset name': 'cifar10_53000:54000', 'alg name': 'Cyclic', 'fold': 1, 'lr': '0.001-0.02', 'accuracy': 0.6111, 'tpr': 0.2826, 'fpr': 0.0805, 'precision': 0.3182, 'auc': 0.7654, 'pr_curve': 0.3504, 'train_time': 5.1933, 'infer_time': 0.989}, {'dataset name': 'cifar10_53000:54000', 'alg name': 'Cyclic', 'fold': 2, 'lr': '0.001-0.02', 'accuracy': 0.4944, 'tpr': 0.4021, 'fpr': 0.0618, 'precision': 0.4435, 'auc': 0.7935, 'pr_curve': 0.4182, 'train_time': 4.1459, 'infer_time': 2.0388}, {'dataset name': 'cifar10_53000:54000', 'alg name': 'Cyclic', 'fold': 3, 'lr': '0.001-0.02', 'accuracy': 0.5142, 'tpr': 0.2451, 'fpr': 0.0822, 'precision': 0.1734, 'auc': 0.7788, 'pr_curve': 0.3334, 'train_time': 4.5177, 'infer_time': 0.2434}, {'dataset name': 'cifar10_53000:54000', 'alg name': 'Cyclic', 'fold': 4, 'lr': '0.001-0.02', 'accuracy': 0.6446, 'tpr': 0.3417, 'fpr': 0.0732, 'precision': 0.3244, 'auc': 0.7384, 'pr_curve': 0.3205, 'train_time': 5.578, 'infer_time': 0.6178}, {'dataset name': 'cifar10_53000:54000', 'alg name': 'Cyclic', 'fold': 5, 'lr': '0.001-0.02', 'accuracy': 0.5867, 'tpr': 0.4067, 'fpr': 0.0695, 'precision': 0.3514, 'auc': 0.7641, 'pr_curve': 0.4069, 'train_time': 4.9586, 'infer_time': 1.6967}, {'dataset name': 'cifar10_53000:54000', 'alg name': 'Cyclic', 'fold': 6, 'lr': '0.001-0.02', 'accuracy': 0.6121, 'tpr': 0.3361, 'fpr': 0.0709, 'precision': 0.377, 'auc': 0.7419, 'pr_curve': 0.3597, 'train_time': 4.8439, 'infer_time': 0.2324}, {'dataset name': 'cifar10_53000:54000', 'alg name': 'Cyclic', 'fold': 7, 'lr': '0.001-0.02', 'accuracy': 0.5751, 'tpr': 0.4368, 'fpr': 0.0709, 'precision': 0.359, 'auc': 0.7529, 'pr_curve': 0.3704, 'train_time': 4.1244, 'infer_time': 0.24}, {'dataset name': 'cifar10_53000:54000', 'alg name': 'Cyclic', 'fold': 8, 'lr': '0.001-0.02', 'accuracy': 0.6164, 'tpr': 0.3239, 'fpr': 0.0769, 'precision': 0.3861, 'auc': 0.7774, 'pr_curve': 0.4528, 'train_time': 4.9104, 'infer_time': 0.2499}, {'dataset name': 'cifar10_53000:54000', 'alg name': 'Cyclic', 'fold': 9, 'lr': '0.001-0.02', 'accuracy': 0.616, 'tpr': 0.207, 'fpr': 0.0838, 'precision': 0.19, 'auc': 0.753, 'pr_curve': 0.3281, 'train_time': 4.8885, 'infer_time': 0.2321}, {'dataset name': 'cifar10_53000:54000', 'alg name': 'Cyclic', 'fold': 10, 'lr': '0.001-0.02', 'accuracy': 0.6363, 'tpr': 0.2361, 'fpr': 0.0812, 'precision': 0.2053, 'auc': 0.7437, 'pr_curve': 0.2904, 'train_time': 5.2438, 'infer_time': 0.9473}, {'dataset name': 'cifar10_53000:54000', 'alg name': 'Ipmroved', 'fold': 1, 'lr': '0.001-0.05', 'accuracy': 0.5894, 'tpr': 0.2568, 'fpr': 0.0827, 'precision': 0.3459, 'auc': 0.7795, 'pr_curve': 0.4094, 'train_time': 3.8063, 'infer_time': 0.24}, {'dataset name': 'cifar10_53000:54000', 'alg name': 'Ipmroved', 'fold': 2, 'lr': '0.002-0.05', 'accuracy': 0.6076, 'tpr': 0.3347, 'fpr': 0.0764, 'precision': 0.3524, 'auc': 0.7382, 'pr_curve': 0.3435, 'train_time': 4.8547, 'infer_time': 1.7074}, {'dataset name': 'cifar10_53000:54000', 'alg name': 'Ipmroved', 'fold': 3, 'lr': '0.001-0.05', 'accuracy': 0.6457, 'tpr': 0.3329, 'fpr': 0.0775, 'precision': 0.2967, 'auc': 0.7533, 'pr_curve': 0.3512, 'train_time': 5.2569, 'infer_time': 0.2326}, {'dataset name': 'cifar10_53000:54000', 'alg name': 'Ipmroved', 'fold': 4, 'lr': '0.002-0.05', 'accuracy': 0.6167, 'tpr': 0.2997, 'fpr': 0.0809, 'precision': 0.3857, 'auc': 0.7437, 'pr_curve': 0.3346, 'train_time': 4.5877, 'infer_time': 0.2616}, {'dataset name': 'cifar10_53000:54000', 'alg name': 'Ipmroved', 'fold': 5, 'lr': '0.001-0.05', 'accuracy': 0.6619, 'tpr': 0.3313, 'fpr': 0.0777, 'precision': 0.374, 'auc': 0.7267, 'pr_curve': 0.3428, 'train_time': 4.8849, 'infer_time': 0.2431}, {'dataset name': 'cifar10_53000:54000', 'alg name': 'Ipmroved', 'fold': 6, 'lr': '0.001-0.05', 'accuracy': 0.5823, 'tpr': 0.3555, 'fpr': 0.0733, 'precision': 0.4032, 'auc': 0.7699, 'pr_curve': 0.345, 'train_time': 4.4872, 'infer_time': 1.705}, {'dataset name': 'cifar10_53000:54000', 'alg name': 'Ipmroved', 'fold': 7, 'lr': '0.001-0.05', 'accuracy': 0.5934, 'tpr': 0.282, 'fpr': 0.0793, 'precision': 0.2543, 'auc': 0.7061, 'pr_curve': 0.2531, 'train_time': 4.9882, 'infer_time': 1.6536}, {'dataset name': 'cifar10_53000:54000', 'alg name': 'Ipmroved', 'fold': 8, 'lr': '0.0-0.05', 'accuracy': 0.4516, 'tpr': 0.2448, 'fpr': 0.085, 'precision': 0.2675, 'auc': 0.7124, 'pr_curve': 0.2757, 'train_time': 4.349, 'infer_time': 1.8739}, {'dataset name': 'cifar10_53000:54000', 'alg name': 'Ipmroved', 'fold': 9, 'lr': '0.002-0.05', 'accuracy': 0.5972, 'tpr': 0.3722, 'fpr': 0.065, 'precision': 0.4084, 'auc': 0.7466, 'pr_curve': 0.3228, 'train_time': 4.1416, 'infer_time': 2.0514}, {'dataset name': 'cifar10_53000:54000', 'alg name': 'Ipmroved', 'fold': 10, 'lr': '0.001-0.05', 'accuracy': 0.5867, 'tpr': 0.3242, 'fpr': 0.0737, 'precision': 0.4082, 'auc': 0.7472, 'pr_curve': 0.3114, 'train_time': 4.124, 'infer_time': 2.0458}]
d5_scores = [{'dataset name': 'cifar10_54000:55000', 'alg name': 'Baseline', 'fold': 1, 'lr': 0.01, 'accuracy': 0.6949, 'tpr': 0.2929, 'fpr': 0.08, 'precision': 0.3174, 'auc': 0.6919, 'pr_curve': 0.2669, 'train_time': 5.9509, 'infer_time': 5.4997}, {'dataset name': 'cifar10_54000:55000', 'alg name': 'Baseline', 'fold': 2, 'lr': 0.004, 'accuracy': 0.7103, 'tpr': 0.2939, 'fpr': 0.0826, 'precision': 0.2612, 'auc': 0.747, 'pr_curve': 0.3625, 'train_time': 4.8421, 'infer_time': 0.2247}, {'dataset name': 'cifar10_54000:55000', 'alg name': 'Baseline', 'fold': 3, 'lr': 0.01, 'accuracy': 0.672, 'tpr': 0.3125, 'fpr': 0.0753, 'precision': 0.4054, 'auc': 0.7574, 'pr_curve': 0.352, 'train_time': 4.782, 'infer_time': 0.226}, {'dataset name': 'cifar10_54000:55000', 'alg name': 'Baseline', 'fold': 4, 'lr': 0.004, 'accuracy': 0.652, 'tpr': 0.3117, 'fpr': 0.0765, 'precision': 0.3836, 'auc': 0.7641, 'pr_curve': 0.3955, 'train_time': 4.8048, 'infer_time': 1.3247}, {'dataset name': 'cifar10_54000:55000', 'alg name': 'Baseline', 'fold': 5, 'lr': 0.004, 'accuracy': 0.7323, 'tpr': 0.4103, 'fpr': 0.0711, 'precision': 0.4285, 'auc': 0.7817, 'pr_curve': 0.4428, 'train_time': 5.2644, 'infer_time': 0.9534}, {'dataset name': 'cifar10_54000:55000', 'alg name': 'Baseline', 'fold': 6, 'lr': 0.005, 'accuracy': 0.6937, 'tpr': 0.3212, 'fpr': 0.0762, 'precision': 0.339, 'auc': 0.75, 'pr_curve': 0.3012, 'train_time': 5.2423, 'infer_time': 0.9245}, {'dataset name': 'cifar10_54000:55000', 'alg name': 'Baseline', 'fold': 7, 'lr': 0.005, 'accuracy': 0.8041, 'tpr': 0.3387, 'fpr': 0.0741, 'precision': 0.3453, 'auc': 0.7404, 'pr_curve': 0.3659, 'train_time': 7.1003, 'infer_time': 0.2243}, {'dataset name': 'cifar10_54000:55000', 'alg name': 'Baseline', 'fold': 8, 'lr': 0.009000000000000001, 'accuracy': 0.7564, 'tpr': 0.444, 'fpr': 0.0636, 'precision': 0.3705, 'auc': 0.8135, 'pr_curve': 0.4409, 'train_time': 5.5854, 'infer_time': 0.2376}, {'dataset name': 'cifar10_54000:55000', 'alg name': 'Baseline', 'fold': 9, 'lr': 0.005, 'accuracy': 0.7216, 'tpr': 0.2844, 'fpr': 0.0784, 'precision': 0.4356, 'auc': 0.7816, 'pr_curve': 0.3458, 'train_time': 5.2578, 'infer_time': 0.2215}, {'dataset name': 'cifar10_54000:55000', 'alg name': 'Baseline', 'fold': 10, 'lr': 0.011, 'accuracy': 0.6695, 'tpr': 0.3656, 'fpr': 0.0723, 'precision': 0.3708, 'auc': 0.7755, 'pr_curve': 0.3848, 'train_time': 6.1129, 'infer_time': 5.2405}, {'dataset name': 'cifar10_54000:55000', 'alg name': 'Cyclic', 'fold': 1, 'lr': '0.001-0.02', 'accuracy': 0.6955, 'tpr': 0.3085, 'fpr': 0.076, 'precision': 0.2958, 'auc': 0.6942, 'pr_curve': 0.2826, 'train_time': 5.6579, 'infer_time': 0.2235}, {'dataset name': 'cifar10_54000:55000', 'alg name': 'Cyclic', 'fold': 2, 'lr': '0.001-0.02', 'accuracy': 0.5233, 'tpr': 0.2974, 'fpr': 0.0829, 'precision': 0.2813, 'auc': 0.7086, 'pr_curve': 0.2853, 'train_time': 4.4959, 'infer_time': 1.666}, {'dataset name': 'cifar10_54000:55000', 'alg name': 'Cyclic', 'fold': 3, 'lr': '0.001-0.02', 'accuracy': 0.5431, 'tpr': 0.3437, 'fpr': 0.0715, 'precision': 0.3612, 'auc': 0.785, 'pr_curve': 0.3588, 'train_time': 4.5054, 'infer_time': 0.2293}, {'dataset name': 'cifar10_54000:55000', 'alg name': 'Cyclic', 'fold': 4, 'lr': '0.001-0.02', 'accuracy': 0.6289, 'tpr': 0.4004, 'fpr': 0.064, 'precision': 0.4002, 'auc': 0.8178, 'pr_curve': 0.4413, 'train_time': 5.7001, 'infer_time': 0.9208}, {'dataset name': 'cifar10_54000:55000', 'alg name': 'Cyclic', 'fold': 5, 'lr': '0.001-0.02', 'accuracy': 0.6021, 'tpr': 0.3806, 'fpr': 0.0686, 'precision': 0.3937, 'auc': 0.7896, 'pr_curve': 0.4176, 'train_time': 5.3487, 'infer_time': 0.2351}, {'dataset name': 'cifar10_54000:55000', 'alg name': 'Cyclic', 'fold': 6, 'lr': '0.001-0.02', 'accuracy': 0.5962, 'tpr': 0.3618, 'fpr': 0.0706, 'precision': 0.4297, 'auc': 0.7634, 'pr_curve': 0.3698, 'train_time': 4.5764, 'infer_time': 0.2384}, {'dataset name': 'cifar10_54000:55000', 'alg name': 'Cyclic', 'fold': 7, 'lr': '0.001-0.02', 'accuracy': 0.6393, 'tpr': 0.3396, 'fpr': 0.0742, 'precision': 0.3167, 'auc': 0.7777, 'pr_curve': 0.3673, 'train_time': 5.263, 'infer_time': 0.9397}, {'dataset name': 'cifar10_54000:55000', 'alg name': 'Cyclic', 'fold': 8, 'lr': '0.001-0.02', 'accuracy': 0.6309, 'tpr': 0.3038, 'fpr': 0.0761, 'precision': 0.3268, 'auc': 0.7594, 'pr_curve': 0.3378, 'train_time': 5.3312, 'infer_time': 0.855}, {'dataset name': 'cifar10_54000:55000', 'alg name': 'Cyclic', 'fold': 9, 'lr': '0.001-0.02', 'accuracy': 0.6546, 'tpr': 0.4763, 'fpr': 0.0582, 'precision': 0.4793, 'auc': 0.8154, 'pr_curve': 0.443, 'train_time': 5.3172, 'infer_time': 0.2531}, {'dataset name': 'cifar10_54000:55000', 'alg name': 'Cyclic', 'fold': 10, 'lr': '0.001-0.02', 'accuracy': 0.5462, 'tpr': 0.3266, 'fpr': 0.0785, 'precision': 0.322, 'auc': 0.787, 'pr_curve': 0.4242, 'train_time': 4.1344, 'infer_time': 2.0722}, {'dataset name': 'cifar10_54000:55000', 'alg name': 'Ipmroved', 'fold': 1, 'lr': '0.001-0.05', 'accuracy': 0.5414, 'tpr': 0.228, 'fpr': 0.0895, 'precision': 0.2073, 'auc': 0.7197, 'pr_curve': 0.2994, 'train_time': 3.8441, 'infer_time': 0.2286}, {'dataset name': 'cifar10_54000:55000', 'alg name': 'Ipmroved', 'fold': 2, 'lr': '0.002-0.05', 'accuracy': 0.5758, 'tpr': 0.2751, 'fpr': 0.0818, 'precision': 0.3209, 'auc': 0.7379, 'pr_curve': 0.2963, 'train_time': 4.1136, 'infer_time': 0.2374}, {'dataset name': 'cifar10_54000:55000', 'alg name': 'Ipmroved', 'fold': 3, 'lr': '0.001-0.05', 'accuracy': 0.6057, 'tpr': 0.3141, 'fpr': 0.0728, 'precision': 0.3941, 'auc': 0.7694, 'pr_curve': 0.3536, 'train_time': 4.121, 'infer_time': 2.0243}, {'dataset name': 'cifar10_54000:55000', 'alg name': 'Ipmroved', 'fold': 4, 'lr': '0.0-0.05', 'accuracy': 0.5913, 'tpr': 0.2447, 'fpr': 0.0888, 'precision': 0.2954, 'auc': 0.7251, 'pr_curve': 0.3016, 'train_time': 4.5349, 'infer_time': 1.6748}, {'dataset name': 'cifar10_54000:55000', 'alg name': 'Ipmroved', 'fold': 5, 'lr': '0.001-0.05', 'accuracy': 0.6102, 'tpr': 0.2873, 'fpr': 0.0801, 'precision': 0.3751, 'auc': 0.7399, 'pr_curve': 0.3763, 'train_time': 4.5211, 'infer_time': 1.6846}, {'dataset name': 'cifar10_54000:55000', 'alg name': 'Ipmroved', 'fold': 6, 'lr': '0.001-0.05', 'accuracy': 0.5198, 'tpr': 0.281, 'fpr': 0.0834, 'precision': 0.2881, 'auc': 0.7151, 'pr_curve': 0.2937, 'train_time': 4.6395, 'infer_time': 0.229}, {'dataset name': 'cifar10_54000:55000', 'alg name': 'Ipmroved', 'fold': 7, 'lr': '0.001-0.05', 'accuracy': 0.5514, 'tpr': 0.2251, 'fpr': 0.0874, 'precision': 0.3908, 'auc': 0.7075, 'pr_curve': 0.2996, 'train_time': 4.2012, 'infer_time': 0.2364}, {'dataset name': 'cifar10_54000:55000', 'alg name': 'Ipmroved', 'fold': 8, 'lr': '0.001-0.05', 'accuracy': 0.5431, 'tpr': 0.358, 'fpr': 0.0761, 'precision': 0.3189, 'auc': 0.7445, 'pr_curve': 0.3259, 'train_time': 4.9336, 'infer_time': 1.2322}, {'dataset name': 'cifar10_54000:55000', 'alg name': 'Ipmroved', 'fold': 9, 'lr': '0.001-0.05', 'accuracy': 0.6186, 'tpr': 0.4124, 'fpr': 0.0642, 'precision': 0.3783, 'auc': 0.7806, 'pr_curve': 0.3804, 'train_time': 5.2502, 'infer_time': 0.9214}, {'dataset name': 'cifar10_54000:55000', 'alg name': 'Ipmroved', 'fold': 10, 'lr': '0.001-0.05', 'accuracy': 0.5388, 'tpr': 0.2596, 'fpr': 0.0798, 'precision': 0.2347, 'auc': 0.7309, 'pr_curve': 0.3257, 'train_time': 4.1265, 'infer_time': 0.2384}]
d6_scores = [{'dataset name': 'cifar10_55000:56000', 'alg name': 'Baseline', 'fold': 1, 'lr': 0.004, 'accuracy': 0.6498, 'tpr': 0.3508, 'fpr': 0.0739, 'precision': 0.5066, 'auc': 0.8012, 'pr_curve': 0.4163, 'train_time': 4.504, 'infer_time': 1.6472}, {'dataset name': 'cifar10_55000:56000', 'alg name': 'Baseline', 'fold': 2, 'lr': 0.005, 'accuracy': 0.7402, 'tpr': 0.3464, 'fpr': 0.0687, 'precision': 0.3671, 'auc': 0.7882, 'pr_curve': 0.3955, 'train_time': 6.3866, 'infer_time': 0.25}, {'dataset name': 'cifar10_55000:56000', 'alg name': 'Baseline', 'fold': 3, 'lr': 0.004, 'accuracy': 0.7248, 'tpr': 0.3101, 'fpr': 0.0771, 'precision': 0.3489, 'auc': 0.74, 'pr_curve': 0.33, 'train_time': 6.4358, 'infer_time': 4.8329}, {'dataset name': 'cifar10_55000:56000', 'alg name': 'Baseline', 'fold': 4, 'lr': 0.003, 'accuracy': 0.695, 'tpr': 0.3693, 'fpr': 0.0741, 'precision': 0.3576, 'auc': 0.766, 'pr_curve': 0.3687, 'train_time': 5.6953, 'infer_time': 0.2202}, {'dataset name': 'cifar10_55000:56000', 'alg name': 'Baseline', 'fold': 5, 'lr': 0.004, 'accuracy': 0.7779, 'tpr': 0.3197, 'fpr': 0.0753, 'precision': 0.3669, 'auc': 0.7677, 'pr_curve': 0.3716, 'train_time': 6.5147, 'infer_time': 4.8282}, {'dataset name': 'cifar10_55000:56000', 'alg name': 'Baseline', 'fold': 6, 'lr': 0.006, 'accuracy': 0.6409, 'tpr': 0.3459, 'fpr': 0.0794, 'precision': 0.2909, 'auc': 0.7513, 'pr_curve': 0.382, 'train_time': 4.5285, 'infer_time': 0.2337}, {'dataset name': 'cifar10_55000:56000', 'alg name': 'Baseline', 'fold': 7, 'lr': 0.008, 'accuracy': 0.6669, 'tpr': 0.2094, 'fpr': 0.0836, 'precision': 0.2853, 'auc': 0.7046, 'pr_curve': 0.2816, 'train_time': 4.56, 'infer_time': 1.6012}, {'dataset name': 'cifar10_55000:56000', 'alg name': 'Baseline', 'fold': 8, 'lr': 0.005, 'accuracy': 0.6326, 'tpr': 0.2876, 'fpr': 0.0815, 'precision': 0.2355, 'auc': 0.725, 'pr_curve': 0.3063, 'train_time': 6.0366, 'infer_time': 0.2303}, {'dataset name': 'cifar10_55000:56000', 'alg name': 'Baseline', 'fold': 9, 'lr': 0.012, 'accuracy': 0.5596, 'tpr': 0.1753, 'fpr': 0.0912, 'precision': 0.1146, 'auc': 0.6634, 'pr_curve': 0.255, 'train_time': 3.789, 'infer_time': 0.2318}, {'dataset name': 'cifar10_55000:56000', 'alg name': 'Baseline', 'fold': 10, 'lr': 0.005, 'accuracy': 0.7034, 'tpr': 0.336, 'fpr': 0.0753, 'precision': 0.3674, 'auc': 0.7593, 'pr_curve': 0.3863, 'train_time': 5.7518, 'infer_time': 0.4132}, {'dataset name': 'cifar10_55000:56000', 'alg name': 'Cyclic', 'fold': 1, 'lr': '0.001-0.02', 'accuracy': 0.5889, 'tpr': 0.2999, 'fpr': 0.077, 'precision': 0.3883, 'auc': 0.7758, 'pr_curve': 0.3534, 'train_time': 4.5587, 'infer_time': 1.6036}, {'dataset name': 'cifar10_55000:56000', 'alg name': 'Cyclic', 'fold': 2, 'lr': '0.001-0.02', 'accuracy': 0.5761, 'tpr': 0.4021, 'fpr': 0.0693, 'precision': 0.3847, 'auc': 0.7648, 'pr_curve': 0.3446, 'train_time': 4.9441, 'infer_time': 1.2446}, {'dataset name': 'cifar10_55000:56000', 'alg name': 'Cyclic', 'fold': 3, 'lr': '0.001-0.02', 'accuracy': 0.534, 'tpr': 0.2247, 'fpr': 0.0897, 'precision': 0.1752, 'auc': 0.7236, 'pr_curve': 0.2745, 'train_time': 4.6351, 'infer_time': 0.2397}, {'dataset name': 'cifar10_55000:56000', 'alg name': 'Cyclic', 'fold': 4, 'lr': '0.001-0.02', 'accuracy': 0.6047, 'tpr': 0.3557, 'fpr': 0.0724, 'precision': 0.3891, 'auc': 0.7588, 'pr_curve': 0.3739, 'train_time': 5.3396, 'infer_time': 0.2373}, {'dataset name': 'cifar10_55000:56000', 'alg name': 'Cyclic', 'fold': 5, 'lr': '0.001-0.02', 'accuracy': 0.3771, 'tpr': 0.1615, 'fpr': 0.0946, 'precision': 0.1458, 'auc': 0.5622, 'pr_curve': 0.1595, 'train_time': 3.4858, 'infer_time': 0.53}, {'dataset name': 'cifar10_55000:56000', 'alg name': 'Cyclic', 'fold': 6, 'lr': '0.001-0.02', 'accuracy': 0.54, 'tpr': 0.3164, 'fpr': 0.0785, 'precision': 0.3955, 'auc': 0.7689, 'pr_curve': 0.3693, 'train_time': 4.2211, 'infer_time': 1.9728}, {'dataset name': 'cifar10_55000:56000', 'alg name': 'Cyclic', 'fold': 7, 'lr': '0.001-0.02', 'accuracy': 0.5953, 'tpr': 0.3769, 'fpr': 0.0713, 'precision': 0.4988, 'auc': 0.7586, 'pr_curve': 0.3873, 'train_time': 5.3644, 'infer_time': 0.2345}, {'dataset name': 'cifar10_55000:56000', 'alg name': 'Cyclic', 'fold': 8, 'lr': '0.001-0.02', 'accuracy': 0.5354, 'tpr': 0.3032, 'fpr': 0.0796, 'precision': 0.3052, 'auc': 0.7488, 'pr_curve': 0.3511, 'train_time': 4.9651, 'infer_time': 1.236}, {'dataset name': 'cifar10_55000:56000', 'alg name': 'Cyclic', 'fold': 9, 'lr': '0.001-0.02', 'accuracy': 0.6805, 'tpr': 0.3032, 'fpr': 0.0769, 'precision': 0.4023, 'auc': 0.7504, 'pr_curve': 0.3962, 'train_time': 5.3281, 'infer_time': 0.8317}, {'dataset name': 'cifar10_55000:56000', 'alg name': 'Cyclic', 'fold': 10, 'lr': '0.001-0.02', 'accuracy': 0.6676, 'tpr': 0.3657, 'fpr': 0.0758, 'precision': 0.3303, 'auc': 0.7461, 'pr_curve': 0.3515, 'train_time': 5.3418, 'infer_time': 0.2389}, {'dataset name': 'cifar10_55000:56000', 'alg name': 'Ipmroved', 'fold': 1, 'lr': '0.002-0.05', 'accuracy': 0.5747, 'tpr': 0.2448, 'fpr': 0.0841, 'precision': 0.3219, 'auc': 0.7223, 'pr_curve': 0.3249, 'train_time': 4.5838, 'infer_time': 0.2238}, {'dataset name': 'cifar10_55000:56000', 'alg name': 'Ipmroved', 'fold': 2, 'lr': '0.002-0.05', 'accuracy': 0.5708, 'tpr': 0.2382, 'fpr': 0.085, 'precision': 0.2612, 'auc': 0.7107, 'pr_curve': 0.2793, 'train_time': 4.6403, 'infer_time': 0.2463}, {'dataset name': 'cifar10_55000:56000', 'alg name': 'Ipmroved', 'fold': 3, 'lr': '0.002-0.05', 'accuracy': 0.6331, 'tpr': 0.2729, 'fpr': 0.0799, 'precision': 0.2625, 'auc': 0.7368, 'pr_curve': 0.3238, 'train_time': 5.7909, 'infer_time': 0.4049}, {'dataset name': 'cifar10_55000:56000', 'alg name': 'Ipmroved', 'fold': 4, 'lr': '0.001-0.05', 'accuracy': 0.6488, 'tpr': 0.2054, 'fpr': 0.0896, 'precision': 0.3648, 'auc': 0.7218, 'pr_curve': 0.3128, 'train_time': 4.9706, 'infer_time': 1.1884}, {'dataset name': 'cifar10_55000:56000', 'alg name': 'Ipmroved', 'fold': 5, 'lr': '0.002-0.05', 'accuracy': 0.6508, 'tpr': 0.2436, 'fpr': 0.0836, 'precision': 0.3461, 'auc': 0.7343, 'pr_curve': 0.3245, 'train_time': 4.6509, 'infer_time': 0.2178}, {'dataset name': 'cifar10_55000:56000', 'alg name': 'Ipmroved', 'fold': 6, 'lr': '0.001-0.05', 'accuracy': 0.4839, 'tpr': 0.0886, 'fpr': 0.0987, 'precision': 0.1977, 'auc': 0.6606, 'pr_curve': 0.2381, 'train_time': 3.0753, 'infer_time': 0.5224}, {'dataset name': 'cifar10_55000:56000', 'alg name': 'Ipmroved', 'fold': 7, 'lr': '0.001-0.021', 'accuracy': 0.6097, 'tpr': 0.3003, 'fpr': 0.0814, 'precision': 0.2426, 'auc': 0.7192, 'pr_curve': 0.292, 'train_time': 5.0486, 'infer_time': 1.1475}, {'dataset name': 'cifar10_55000:56000', 'alg name': 'Ipmroved', 'fold': 8, 'lr': '0.001-0.05', 'accuracy': 0.6569, 'tpr': 0.309, 'fpr': 0.0799, 'precision': 0.3259, 'auc': 0.7449, 'pr_curve': 0.3602, 'train_time': 5.4295, 'infer_time': 0.791}, {'dataset name': 'cifar10_55000:56000', 'alg name': 'Ipmroved', 'fold': 9, 'lr': '0.001-0.05', 'accuracy': 0.5719, 'tpr': 0.2251, 'fpr': 0.086, 'precision': 0.4082, 'auc': 0.6745, 'pr_curve': 0.3108, 'train_time': 4.2064, 'infer_time': 1.9561}, {'dataset name': 'cifar10_55000:56000', 'alg name': 'Ipmroved', 'fold': 10, 'lr': '0.001-0.05', 'accuracy': 0.5855, 'tpr': 0.3022, 'fpr': 0.0774, 'precision': 0.3487, 'auc': 0.7506, 'pr_curve': 0.3373, 'train_time': 4.6148, 'infer_time': 1.5108}]
d7_scores = [{'dataset name': 'cifar10_56000:57000', 'alg name': 'Baseline', 'fold': 1, 'lr': 0.01, 'accuracy': 0.7017, 'tpr': 0.3692, 'fpr': 0.0767, 'precision': 0.3605, 'auc': 0.7573, 'pr_curve': 0.3401, 'train_time': 1.4502, 'infer_time': 0.8063}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Baseline', 'fold': 2, 'lr': 0.005, 'accuracy': 0.6424, 'tpr': 0.2864, 'fpr': 0.0769, 'precision': 0.4338, 'auc': 0.7301, 'pr_curve': 0.3174, 'train_time': 1.5167, 'infer_time': 0.6269}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Baseline', 'fold': 3, 'lr': 0.004, 'accuracy': 0.7358, 'tpr': 0.3709, 'fpr': 0.0726, 'precision': 0.3136, 'auc': 0.8134, 'pr_curve': 0.465, 'train_time': 1.6853, 'infer_time': 0.4795}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Baseline', 'fold': 4, 'lr': 0.007, 'accuracy': 0.7055, 'tpr': 0.3315, 'fpr': 0.077, 'precision': 0.3032, 'auc': 0.7722, 'pr_curve': 0.3159, 'train_time': 1.4795, 'infer_time': 0.1646}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Baseline', 'fold': 5, 'lr': 0.006, 'accuracy': 0.7959, 'tpr': 0.3655, 'fpr': 0.0672, 'precision': 0.4731, 'auc': 0.7909, 'pr_curve': 0.3812, 'train_time': 1.6786, 'infer_time': 0.4497}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Baseline', 'fold': 6, 'lr': 0.01, 'accuracy': 0.747, 'tpr': 0.3889, 'fpr': 0.0714, 'precision': 0.3911, 'auc': 0.7942, 'pr_curve': 0.381, 'train_time': 1.6178, 'infer_time': 0.1802}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Baseline', 'fold': 7, 'lr': 0.011, 'accuracy': 0.3633, 'tpr': 0.2087, 'fpr': 0.0902, 'precision': 0.1681, 'auc': 0.6607, 'pr_curve': 0.2173, 'train_time': 1.2551, 'infer_time': 0.19}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Baseline', 'fold': 8, 'lr': 0.009000000000000001, 'accuracy': 0.7156, 'tpr': 0.3431, 'fpr': 0.0733, 'precision': 0.4184, 'auc': 0.7591, 'pr_curve': 0.3263, 'train_time': 1.5102, 'infer_time': 0.5926}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Baseline', 'fold': 9, 'lr': 0.007, 'accuracy': 0.6136, 'tpr': 0.2808, 'fpr': 0.0819, 'precision': 0.2981, 'auc': 0.7169, 'pr_curve': 0.3127, 'train_time': 1.4543, 'infer_time': 0.1776}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Baseline', 'fold': 10, 'lr': 0.007, 'accuracy': 0.6001, 'tpr': 0.3068, 'fpr': 0.0821, 'precision': 0.3068, 'auc': 0.6824, 'pr_curve': 0.2765, 'train_time': 1.2574, 'infer_time': 0.1804}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Cyclic', 'fold': 1, 'lr': '0.001-0.02', 'accuracy': 0.5913, 'tpr': 0.2345, 'fpr': 0.0893, 'precision': 0.2358, 'auc': 0.7262, 'pr_curve': 0.2805, 'train_time': 1.4322, 'infer_time': 0.6631}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Cyclic', 'fold': 2, 'lr': '0.001-0.02', 'accuracy': 0.5926, 'tpr': 0.3073, 'fpr': 0.0791, 'precision': 0.3939, 'auc': 0.7406, 'pr_curve': 0.3474, 'train_time': 1.4175, 'infer_time': 0.6956}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Cyclic', 'fold': 3, 'lr': '0.001-0.02', 'accuracy': 0.633, 'tpr': 0.3861, 'fpr': 0.0689, 'precision': 0.4704, 'auc': 0.7759, 'pr_curve': 0.3791, 'train_time': 1.4497, 'infer_time': 0.1898}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Cyclic', 'fold': 4, 'lr': '0.001-0.02', 'accuracy': 0.5431, 'tpr': 0.3035, 'fpr': 0.0789, 'precision': 0.3016, 'auc': 0.7794, 'pr_curve': 0.4071, 'train_time': 1.374, 'infer_time': 0.1771}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Cyclic', 'fold': 5, 'lr': '0.001-0.02', 'accuracy': 0.5203, 'tpr': 0.4278, 'fpr': 0.0653, 'precision': 0.4728, 'auc': 0.7902, 'pr_curve': 0.4152, 'train_time': 1.8005, 'infer_time': 0.6427}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Cyclic', 'fold': 6, 'lr': '0.001-0.02', 'accuracy': 0.5961, 'tpr': 0.2846, 'fpr': 0.0832, 'precision': 0.4406, 'auc': 0.7413, 'pr_curve': 0.3659, 'train_time': 1.4223, 'infer_time': 0.6702}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Cyclic', 'fold': 7, 'lr': '0.001-0.02', 'accuracy': 0.5996, 'tpr': 0.2166, 'fpr': 0.0883, 'precision': 0.2561, 'auc': 0.7278, 'pr_curve': 0.2694, 'train_time': 1.3972, 'infer_time': 0.6958}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Cyclic', 'fold': 8, 'lr': '0.001-0.02', 'accuracy': 0.5805, 'tpr': 0.2533, 'fpr': 0.0791, 'precision': 0.4264, 'auc': 0.7576, 'pr_curve': 0.3564, 'train_time': 1.3501, 'infer_time': 0.1839}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Cyclic', 'fold': 9, 'lr': '0.001-0.02', 'accuracy': 0.5535, 'tpr': 0.2896, 'fpr': 0.0791, 'precision': 0.3792, 'auc': 0.6944, 'pr_curve': 0.2927, 'train_time': 1.4052, 'infer_time': 0.6779}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Cyclic', 'fold': 10, 'lr': '0.001-0.02', 'accuracy': 0.608, 'tpr': 0.1932, 'fpr': 0.0919, 'precision': 0.1748, 'auc': 0.733, 'pr_curve': 0.3012, 'train_time': 1.4387, 'infer_time': 0.6691}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Ipmroved', 'fold': 1, 'lr': '0.001-0.05', 'accuracy': 0.5482, 'tpr': 0.2806, 'fpr': 0.0847, 'precision': 0.1966, 'auc': 0.7367, 'pr_curve': 0.2609, 'train_time': 1.2382, 'infer_time': 0.1759}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Ipmroved', 'fold': 2, 'lr': '0.002-0.05', 'accuracy': 0.6559, 'tpr': 0.2128, 'fpr': 0.0801, 'precision': 0.2172, 'auc': 0.7458, 'pr_curve': 0.3257, 'train_time': 1.6097, 'infer_time': 0.1825}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Ipmroved', 'fold': 3, 'lr': '0.001-0.05', 'accuracy': 0.5375, 'tpr': 0.2599, 'fpr': 0.081, 'precision': 0.2855, 'auc': 0.7209, 'pr_curve': 0.3301, 'train_time': 1.2295, 'infer_time': 0.2167}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Ipmroved', 'fold': 4, 'lr': '0.002-0.05', 'accuracy': 0.5679, 'tpr': 0.2672, 'fpr': 0.0828, 'precision': 0.3023, 'auc': 0.7576, 'pr_curve': 0.3349, 'train_time': 1.3997, 'infer_time': 0.2018}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Ipmroved', 'fold': 5, 'lr': '0.0-0.05', 'accuracy': 0.5654, 'tpr': 0.266, 'fpr': 0.0813, 'precision': 0.341, 'auc': 0.7483, 'pr_curve': 0.3268, 'train_time': 1.4787, 'infer_time': 0.1854}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Ipmroved', 'fold': 6, 'lr': '0.0-0.05', 'accuracy': 0.5269, 'tpr': 0.2411, 'fpr': 0.0848, 'precision': 0.3534, 'auc': 0.7119, 'pr_curve': 0.3047, 'train_time': 1.3442, 'infer_time': 0.1914}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Ipmroved', 'fold': 7, 'lr': '0.001-0.05', 'accuracy': 0.549, 'tpr': 0.3433, 'fpr': 0.072, 'precision': 0.3335, 'auc': 0.758, 'pr_curve': 0.3317, 'train_time': 1.4406, 'infer_time': 0.693}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Ipmroved', 'fold': 8, 'lr': '0.001-0.05', 'accuracy': 0.6051, 'tpr': 0.2938, 'fpr': 0.0786, 'precision': 0.4195, 'auc': 0.7602, 'pr_curve': 0.3613, 'train_time': 1.4795, 'infer_time': 0.6812}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Ipmroved', 'fold': 9, 'lr': '0.002-0.05', 'accuracy': 0.5992, 'tpr': 0.315, 'fpr': 0.0769, 'precision': 0.3543, 'auc': 0.7075, 'pr_curve': 0.2696, 'train_time': 1.4278, 'infer_time': 0.7343}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Ipmroved', 'fold': 10, 'lr': '0.001-0.05', 'accuracy': 0.5337, 'tpr': 0.1799, 'fpr': 0.0931, 'precision': 0.2203, 'auc': 0.6896, 'pr_curve': 0.2677, 'train_time': 1.382, 'infer_time': 0.1875}]
d8_scores = [{'dataset name': 'cifar10_57000:58000', 'alg name': 'Baseline', 'fold': 1, 'lr': 0.003, 'accuracy': 0.6779, 'tpr': 0.3383, 'fpr': 0.0731, 'precision': 0.3602, 'auc': 0.7405, 'pr_curve': 0.3009, 'train_time': 1.5906, 'infer_time': 0.4768}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Baseline', 'fold': 2, 'lr': 0.007, 'accuracy': 0.6072, 'tpr': 0.2814, 'fpr': 0.0861, 'precision': 0.2907, 'auc': 0.7556, 'pr_curve': 0.3082, 'train_time': 1.4454, 'infer_time': 0.7578}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Baseline', 'fold': 3, 'lr': 0.009000000000000001, 'accuracy': 0.6924, 'tpr': 0.3211, 'fpr': 0.0756, 'precision': 0.3722, 'auc': 0.776, 'pr_curve': 0.3955, 'train_time': 1.5817, 'infer_time': 0.5091}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Baseline', 'fold': 4, 'lr': 0.006, 'accuracy': 0.7043, 'tpr': 0.435, 'fpr': 0.0659, 'precision': 0.4013, 'auc': 0.793, 'pr_curve': 0.4503, 'train_time': 1.5134, 'infer_time': 0.2027}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Baseline', 'fold': 5, 'lr': 0.004, 'accuracy': 0.7058, 'tpr': 0.2809, 'fpr': 0.0792, 'precision': 0.3173, 'auc': 0.7403, 'pr_curve': 0.3178, 'train_time': 1.4689, 'infer_time': 0.1665}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Baseline', 'fold': 6, 'lr': 0.007, 'accuracy': 0.7274, 'tpr': 0.367, 'fpr': 0.0712, 'precision': 0.4886, 'auc': 0.7594, 'pr_curve': 0.3695, 'train_time': 1.6546, 'infer_time': 0.1814}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Baseline', 'fold': 7, 'lr': 0.004, 'accuracy': 0.6409, 'tpr': 0.398, 'fpr': 0.0631, 'precision': 0.4152, 'auc': 0.8212, 'pr_curve': 0.4404, 'train_time': 1.4196, 'infer_time': 0.1859}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Baseline', 'fold': 8, 'lr': 0.004, 'accuracy': 0.7254, 'tpr': 0.2995, 'fpr': 0.0806, 'precision': 0.2591, 'auc': 0.7692, 'pr_curve': 0.3403, 'train_time': 1.5581, 'infer_time': 0.5132}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Baseline', 'fold': 9, 'lr': 0.003, 'accuracy': 0.6231, 'tpr': 0.3144, 'fpr': 0.0731, 'precision': 0.3162, 'auc': 0.7831, 'pr_curve': 0.3575, 'train_time': 1.3888, 'infer_time': 0.1835}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Baseline', 'fold': 10, 'lr': 0.002, 'accuracy': 0.629, 'tpr': 0.3624, 'fpr': 0.0734, 'precision': 0.332, 'auc': 0.7355, 'pr_curve': 0.3193, 'train_time': 1.8538, 'infer_time': 0.3292}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Cyclic', 'fold': 1, 'lr': '0.001-0.02', 'accuracy': 0.6576, 'tpr': 0.3086, 'fpr': 0.0759, 'precision': 0.2668, 'auc': 0.7766, 'pr_curve': 0.3431, 'train_time': 1.5885, 'infer_time': 0.5649}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Cyclic', 'fold': 2, 'lr': '0.001-0.02', 'accuracy': 0.6036, 'tpr': 0.3508, 'fpr': 0.0742, 'precision': 0.3904, 'auc': 0.7565, 'pr_curve': 0.3323, 'train_time': 1.6359, 'infer_time': 0.1882}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Cyclic', 'fold': 3, 'lr': '0.001-0.02', 'accuracy': 0.6421, 'tpr': 0.4306, 'fpr': 0.0646, 'precision': 0.4662, 'auc': 0.7835, 'pr_curve': 0.3866, 'train_time': 1.6841, 'infer_time': 0.481}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Cyclic', 'fold': 4, 'lr': '0.001-0.02', 'accuracy': 0.6592, 'tpr': 0.4342, 'fpr': 0.0663, 'precision': 0.4628, 'auc': 0.8363, 'pr_curve': 0.5019, 'train_time': 1.527, 'infer_time': 0.1734}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Cyclic', 'fold': 5, 'lr': '0.001-0.02', 'accuracy': 0.6462, 'tpr': 0.2953, 'fpr': 0.0829, 'precision': 0.3071, 'auc': 0.7543, 'pr_curve': 0.3311, 'train_time': 1.4434, 'infer_time': 0.6728}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Cyclic', 'fold': 6, 'lr': '0.001-0.02', 'accuracy': 0.5772, 'tpr': 0.3603, 'fpr': 0.0721, 'precision': 0.387, 'auc': 0.7737, 'pr_curve': 0.361, 'train_time': 1.4946, 'infer_time': 0.6052}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Cyclic', 'fold': 7, 'lr': '0.001-0.02', 'accuracy': 0.6346, 'tpr': 0.4001, 'fpr': 0.0648, 'precision': 0.5652, 'auc': 0.7838, 'pr_curve': 0.3669, 'train_time': 1.8195, 'infer_time': 0.6323}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Cyclic', 'fold': 8, 'lr': '0.001-0.02', 'accuracy': 0.5513, 'tpr': 0.3481, 'fpr': 0.0797, 'precision': 0.3451, 'auc': 0.7651, 'pr_curve': 0.3639, 'train_time': 1.401, 'infer_time': 0.6956}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Cyclic', 'fold': 9, 'lr': '0.001-0.02', 'accuracy': 0.6678, 'tpr': 0.3967, 'fpr': 0.0702, 'precision': 0.3623, 'auc': 0.7858, 'pr_curve': 0.3839, 'train_time': 1.5561, 'infer_time': 0.1917}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Cyclic', 'fold': 10, 'lr': '0.001-0.02', 'accuracy': 0.6102, 'tpr': 0.3523, 'fpr': 0.0706, 'precision': 0.4902, 'auc': 0.7925, 'pr_curve': 0.3759, 'train_time': 1.4995, 'infer_time': 0.1922}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Ipmroved', 'fold': 1, 'lr': '0.002-0.05', 'accuracy': 0.6626, 'tpr': 0.3089, 'fpr': 0.075, 'precision': 0.3609, 'auc': 0.7841, 'pr_curve': 0.3941, 'train_time': 1.4177, 'infer_time': 0.6864}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Ipmroved', 'fold': 2, 'lr': '0.002-0.05', 'accuracy': 0.6574, 'tpr': 0.2563, 'fpr': 0.0822, 'precision': 0.2507, 'auc': 0.7516, 'pr_curve': 0.3072, 'train_time': 1.3506, 'infer_time': 0.728}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Ipmroved', 'fold': 3, 'lr': '0.001-0.05', 'accuracy': 0.5854, 'tpr': 0.3033, 'fpr': 0.0783, 'precision': 0.3813, 'auc': 0.7422, 'pr_curve': 0.3303, 'train_time': 1.6674, 'infer_time': 0.4057}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Ipmroved', 'fold': 4, 'lr': '0.001-0.05', 'accuracy': 0.6585, 'tpr': 0.3051, 'fpr': 0.0843, 'precision': 0.3304, 'auc': 0.7254, 'pr_curve': 0.3618, 'train_time': 1.4737, 'infer_time': 0.6313}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Ipmroved', 'fold': 5, 'lr': '0.002-0.05', 'accuracy': 0.6018, 'tpr': 0.2347, 'fpr': 0.083, 'precision': 0.2889, 'auc': 0.734, 'pr_curve': 0.3489, 'train_time': 1.2697, 'infer_time': 0.8191}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Ipmroved', 'fold': 6, 'lr': '0.001-0.05', 'accuracy': 0.5579, 'tpr': 0.3406, 'fpr': 0.0733, 'precision': 0.3867, 'auc': 0.7413, 'pr_curve': 0.3199, 'train_time': 1.322, 'infer_time': 0.1627}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Ipmroved', 'fold': 7, 'lr': '0.001-0.05', 'accuracy': 0.5286, 'tpr': 0.2228, 'fpr': 0.0801, 'precision': 0.3907, 'auc': 0.759, 'pr_curve': 0.3733, 'train_time': 1.302, 'infer_time': 0.7681}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Ipmroved', 'fold': 8, 'lr': '0.0-0.05', 'accuracy': 0.5896, 'tpr': 0.2552, 'fpr': 0.0856, 'precision': 0.3042, 'auc': 0.7915, 'pr_curve': 0.3992, 'train_time': 1.9264, 'infer_time': 0.586}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Ipmroved', 'fold': 9, 'lr': '0.001-0.05', 'accuracy': 0.5444, 'tpr': 0.2284, 'fpr': 0.0847, 'precision': 0.3521, 'auc': 0.7467, 'pr_curve': 0.3331, 'train_time': 1.4032, 'infer_time': 0.196}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Ipmroved', 'fold': 10, 'lr': '0.002-0.05', 'accuracy': 0.6175, 'tpr': 0.2958, 'fpr': 0.0779, 'precision': 0.3259, 'auc': 0.7484, 'pr_curve': 0.3311, 'train_time': 1.4472, 'infer_time': 0.7192}]
d9_scores = [{'dataset name': 'cifar10_58000:59000', 'alg name': 'Baseline', 'fold': 1, 'lr': 0.007, 'accuracy': 0.7525, 'tpr': 0.2772, 'fpr': 0.0818, 'precision': 0.3201, 'auc': 0.7046, 'pr_curve': 0.2619, 'train_time': 1.5773, 'infer_time': 0.185}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Baseline', 'fold': 2, 'lr': 0.016, 'accuracy': 0.688, 'tpr': 0.1486, 'fpr': 0.0948, 'precision': 0.1248, 'auc': 0.6744, 'pr_curve': 0.2405, 'train_time': 1.6676, 'infer_time': 0.4034}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Baseline', 'fold': 3, 'lr': 0.003, 'accuracy': 0.6504, 'tpr': 0.3104, 'fpr': 0.0775, 'precision': 0.3445, 'auc': 0.765, 'pr_curve': 0.3431, 'train_time': 1.8117, 'infer_time': 0.2816}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Baseline', 'fold': 4, 'lr': 0.005, 'accuracy': 0.7411, 'tpr': 0.3377, 'fpr': 0.0746, 'precision': 0.3662, 'auc': 0.7409, 'pr_curve': 0.3596, 'train_time': 1.6055, 'infer_time': 0.4999}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Baseline', 'fold': 5, 'lr': 0.005, 'accuracy': 0.7641, 'tpr': 0.32, 'fpr': 0.0742, 'precision': 0.3766, 'auc': 0.7174, 'pr_curve': 0.3205, 'train_time': 1.8675, 'infer_time': 0.1862}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Baseline', 'fold': 6, 'lr': 0.006, 'accuracy': 0.7914, 'tpr': 0.3238, 'fpr': 0.0738, 'precision': 0.3166, 'auc': 0.7793, 'pr_curve': 0.3438, 'train_time': 2.0568, 'infer_time': 1.3346}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Baseline', 'fold': 7, 'lr': 0.008, 'accuracy': 0.6285, 'tpr': 0.1953, 'fpr': 0.0929, 'precision': 0.188, 'auc': 0.6452, 'pr_curve': 0.2224, 'train_time': 1.5891, 'infer_time': 0.5498}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Baseline', 'fold': 8, 'lr': 0.011, 'accuracy': 0.5861, 'tpr': 0.2575, 'fpr': 0.0856, 'precision': 0.2046, 'auc': 0.699, 'pr_curve': 0.2971, 'train_time': 1.293, 'infer_time': 0.1859}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Baseline', 'fold': 9, 'lr': 0.005, 'accuracy': 0.593, 'tpr': 0.2848, 'fpr': 0.0801, 'precision': 0.2529, 'auc': 0.7386, 'pr_curve': 0.2891, 'train_time': 1.5298, 'infer_time': 0.6104}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Baseline', 'fold': 10, 'lr': 0.009000000000000001, 'accuracy': 0.704, 'tpr': 0.2969, 'fpr': 0.0765, 'precision': 0.2945, 'auc': 0.7016, 'pr_curve': 0.2767, 'train_time': 1.6528, 'infer_time': 0.4967}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Cyclic', 'fold': 1, 'lr': '0.001-0.02', 'accuracy': 0.54, 'tpr': 0.3231, 'fpr': 0.0767, 'precision': 0.3132, 'auc': 0.7439, 'pr_curve': 0.3399, 'train_time': 1.518, 'infer_time': 0.6446}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Cyclic', 'fold': 2, 'lr': '0.001-0.02', 'accuracy': 0.5774, 'tpr': 0.2171, 'fpr': 0.0842, 'precision': 0.2028, 'auc': 0.7629, 'pr_curve': 0.2754, 'train_time': 1.5461, 'infer_time': 0.1988}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Cyclic', 'fold': 3, 'lr': '0.001-0.02', 'accuracy': 0.6179, 'tpr': 0.3019, 'fpr': 0.0769, 'precision': 0.3022, 'auc': 0.8011, 'pr_curve': 0.3421, 'train_time': 1.6184, 'infer_time': 0.4801}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Cyclic', 'fold': 4, 'lr': '0.001-0.02', 'accuracy': 0.5203, 'tpr': 0.2655, 'fpr': 0.0834, 'precision': 0.3041, 'auc': 0.6781, 'pr_curve': 0.2965, 'train_time': 1.7746, 'infer_time': 0.2096}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Cyclic', 'fold': 5, 'lr': '0.001-0.02', 'accuracy': 0.6336, 'tpr': 0.3278, 'fpr': 0.0773, 'precision': 0.3255, 'auc': 0.735, 'pr_curve': 0.2952, 'train_time': 1.5597, 'infer_time': 0.1709}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Cyclic', 'fold': 6, 'lr': '0.001-0.02', 'accuracy': 0.547, 'tpr': 0.2372, 'fpr': 0.0876, 'precision': 0.4835, 'auc': 0.7342, 'pr_curve': 0.3186, 'train_time': 1.3292, 'infer_time': 0.7522}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Cyclic', 'fold': 7, 'lr': '0.001-0.02', 'accuracy': 0.5968, 'tpr': 0.2356, 'fpr': 0.0875, 'precision': 0.2515, 'auc': 0.7097, 'pr_curve': 0.2935, 'train_time': 1.4578, 'infer_time': 0.1847}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Cyclic', 'fold': 8, 'lr': '0.001-0.02', 'accuracy': 0.572, 'tpr': 0.2998, 'fpr': 0.078, 'precision': 0.3101, 'auc': 0.7221, 'pr_curve': 0.3028, 'train_time': 1.4783, 'infer_time': 0.1805}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Cyclic', 'fold': 9, 'lr': '0.001-0.02', 'accuracy': 0.5875, 'tpr': 0.3178, 'fpr': 0.0786, 'precision': 0.2849, 'auc': 0.752, 'pr_curve': 0.3175, 'train_time': 1.3514, 'infer_time': 0.7456}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Cyclic', 'fold': 10, 'lr': '0.001-0.02', 'accuracy': 0.6035, 'tpr': 0.3296, 'fpr': 0.0741, 'precision': 0.4566, 'auc': 0.737, 'pr_curve': 0.338, 'train_time': 1.5472, 'infer_time': 0.5685}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Ipmroved', 'fold': 1, 'lr': '0.001-0.05', 'accuracy': 0.5015, 'tpr': 0.2602, 'fpr': 0.0824, 'precision': 0.2827, 'auc': 0.7087, 'pr_curve': 0.3138, 'train_time': 1.2716, 'infer_time': 0.1858}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Ipmroved', 'fold': 2, 'lr': '0.002-0.05', 'accuracy': 0.568, 'tpr': 0.2175, 'fpr': 0.0842, 'precision': 0.2331, 'auc': 0.704, 'pr_curve': 0.2719, 'train_time': 1.4024, 'infer_time': 0.7014}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Ipmroved', 'fold': 3, 'lr': '0.001-0.05', 'accuracy': 0.5438, 'tpr': 0.2773, 'fpr': 0.0816, 'precision': 0.3485, 'auc': 0.7367, 'pr_curve': 0.3103, 'train_time': 1.3848, 'infer_time': 0.1822}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Ipmroved', 'fold': 4, 'lr': '0.001-0.05', 'accuracy': 0.498, 'tpr': 0.2476, 'fpr': 0.085, 'precision': 0.2885, 'auc': 0.6929, 'pr_curve': 0.3006, 'train_time': 1.29, 'infer_time': 0.1662}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Ipmroved', 'fold': 5, 'lr': '0.002-0.05', 'accuracy': 0.5422, 'tpr': 0.2548, 'fpr': 0.0867, 'precision': 0.1856, 'auc': 0.6911, 'pr_curve': 0.2519, 'train_time': 1.3076, 'infer_time': 0.7692}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Ipmroved', 'fold': 6, 'lr': '0.001-0.05', 'accuracy': 0.5235, 'tpr': 0.3501, 'fpr': 0.0724, 'precision': 0.3362, 'auc': 0.709, 'pr_curve': 0.292, 'train_time': 1.7503, 'infer_time': 0.2093}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Ipmroved', 'fold': 7, 'lr': '0.001-0.05', 'accuracy': 0.5656, 'tpr': 0.171, 'fpr': 0.093, 'precision': 0.2293, 'auc': 0.7037, 'pr_curve': 0.2454, 'train_time': 1.7519, 'infer_time': 0.3893}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Ipmroved', 'fold': 8, 'lr': '0.0-0.05', 'accuracy': 0.6307, 'tpr': 0.2108, 'fpr': 0.0879, 'precision': 0.1814, 'auc': 0.6915, 'pr_curve': 0.2724, 'train_time': 1.6321, 'infer_time': 0.5247}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Ipmroved', 'fold': 9, 'lr': '0.001-0.05', 'accuracy': 0.56, 'tpr': 0.2453, 'fpr': 0.0857, 'precision': 0.2936, 'auc': 0.7096, 'pr_curve': 0.3099, 'train_time': 1.4284, 'infer_time': 0.1905}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Ipmroved', 'fold': 10, 'lr': '0.002-0.05', 'accuracy': 0.6005, 'tpr': 0.3036, 'fpr': 0.0782, 'precision': 0.2566, 'auc': 0.7475, 'pr_curve': 0.286, 'train_time': 1.4158, 'infer_time': 0.7552}]
d10_scores = [{'dataset name': 'cifar10_59000:60000', 'alg name': 'Baseline', 'fold': 1, 'lr': 0.004, 'accuracy': 0.743, 'tpr': 0.2912, 'fpr': 0.0814, 'precision': 0.2902, 'auc': 0.7584, 'pr_curve': 0.3185, 'train_time': 1.6622, 'infer_time': 0.4345}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Baseline', 'fold': 2, 'lr': 0.01, 'accuracy': 0.6437, 'tpr': 0.27, 'fpr': 0.083, 'precision': 0.3373, 'auc': 0.7701, 'pr_curve': 0.3284, 'train_time': 1.5967, 'infer_time': 0.6191}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Baseline', 'fold': 3, 'lr': 0.006, 'accuracy': 0.735, 'tpr': 0.2433, 'fpr': 0.0824, 'precision': 0.3074, 'auc': 0.7384, 'pr_curve': 0.312, 'train_time': 1.5621, 'infer_time': 0.1641}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Baseline', 'fold': 4, 'lr': 0.008, 'accuracy': 0.7352, 'tpr': 0.2862, 'fpr': 0.0782, 'precision': 0.2785, 'auc': 0.723, 'pr_curve': 0.3299, 'train_time': 1.5873, 'infer_time': 0.1979}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Baseline', 'fold': 5, 'lr': 0.002, 'accuracy': 0.6747, 'tpr': 0.2664, 'fpr': 0.0817, 'precision': 0.2814, 'auc': 0.7266, 'pr_curve': 0.2913, 'train_time': 1.6604, 'infer_time': 0.4359}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Baseline', 'fold': 6, 'lr': 0.008, 'accuracy': 0.7248, 'tpr': 0.316, 'fpr': 0.0767, 'precision': 0.3413, 'auc': 0.7651, 'pr_curve': 0.3393, 'train_time': 1.6266, 'infer_time': 0.4786}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Baseline', 'fold': 7, 'lr': 0.003, 'accuracy': 0.6496, 'tpr': 0.2979, 'fpr': 0.0771, 'precision': 0.3131, 'auc': 0.7738, 'pr_curve': 0.3433, 'train_time': 1.6155, 'infer_time': 0.1926}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Baseline', 'fold': 8, 'lr': 0.004, 'accuracy': 0.6816, 'tpr': 0.2843, 'fpr': 0.0822, 'precision': 0.2418, 'auc': 0.8002, 'pr_curve': 0.3877, 'train_time': 1.693, 'infer_time': 0.4579}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Baseline', 'fold': 9, 'lr': 0.004, 'accuracy': 0.6874, 'tpr': 0.2797, 'fpr': 0.0813, 'precision': 0.3823, 'auc': 0.7273, 'pr_curve': 0.2942, 'train_time': 1.4472, 'infer_time': 0.1909}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Baseline', 'fold': 10, 'lr': 0.008, 'accuracy': 0.661, 'tpr': 0.3161, 'fpr': 0.0778, 'precision': 0.2517, 'auc': 0.7313, 'pr_curve': 0.3143, 'train_time': 1.4746, 'infer_time': 0.1815}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Cyclic', 'fold': 1, 'lr': '0.001-0.02', 'accuracy': 0.6221, 'tpr': 0.2921, 'fpr': 0.0814, 'precision': 0.3063, 'auc': 0.7359, 'pr_curve': 0.2957, 'train_time': 1.5169, 'infer_time': 0.1982}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Cyclic', 'fold': 2, 'lr': '0.001-0.02', 'accuracy': 0.5689, 'tpr': 0.3269, 'fpr': 0.0758, 'precision': 0.3641, 'auc': 0.7603, 'pr_curve': 0.3653, 'train_time': 1.533, 'infer_time': 0.1901}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Cyclic', 'fold': 3, 'lr': '0.001-0.02', 'accuracy': 0.6262, 'tpr': 0.3088, 'fpr': 0.0796, 'precision': 0.271, 'auc': 0.7463, 'pr_curve': 0.3243, 'train_time': 1.4687, 'infer_time': 0.1909}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Cyclic', 'fold': 4, 'lr': '0.001-0.02', 'accuracy': 0.6065, 'tpr': 0.3063, 'fpr': 0.0768, 'precision': 0.3767, 'auc': 0.7315, 'pr_curve': 0.3089, 'train_time': 1.8279, 'infer_time': 0.3118}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Cyclic', 'fold': 5, 'lr': '0.001-0.02', 'accuracy': 0.5802, 'tpr': 0.3046, 'fpr': 0.0794, 'precision': 0.3165, 'auc': 0.7179, 'pr_curve': 0.3193, 'train_time': 1.4929, 'infer_time': 0.1881}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Cyclic', 'fold': 6, 'lr': '0.001-0.02', 'accuracy': 0.582, 'tpr': 0.3424, 'fpr': 0.08, 'precision': 0.2651, 'auc': 0.7714, 'pr_curve': 0.3343, 'train_time': 1.6084, 'infer_time': 0.1897}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Cyclic', 'fold': 7, 'lr': '0.001-0.02', 'accuracy': 0.5214, 'tpr': 0.2779, 'fpr': 0.0821, 'precision': 0.2895, 'auc': 0.753, 'pr_curve': 0.3145, 'train_time': 1.4945, 'infer_time': 0.7115}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Cyclic', 'fold': 8, 'lr': '0.001-0.02', 'accuracy': 0.4751, 'tpr': 0.1939, 'fpr': 0.0906, 'precision': 0.2056, 'auc': 0.7529, 'pr_curve': 0.2937, 'train_time': 1.4201, 'infer_time': 0.2126}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Cyclic', 'fold': 9, 'lr': '0.001-0.02', 'accuracy': 0.6115, 'tpr': 0.2842, 'fpr': 0.0768, 'precision': 0.2863, 'auc': 0.7562, 'pr_curve': 0.3356, 'train_time': 1.5593, 'infer_time': 0.6221}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Cyclic', 'fold': 10, 'lr': '0.001-0.02', 'accuracy': 0.6188, 'tpr': 0.2708, 'fpr': 0.0835, 'precision': 0.3155, 'auc': 0.7368, 'pr_curve': 0.2991, 'train_time': 1.6677, 'infer_time': 0.5179}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Ipmroved', 'fold': 1, 'lr': '0.002-0.05', 'accuracy': 0.5658, 'tpr': 0.2612, 'fpr': 0.0824, 'precision': 0.3297, 'auc': 0.6917, 'pr_curve': 0.2751, 'train_time': 1.4225, 'infer_time': 0.1879}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Ipmroved', 'fold': 2, 'lr': '0.002-0.05', 'accuracy': 0.5256, 'tpr': 0.2957, 'fpr': 0.0788, 'precision': 0.3105, 'auc': 0.768, 'pr_curve': 0.3619, 'train_time': 1.4149, 'infer_time': 0.1844}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Ipmroved', 'fold': 3, 'lr': '0.001-0.05', 'accuracy': 0.6203, 'tpr': 0.3017, 'fpr': 0.0745, 'precision': 0.3483, 'auc': 0.7631, 'pr_curve': 0.3441, 'train_time': 1.6121, 'infer_time': 0.1851}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Ipmroved', 'fold': 4, 'lr': '0.001-0.05', 'accuracy': 0.4788, 'tpr': 0.2532, 'fpr': 0.087, 'precision': 0.2664, 'auc': 0.6974, 'pr_curve': 0.2788, 'train_time': 1.4484, 'infer_time': 0.7188}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Ipmroved', 'fold': 5, 'lr': '0.001-0.05', 'accuracy': 0.526, 'tpr': 0.2796, 'fpr': 0.0807, 'precision': 0.3313, 'auc': 0.7243, 'pr_curve': 0.3215, 'train_time': 1.4404, 'infer_time': 0.7005}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Ipmroved', 'fold': 6, 'lr': '0.002-0.05', 'accuracy': 0.6786, 'tpr': 0.3539, 'fpr': 0.0765, 'precision': 0.31, 'auc': 0.8003, 'pr_curve': 0.3915, 'train_time': 1.5425, 'infer_time': 0.5982}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Ipmroved', 'fold': 7, 'lr': '0.002-0.05', 'accuracy': 0.5552, 'tpr': 0.2119, 'fpr': 0.0874, 'precision': 0.2211, 'auc': 0.7191, 'pr_curve': 0.2839, 'train_time': 1.3359, 'infer_time': 0.1635}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Ipmroved', 'fold': 8, 'lr': '0.001-0.05', 'accuracy': 0.5769, 'tpr': 0.2649, 'fpr': 0.0828, 'precision': 0.2666, 'auc': 0.7444, 'pr_curve': 0.3132, 'train_time': 1.3631, 'infer_time': 0.7471}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Ipmroved', 'fold': 9, 'lr': '0.001-0.05', 'accuracy': 0.5246, 'tpr': 0.2776, 'fpr': 0.0747, 'precision': 0.4056, 'auc': 0.7684, 'pr_curve': 0.3188, 'train_time': 1.3213, 'infer_time': 0.1876}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Ipmroved', 'fold': 10, 'lr': '0.001-0.05', 'accuracy': 0.5591, 'tpr': 0.3133, 'fpr': 0.0789, 'precision': 0.2995, 'auc': 0.724, 'pr_curve': 0.2923, 'train_time': 1.4847, 'infer_time': 0.636}]
d6_10_scores = [{'dataset name': 'cifar10_56000:57000', 'alg name': 'Baseline', 'fold': 1, 'lr': 0.01, 'accuracy': 0.7017, 'tpr': 0.3692, 'fpr': 0.0767, 'precision': 0.3605, 'auc': 0.7573, 'pr_curve': 0.3401, 'train_time': 1.4502, 'infer_time': 0.8063}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Baseline', 'fold': 2, 'lr': 0.005, 'accuracy': 0.6424, 'tpr': 0.2864, 'fpr': 0.0769, 'precision': 0.4338, 'auc': 0.7301, 'pr_curve': 0.3174, 'train_time': 1.5167, 'infer_time': 0.6269}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Baseline', 'fold': 3, 'lr': 0.004, 'accuracy': 0.7358, 'tpr': 0.3709, 'fpr': 0.0726, 'precision': 0.3136, 'auc': 0.8134, 'pr_curve': 0.465, 'train_time': 1.6853, 'infer_time': 0.4795}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Baseline', 'fold': 4, 'lr': 0.007, 'accuracy': 0.7055, 'tpr': 0.3315, 'fpr': 0.077, 'precision': 0.3032, 'auc': 0.7722, 'pr_curve': 0.3159, 'train_time': 1.4795, 'infer_time': 0.1646}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Baseline', 'fold': 5, 'lr': 0.006, 'accuracy': 0.7959, 'tpr': 0.3655, 'fpr': 0.0672, 'precision': 0.4731, 'auc': 0.7909, 'pr_curve': 0.3812, 'train_time': 1.6786, 'infer_time': 0.4497}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Baseline', 'fold': 6, 'lr': 0.01, 'accuracy': 0.747, 'tpr': 0.3889, 'fpr': 0.0714, 'precision': 0.3911, 'auc': 0.7942, 'pr_curve': 0.381, 'train_time': 1.6178, 'infer_time': 0.1802}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Baseline', 'fold': 7, 'lr': 0.011, 'accuracy': 0.3633, 'tpr': 0.2087, 'fpr': 0.0902, 'precision': 0.1681, 'auc': 0.6607, 'pr_curve': 0.2173, 'train_time': 1.2551, 'infer_time': 0.19}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Baseline', 'fold': 8, 'lr': 0.009000000000000001, 'accuracy': 0.7156, 'tpr': 0.3431, 'fpr': 0.0733, 'precision': 0.4184, 'auc': 0.7591, 'pr_curve': 0.3263, 'train_time': 1.5102, 'infer_time': 0.5926}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Baseline', 'fold': 9, 'lr': 0.007, 'accuracy': 0.6136, 'tpr': 0.2808, 'fpr': 0.0819, 'precision': 0.2981, 'auc': 0.7169, 'pr_curve': 0.3127, 'train_time': 1.4543, 'infer_time': 0.1776}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Baseline', 'fold': 10, 'lr': 0.007, 'accuracy': 0.6001, 'tpr': 0.3068, 'fpr': 0.0821, 'precision': 0.3068, 'auc': 0.6824, 'pr_curve': 0.2765, 'train_time': 1.2574, 'infer_time': 0.1804}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Cyclic', 'fold': 1, 'lr': '0.001-0.02', 'accuracy': 0.5913, 'tpr': 0.2345, 'fpr': 0.0893, 'precision': 0.2358, 'auc': 0.7262, 'pr_curve': 0.2805, 'train_time': 1.4322, 'infer_time': 0.6631}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Cyclic', 'fold': 2, 'lr': '0.001-0.02', 'accuracy': 0.5926, 'tpr': 0.3073, 'fpr': 0.0791, 'precision': 0.3939, 'auc': 0.7406, 'pr_curve': 0.3474, 'train_time': 1.4175, 'infer_time': 0.6956}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Cyclic', 'fold': 3, 'lr': '0.001-0.02', 'accuracy': 0.633, 'tpr': 0.3861, 'fpr': 0.0689, 'precision': 0.4704, 'auc': 0.7759, 'pr_curve': 0.3791, 'train_time': 1.4497, 'infer_time': 0.1898}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Cyclic', 'fold': 4, 'lr': '0.001-0.02', 'accuracy': 0.5431, 'tpr': 0.3035, 'fpr': 0.0789, 'precision': 0.3016, 'auc': 0.7794, 'pr_curve': 0.4071, 'train_time': 1.374, 'infer_time': 0.1771}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Cyclic', 'fold': 5, 'lr': '0.001-0.02', 'accuracy': 0.5203, 'tpr': 0.4278, 'fpr': 0.0653, 'precision': 0.4728, 'auc': 0.7902, 'pr_curve': 0.4152, 'train_time': 1.8005, 'infer_time': 0.6427}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Cyclic', 'fold': 6, 'lr': '0.001-0.02', 'accuracy': 0.5961, 'tpr': 0.2846, 'fpr': 0.0832, 'precision': 0.4406, 'auc': 0.7413, 'pr_curve': 0.3659, 'train_time': 1.4223, 'infer_time': 0.6702}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Cyclic', 'fold': 7, 'lr': '0.001-0.02', 'accuracy': 0.5996, 'tpr': 0.2166, 'fpr': 0.0883, 'precision': 0.2561, 'auc': 0.7278, 'pr_curve': 0.2694, 'train_time': 1.3972, 'infer_time': 0.6958}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Cyclic', 'fold': 8, 'lr': '0.001-0.02', 'accuracy': 0.5805, 'tpr': 0.2533, 'fpr': 0.0791, 'precision': 0.4264, 'auc': 0.7576, 'pr_curve': 0.3564, 'train_time': 1.3501, 'infer_time': 0.1839}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Cyclic', 'fold': 9, 'lr': '0.001-0.02', 'accuracy': 0.5535, 'tpr': 0.2896, 'fpr': 0.0791, 'precision': 0.3792, 'auc': 0.6944, 'pr_curve': 0.2927, 'train_time': 1.4052, 'infer_time': 0.6779}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Cyclic', 'fold': 10, 'lr': '0.001-0.02', 'accuracy': 0.608, 'tpr': 0.1932, 'fpr': 0.0919, 'precision': 0.1748, 'auc': 0.733, 'pr_curve': 0.3012, 'train_time': 1.4387, 'infer_time': 0.6691}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Ipmroved', 'fold': 1, 'lr': '0.001-0.05', 'accuracy': 0.5482, 'tpr': 0.2806, 'fpr': 0.0847, 'precision': 0.1966, 'auc': 0.7367, 'pr_curve': 0.2609, 'train_time': 1.2382, 'infer_time': 0.1759}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Ipmroved', 'fold': 2, 'lr': '0.002-0.05', 'accuracy': 0.6559, 'tpr': 0.2128, 'fpr': 0.0801, 'precision': 0.2172, 'auc': 0.7458, 'pr_curve': 0.3257, 'train_time': 1.6097, 'infer_time': 0.1825}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Ipmroved', 'fold': 3, 'lr': '0.001-0.05', 'accuracy': 0.5375, 'tpr': 0.2599, 'fpr': 0.081, 'precision': 0.2855, 'auc': 0.7209, 'pr_curve': 0.3301, 'train_time': 1.2295, 'infer_time': 0.2167}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Ipmroved', 'fold': 4, 'lr': '0.002-0.05', 'accuracy': 0.5679, 'tpr': 0.2672, 'fpr': 0.0828, 'precision': 0.3023, 'auc': 0.7576, 'pr_curve': 0.3349, 'train_time': 1.3997, 'infer_time': 0.2018}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Ipmroved', 'fold': 5, 'lr': '0.0-0.05', 'accuracy': 0.5654, 'tpr': 0.266, 'fpr': 0.0813, 'precision': 0.341, 'auc': 0.7483, 'pr_curve': 0.3268, 'train_time': 1.4787, 'infer_time': 0.1854}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Ipmroved', 'fold': 6, 'lr': '0.0-0.05', 'accuracy': 0.5269, 'tpr': 0.2411, 'fpr': 0.0848, 'precision': 0.3534, 'auc': 0.7119, 'pr_curve': 0.3047, 'train_time': 1.3442, 'infer_time': 0.1914}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Ipmroved', 'fold': 7, 'lr': '0.001-0.05', 'accuracy': 0.549, 'tpr': 0.3433, 'fpr': 0.072, 'precision': 0.3335, 'auc': 0.758, 'pr_curve': 0.3317, 'train_time': 1.4406, 'infer_time': 0.693}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Ipmroved', 'fold': 8, 'lr': '0.001-0.05', 'accuracy': 0.6051, 'tpr': 0.2938, 'fpr': 0.0786, 'precision': 0.4195, 'auc': 0.7602, 'pr_curve': 0.3613, 'train_time': 1.4795, 'infer_time': 0.6812}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Ipmroved', 'fold': 9, 'lr': '0.002-0.05', 'accuracy': 0.5992, 'tpr': 0.315, 'fpr': 0.0769, 'precision': 0.3543, 'auc': 0.7075, 'pr_curve': 0.2696, 'train_time': 1.4278, 'infer_time': 0.7343}, {'dataset name': 'cifar10_56000:57000', 'alg name': 'Ipmroved', 'fold': 10, 'lr': '0.001-0.05', 'accuracy': 0.5337, 'tpr': 0.1799, 'fpr': 0.0931, 'precision': 0.2203, 'auc': 0.6896, 'pr_curve': 0.2677, 'train_time': 1.382, 'infer_time': 0.1875}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Baseline', 'fold': 1, 'lr': 0.003, 'accuracy': 0.6779, 'tpr': 0.3383, 'fpr': 0.0731, 'precision': 0.3602, 'auc': 0.7405, 'pr_curve': 0.3009, 'train_time': 1.5906, 'infer_time': 0.4768}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Baseline', 'fold': 2, 'lr': 0.007, 'accuracy': 0.6072, 'tpr': 0.2814, 'fpr': 0.0861, 'precision': 0.2907, 'auc': 0.7556, 'pr_curve': 0.3082, 'train_time': 1.4454, 'infer_time': 0.7578}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Baseline', 'fold': 3, 'lr': 0.009000000000000001, 'accuracy': 0.6924, 'tpr': 0.3211, 'fpr': 0.0756, 'precision': 0.3722, 'auc': 0.776, 'pr_curve': 0.3955, 'train_time': 1.5817, 'infer_time': 0.5091}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Baseline', 'fold': 4, 'lr': 0.006, 'accuracy': 0.7043, 'tpr': 0.435, 'fpr': 0.0659, 'precision': 0.4013, 'auc': 0.793, 'pr_curve': 0.4503, 'train_time': 1.5134, 'infer_time': 0.2027}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Baseline', 'fold': 5, 'lr': 0.004, 'accuracy': 0.7058, 'tpr': 0.2809, 'fpr': 0.0792, 'precision': 0.3173, 'auc': 0.7403, 'pr_curve': 0.3178, 'train_time': 1.4689, 'infer_time': 0.1665}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Baseline', 'fold': 6, 'lr': 0.007, 'accuracy': 0.7274, 'tpr': 0.367, 'fpr': 0.0712, 'precision': 0.4886, 'auc': 0.7594, 'pr_curve': 0.3695, 'train_time': 1.6546, 'infer_time': 0.1814}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Baseline', 'fold': 7, 'lr': 0.004, 'accuracy': 0.6409, 'tpr': 0.398, 'fpr': 0.0631, 'precision': 0.4152, 'auc': 0.8212, 'pr_curve': 0.4404, 'train_time': 1.4196, 'infer_time': 0.1859}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Baseline', 'fold': 8, 'lr': 0.004, 'accuracy': 0.7254, 'tpr': 0.2995, 'fpr': 0.0806, 'precision': 0.2591, 'auc': 0.7692, 'pr_curve': 0.3403, 'train_time': 1.5581, 'infer_time': 0.5132}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Baseline', 'fold': 9, 'lr': 0.003, 'accuracy': 0.6231, 'tpr': 0.3144, 'fpr': 0.0731, 'precision': 0.3162, 'auc': 0.7831, 'pr_curve': 0.3575, 'train_time': 1.3888, 'infer_time': 0.1835}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Baseline', 'fold': 10, 'lr': 0.002, 'accuracy': 0.629, 'tpr': 0.3624, 'fpr': 0.0734, 'precision': 0.332, 'auc': 0.7355, 'pr_curve': 0.3193, 'train_time': 1.8538, 'infer_time': 0.3292}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Cyclic', 'fold': 1, 'lr': '0.001-0.02', 'accuracy': 0.6576, 'tpr': 0.3086, 'fpr': 0.0759, 'precision': 0.2668, 'auc': 0.7766, 'pr_curve': 0.3431, 'train_time': 1.5885, 'infer_time': 0.5649}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Cyclic', 'fold': 2, 'lr': '0.001-0.02', 'accuracy': 0.6036, 'tpr': 0.3508, 'fpr': 0.0742, 'precision': 0.3904, 'auc': 0.7565, 'pr_curve': 0.3323, 'train_time': 1.6359, 'infer_time': 0.1882}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Cyclic', 'fold': 3, 'lr': '0.001-0.02', 'accuracy': 0.6421, 'tpr': 0.4306, 'fpr': 0.0646, 'precision': 0.4662, 'auc': 0.7835, 'pr_curve': 0.3866, 'train_time': 1.6841, 'infer_time': 0.481}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Cyclic', 'fold': 4, 'lr': '0.001-0.02', 'accuracy': 0.6592, 'tpr': 0.4342, 'fpr': 0.0663, 'precision': 0.4628, 'auc': 0.8363, 'pr_curve': 0.5019, 'train_time': 1.527, 'infer_time': 0.1734}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Cyclic', 'fold': 5, 'lr': '0.001-0.02', 'accuracy': 0.6462, 'tpr': 0.2953, 'fpr': 0.0829, 'precision': 0.3071, 'auc': 0.7543, 'pr_curve': 0.3311, 'train_time': 1.4434, 'infer_time': 0.6728}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Cyclic', 'fold': 6, 'lr': '0.001-0.02', 'accuracy': 0.5772, 'tpr': 0.3603, 'fpr': 0.0721, 'precision': 0.387, 'auc': 0.7737, 'pr_curve': 0.361, 'train_time': 1.4946, 'infer_time': 0.6052}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Cyclic', 'fold': 7, 'lr': '0.001-0.02', 'accuracy': 0.6346, 'tpr': 0.4001, 'fpr': 0.0648, 'precision': 0.5652, 'auc': 0.7838, 'pr_curve': 0.3669, 'train_time': 1.8195, 'infer_time': 0.6323}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Cyclic', 'fold': 8, 'lr': '0.001-0.02', 'accuracy': 0.5513, 'tpr': 0.3481, 'fpr': 0.0797, 'precision': 0.3451, 'auc': 0.7651, 'pr_curve': 0.3639, 'train_time': 1.401, 'infer_time': 0.6956}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Cyclic', 'fold': 9, 'lr': '0.001-0.02', 'accuracy': 0.6678, 'tpr': 0.3967, 'fpr': 0.0702, 'precision': 0.3623, 'auc': 0.7858, 'pr_curve': 0.3839, 'train_time': 1.5561, 'infer_time': 0.1917}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Cyclic', 'fold': 10, 'lr': '0.001-0.02', 'accuracy': 0.6102, 'tpr': 0.3523, 'fpr': 0.0706, 'precision': 0.4902, 'auc': 0.7925, 'pr_curve': 0.3759, 'train_time': 1.4995, 'infer_time': 0.1922}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Ipmroved', 'fold': 1, 'lr': '0.002-0.05', 'accuracy': 0.6626, 'tpr': 0.3089, 'fpr': 0.075, 'precision': 0.3609, 'auc': 0.7841, 'pr_curve': 0.3941, 'train_time': 1.4177, 'infer_time': 0.6864}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Ipmroved', 'fold': 2, 'lr': '0.002-0.05', 'accuracy': 0.6574, 'tpr': 0.2563, 'fpr': 0.0822, 'precision': 0.2507, 'auc': 0.7516, 'pr_curve': 0.3072, 'train_time': 1.3506, 'infer_time': 0.728}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Ipmroved', 'fold': 3, 'lr': '0.001-0.05', 'accuracy': 0.5854, 'tpr': 0.3033, 'fpr': 0.0783, 'precision': 0.3813, 'auc': 0.7422, 'pr_curve': 0.3303, 'train_time': 1.6674, 'infer_time': 0.4057}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Ipmroved', 'fold': 4, 'lr': '0.001-0.05', 'accuracy': 0.6585, 'tpr': 0.3051, 'fpr': 0.0843, 'precision': 0.3304, 'auc': 0.7254, 'pr_curve': 0.3618, 'train_time': 1.4737, 'infer_time': 0.6313}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Ipmroved', 'fold': 5, 'lr': '0.002-0.05', 'accuracy': 0.6018, 'tpr': 0.2347, 'fpr': 0.083, 'precision': 0.2889, 'auc': 0.734, 'pr_curve': 0.3489, 'train_time': 1.2697, 'infer_time': 0.8191}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Ipmroved', 'fold': 6, 'lr': '0.001-0.05', 'accuracy': 0.5579, 'tpr': 0.3406, 'fpr': 0.0733, 'precision': 0.3867, 'auc': 0.7413, 'pr_curve': 0.3199, 'train_time': 1.322, 'infer_time': 0.1627}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Ipmroved', 'fold': 7, 'lr': '0.001-0.05', 'accuracy': 0.5286, 'tpr': 0.2228, 'fpr': 0.0801, 'precision': 0.3907, 'auc': 0.759, 'pr_curve': 0.3733, 'train_time': 1.302, 'infer_time': 0.7681}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Ipmroved', 'fold': 8, 'lr': '0.0-0.05', 'accuracy': 0.5896, 'tpr': 0.2552, 'fpr': 0.0856, 'precision': 0.3042, 'auc': 0.7915, 'pr_curve': 0.3992, 'train_time': 1.9264, 'infer_time': 0.586}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Ipmroved', 'fold': 9, 'lr': '0.001-0.05', 'accuracy': 0.5444, 'tpr': 0.2284, 'fpr': 0.0847, 'precision': 0.3521, 'auc': 0.7467, 'pr_curve': 0.3331, 'train_time': 1.4032, 'infer_time': 0.196}, {'dataset name': 'cifar10_57000:58000', 'alg name': 'Ipmroved', 'fold': 10, 'lr': '0.002-0.05', 'accuracy': 0.6175, 'tpr': 0.2958, 'fpr': 0.0779, 'precision': 0.3259, 'auc': 0.7484, 'pr_curve': 0.3311, 'train_time': 1.4472, 'infer_time': 0.7192}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Baseline', 'fold': 1, 'lr': 0.007, 'accuracy': 0.7525, 'tpr': 0.2772, 'fpr': 0.0818, 'precision': 0.3201, 'auc': 0.7046, 'pr_curve': 0.2619, 'train_time': 1.5773, 'infer_time': 0.185}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Baseline', 'fold': 2, 'lr': 0.016, 'accuracy': 0.688, 'tpr': 0.1486, 'fpr': 0.0948, 'precision': 0.1248, 'auc': 0.6744, 'pr_curve': 0.2405, 'train_time': 1.6676, 'infer_time': 0.4034}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Baseline', 'fold': 3, 'lr': 0.003, 'accuracy': 0.6504, 'tpr': 0.3104, 'fpr': 0.0775, 'precision': 0.3445, 'auc': 0.765, 'pr_curve': 0.3431, 'train_time': 1.8117, 'infer_time': 0.2816}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Baseline', 'fold': 4, 'lr': 0.005, 'accuracy': 0.7411, 'tpr': 0.3377, 'fpr': 0.0746, 'precision': 0.3662, 'auc': 0.7409, 'pr_curve': 0.3596, 'train_time': 1.6055, 'infer_time': 0.4999}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Baseline', 'fold': 5, 'lr': 0.005, 'accuracy': 0.7641, 'tpr': 0.32, 'fpr': 0.0742, 'precision': 0.3766, 'auc': 0.7174, 'pr_curve': 0.3205, 'train_time': 1.8675, 'infer_time': 0.1862}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Baseline', 'fold': 6, 'lr': 0.006, 'accuracy': 0.7914, 'tpr': 0.3238, 'fpr': 0.0738, 'precision': 0.3166, 'auc': 0.7793, 'pr_curve': 0.3438, 'train_time': 2.0568, 'infer_time': 1.3346}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Baseline', 'fold': 7, 'lr': 0.008, 'accuracy': 0.6285, 'tpr': 0.1953, 'fpr': 0.0929, 'precision': 0.188, 'auc': 0.6452, 'pr_curve': 0.2224, 'train_time': 1.5891, 'infer_time': 0.5498}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Baseline', 'fold': 8, 'lr': 0.011, 'accuracy': 0.5861, 'tpr': 0.2575, 'fpr': 0.0856, 'precision': 0.2046, 'auc': 0.699, 'pr_curve': 0.2971, 'train_time': 1.293, 'infer_time': 0.1859}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Baseline', 'fold': 9, 'lr': 0.005, 'accuracy': 0.593, 'tpr': 0.2848, 'fpr': 0.0801, 'precision': 0.2529, 'auc': 0.7386, 'pr_curve': 0.2891, 'train_time': 1.5298, 'infer_time': 0.6104}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Baseline', 'fold': 10, 'lr': 0.009000000000000001, 'accuracy': 0.704, 'tpr': 0.2969, 'fpr': 0.0765, 'precision': 0.2945, 'auc': 0.7016, 'pr_curve': 0.2767, 'train_time': 1.6528, 'infer_time': 0.4967}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Cyclic', 'fold': 1, 'lr': '0.001-0.02', 'accuracy': 0.54, 'tpr': 0.3231, 'fpr': 0.0767, 'precision': 0.3132, 'auc': 0.7439, 'pr_curve': 0.3399, 'train_time': 1.518, 'infer_time': 0.6446}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Cyclic', 'fold': 2, 'lr': '0.001-0.02', 'accuracy': 0.5774, 'tpr': 0.2171, 'fpr': 0.0842, 'precision': 0.2028, 'auc': 0.7629, 'pr_curve': 0.2754, 'train_time': 1.5461, 'infer_time': 0.1988}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Cyclic', 'fold': 3, 'lr': '0.001-0.02', 'accuracy': 0.6179, 'tpr': 0.3019, 'fpr': 0.0769, 'precision': 0.3022, 'auc': 0.8011, 'pr_curve': 0.3421, 'train_time': 1.6184, 'infer_time': 0.4801}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Cyclic', 'fold': 4, 'lr': '0.001-0.02', 'accuracy': 0.5203, 'tpr': 0.2655, 'fpr': 0.0834, 'precision': 0.3041, 'auc': 0.6781, 'pr_curve': 0.2965, 'train_time': 1.7746, 'infer_time': 0.2096}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Cyclic', 'fold': 5, 'lr': '0.001-0.02', 'accuracy': 0.6336, 'tpr': 0.3278, 'fpr': 0.0773, 'precision': 0.3255, 'auc': 0.735, 'pr_curve': 0.2952, 'train_time': 1.5597, 'infer_time': 0.1709}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Cyclic', 'fold': 6, 'lr': '0.001-0.02', 'accuracy': 0.547, 'tpr': 0.2372, 'fpr': 0.0876, 'precision': 0.4835, 'auc': 0.7342, 'pr_curve': 0.3186, 'train_time': 1.3292, 'infer_time': 0.7522}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Cyclic', 'fold': 7, 'lr': '0.001-0.02', 'accuracy': 0.5968, 'tpr': 0.2356, 'fpr': 0.0875, 'precision': 0.2515, 'auc': 0.7097, 'pr_curve': 0.2935, 'train_time': 1.4578, 'infer_time': 0.1847}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Cyclic', 'fold': 8, 'lr': '0.001-0.02', 'accuracy': 0.572, 'tpr': 0.2998, 'fpr': 0.078, 'precision': 0.3101, 'auc': 0.7221, 'pr_curve': 0.3028, 'train_time': 1.4783, 'infer_time': 0.1805}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Cyclic', 'fold': 9, 'lr': '0.001-0.02', 'accuracy': 0.5875, 'tpr': 0.3178, 'fpr': 0.0786, 'precision': 0.2849, 'auc': 0.752, 'pr_curve': 0.3175, 'train_time': 1.3514, 'infer_time': 0.7456}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Cyclic', 'fold': 10, 'lr': '0.001-0.02', 'accuracy': 0.6035, 'tpr': 0.3296, 'fpr': 0.0741, 'precision': 0.4566, 'auc': 0.737, 'pr_curve': 0.338, 'train_time': 1.5472, 'infer_time': 0.5685}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Ipmroved', 'fold': 1, 'lr': '0.001-0.05', 'accuracy': 0.5015, 'tpr': 0.2602, 'fpr': 0.0824, 'precision': 0.2827, 'auc': 0.7087, 'pr_curve': 0.3138, 'train_time': 1.2716, 'infer_time': 0.1858}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Ipmroved', 'fold': 2, 'lr': '0.002-0.05', 'accuracy': 0.568, 'tpr': 0.2175, 'fpr': 0.0842, 'precision': 0.2331, 'auc': 0.704, 'pr_curve': 0.2719, 'train_time': 1.4024, 'infer_time': 0.7014}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Ipmroved', 'fold': 3, 'lr': '0.001-0.05', 'accuracy': 0.5438, 'tpr': 0.2773, 'fpr': 0.0816, 'precision': 0.3485, 'auc': 0.7367, 'pr_curve': 0.3103, 'train_time': 1.3848, 'infer_time': 0.1822}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Ipmroved', 'fold': 4, 'lr': '0.001-0.05', 'accuracy': 0.498, 'tpr': 0.2476, 'fpr': 0.085, 'precision': 0.2885, 'auc': 0.6929, 'pr_curve': 0.3006, 'train_time': 1.29, 'infer_time': 0.1662}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Ipmroved', 'fold': 5, 'lr': '0.002-0.05', 'accuracy': 0.5422, 'tpr': 0.2548, 'fpr': 0.0867, 'precision': 0.1856, 'auc': 0.6911, 'pr_curve': 0.2519, 'train_time': 1.3076, 'infer_time': 0.7692}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Ipmroved', 'fold': 6, 'lr': '0.001-0.05', 'accuracy': 0.5235, 'tpr': 0.3501, 'fpr': 0.0724, 'precision': 0.3362, 'auc': 0.709, 'pr_curve': 0.292, 'train_time': 1.7503, 'infer_time': 0.2093}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Ipmroved', 'fold': 7, 'lr': '0.001-0.05', 'accuracy': 0.5656, 'tpr': 0.171, 'fpr': 0.093, 'precision': 0.2293, 'auc': 0.7037, 'pr_curve': 0.2454, 'train_time': 1.7519, 'infer_time': 0.3893}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Ipmroved', 'fold': 8, 'lr': '0.0-0.05', 'accuracy': 0.6307, 'tpr': 0.2108, 'fpr': 0.0879, 'precision': 0.1814, 'auc': 0.6915, 'pr_curve': 0.2724, 'train_time': 1.6321, 'infer_time': 0.5247}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Ipmroved', 'fold': 9, 'lr': '0.001-0.05', 'accuracy': 0.56, 'tpr': 0.2453, 'fpr': 0.0857, 'precision': 0.2936, 'auc': 0.7096, 'pr_curve': 0.3099, 'train_time': 1.4284, 'infer_time': 0.1905}, {'dataset name': 'cifar10_58000:59000', 'alg name': 'Ipmroved', 'fold': 10, 'lr': '0.002-0.05', 'accuracy': 0.6005, 'tpr': 0.3036, 'fpr': 0.0782, 'precision': 0.2566, 'auc': 0.7475, 'pr_curve': 0.286, 'train_time': 1.4158, 'infer_time': 0.7552}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Baseline', 'fold': 1, 'lr': 0.004, 'accuracy': 0.743, 'tpr': 0.2912, 'fpr': 0.0814, 'precision': 0.2902, 'auc': 0.7584, 'pr_curve': 0.3185, 'train_time': 1.6622, 'infer_time': 0.4345}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Baseline', 'fold': 2, 'lr': 0.01, 'accuracy': 0.6437, 'tpr': 0.27, 'fpr': 0.083, 'precision': 0.3373, 'auc': 0.7701, 'pr_curve': 0.3284, 'train_time': 1.5967, 'infer_time': 0.6191}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Baseline', 'fold': 3, 'lr': 0.006, 'accuracy': 0.735, 'tpr': 0.2433, 'fpr': 0.0824, 'precision': 0.3074, 'auc': 0.7384, 'pr_curve': 0.312, 'train_time': 1.5621, 'infer_time': 0.1641}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Baseline', 'fold': 4, 'lr': 0.008, 'accuracy': 0.7352, 'tpr': 0.2862, 'fpr': 0.0782, 'precision': 0.2785, 'auc': 0.723, 'pr_curve': 0.3299, 'train_time': 1.5873, 'infer_time': 0.1979}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Baseline', 'fold': 5, 'lr': 0.002, 'accuracy': 0.6747, 'tpr': 0.2664, 'fpr': 0.0817, 'precision': 0.2814, 'auc': 0.7266, 'pr_curve': 0.2913, 'train_time': 1.6604, 'infer_time': 0.4359}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Baseline', 'fold': 6, 'lr': 0.008, 'accuracy': 0.7248, 'tpr': 0.316, 'fpr': 0.0767, 'precision': 0.3413, 'auc': 0.7651, 'pr_curve': 0.3393, 'train_time': 1.6266, 'infer_time': 0.4786}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Baseline', 'fold': 7, 'lr': 0.003, 'accuracy': 0.6496, 'tpr': 0.2979, 'fpr': 0.0771, 'precision': 0.3131, 'auc': 0.7738, 'pr_curve': 0.3433, 'train_time': 1.6155, 'infer_time': 0.1926}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Baseline', 'fold': 8, 'lr': 0.004, 'accuracy': 0.6816, 'tpr': 0.2843, 'fpr': 0.0822, 'precision': 0.2418, 'auc': 0.8002, 'pr_curve': 0.3877, 'train_time': 1.693, 'infer_time': 0.4579}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Baseline', 'fold': 9, 'lr': 0.004, 'accuracy': 0.6874, 'tpr': 0.2797, 'fpr': 0.0813, 'precision': 0.3823, 'auc': 0.7273, 'pr_curve': 0.2942, 'train_time': 1.4472, 'infer_time': 0.1909}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Baseline', 'fold': 10, 'lr': 0.008, 'accuracy': 0.661, 'tpr': 0.3161, 'fpr': 0.0778, 'precision': 0.2517, 'auc': 0.7313, 'pr_curve': 0.3143, 'train_time': 1.4746, 'infer_time': 0.1815}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Cyclic', 'fold': 1, 'lr': '0.001-0.02', 'accuracy': 0.6221, 'tpr': 0.2921, 'fpr': 0.0814, 'precision': 0.3063, 'auc': 0.7359, 'pr_curve': 0.2957, 'train_time': 1.5169, 'infer_time': 0.1982}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Cyclic', 'fold': 2, 'lr': '0.001-0.02', 'accuracy': 0.5689, 'tpr': 0.3269, 'fpr': 0.0758, 'precision': 0.3641, 'auc': 0.7603, 'pr_curve': 0.3653, 'train_time': 1.533, 'infer_time': 0.1901}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Cyclic', 'fold': 3, 'lr': '0.001-0.02', 'accuracy': 0.6262, 'tpr': 0.3088, 'fpr': 0.0796, 'precision': 0.271, 'auc': 0.7463, 'pr_curve': 0.3243, 'train_time': 1.4687, 'infer_time': 0.1909}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Cyclic', 'fold': 4, 'lr': '0.001-0.02', 'accuracy': 0.6065, 'tpr': 0.3063, 'fpr': 0.0768, 'precision': 0.3767, 'auc': 0.7315, 'pr_curve': 0.3089, 'train_time': 1.8279, 'infer_time': 0.3118}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Cyclic', 'fold': 5, 'lr': '0.001-0.02', 'accuracy': 0.5802, 'tpr': 0.3046, 'fpr': 0.0794, 'precision': 0.3165, 'auc': 0.7179, 'pr_curve': 0.3193, 'train_time': 1.4929, 'infer_time': 0.1881}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Cyclic', 'fold': 6, 'lr': '0.001-0.02', 'accuracy': 0.582, 'tpr': 0.3424, 'fpr': 0.08, 'precision': 0.2651, 'auc': 0.7714, 'pr_curve': 0.3343, 'train_time': 1.6084, 'infer_time': 0.1897}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Cyclic', 'fold': 7, 'lr': '0.001-0.02', 'accuracy': 0.5214, 'tpr': 0.2779, 'fpr': 0.0821, 'precision': 0.2895, 'auc': 0.753, 'pr_curve': 0.3145, 'train_time': 1.4945, 'infer_time': 0.7115}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Cyclic', 'fold': 8, 'lr': '0.001-0.02', 'accuracy': 0.4751, 'tpr': 0.1939, 'fpr': 0.0906, 'precision': 0.2056, 'auc': 0.7529, 'pr_curve': 0.2937, 'train_time': 1.4201, 'infer_time': 0.2126}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Cyclic', 'fold': 9, 'lr': '0.001-0.02', 'accuracy': 0.6115, 'tpr': 0.2842, 'fpr': 0.0768, 'precision': 0.2863, 'auc': 0.7562, 'pr_curve': 0.3356, 'train_time': 1.5593, 'infer_time': 0.6221}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Cyclic', 'fold': 10, 'lr': '0.001-0.02', 'accuracy': 0.6188, 'tpr': 0.2708, 'fpr': 0.0835, 'precision': 0.3155, 'auc': 0.7368, 'pr_curve': 0.2991, 'train_time': 1.6677, 'infer_time': 0.5179}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Ipmroved', 'fold': 1, 'lr': '0.002-0.05', 'accuracy': 0.5658, 'tpr': 0.2612, 'fpr': 0.0824, 'precision': 0.3297, 'auc': 0.6917, 'pr_curve': 0.2751, 'train_time': 1.4225, 'infer_time': 0.1879}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Ipmroved', 'fold': 2, 'lr': '0.002-0.05', 'accuracy': 0.5256, 'tpr': 0.2957, 'fpr': 0.0788, 'precision': 0.3105, 'auc': 0.768, 'pr_curve': 0.3619, 'train_time': 1.4149, 'infer_time': 0.1844}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Ipmroved', 'fold': 3, 'lr': '0.001-0.05', 'accuracy': 0.6203, 'tpr': 0.3017, 'fpr': 0.0745, 'precision': 0.3483, 'auc': 0.7631, 'pr_curve': 0.3441, 'train_time': 1.6121, 'infer_time': 0.1851}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Ipmroved', 'fold': 4, 'lr': '0.001-0.05', 'accuracy': 0.4788, 'tpr': 0.2532, 'fpr': 0.087, 'precision': 0.2664, 'auc': 0.6974, 'pr_curve': 0.2788, 'train_time': 1.4484, 'infer_time': 0.7188}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Ipmroved', 'fold': 5, 'lr': '0.001-0.05', 'accuracy': 0.526, 'tpr': 0.2796, 'fpr': 0.0807, 'precision': 0.3313, 'auc': 0.7243, 'pr_curve': 0.3215, 'train_time': 1.4404, 'infer_time': 0.7005}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Ipmroved', 'fold': 6, 'lr': '0.002-0.05', 'accuracy': 0.6786, 'tpr': 0.3539, 'fpr': 0.0765, 'precision': 0.31, 'auc': 0.8003, 'pr_curve': 0.3915, 'train_time': 1.5425, 'infer_time': 0.5982}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Ipmroved', 'fold': 7, 'lr': '0.002-0.05', 'accuracy': 0.5552, 'tpr': 0.2119, 'fpr': 0.0874, 'precision': 0.2211, 'auc': 0.7191, 'pr_curve': 0.2839, 'train_time': 1.3359, 'infer_time': 0.1635}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Ipmroved', 'fold': 8, 'lr': '0.001-0.05', 'accuracy': 0.5769, 'tpr': 0.2649, 'fpr': 0.0828, 'precision': 0.2666, 'auc': 0.7444, 'pr_curve': 0.3132, 'train_time': 1.3631, 'infer_time': 0.7471}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Ipmroved', 'fold': 9, 'lr': '0.001-0.05', 'accuracy': 0.5246, 'tpr': 0.2776, 'fpr': 0.0747, 'precision': 0.4056, 'auc': 0.7684, 'pr_curve': 0.3188, 'train_time': 1.3213, 'infer_time': 0.1876}, {'dataset name': 'cifar10_59000:60000', 'alg name': 'Ipmroved', 'fold': 10, 'lr': '0.001-0.05', 'accuracy': 0.5591, 'tpr': 0.3133, 'fpr': 0.0789, 'precision': 0.2995, 'auc': 0.724, 'pr_curve': 0.2923, 'train_time': 1.4847, 'infer_time': 0.636}]

test_datasets = [d1_scores, d2_scores, d3_scores, d4_scores, d5_scores, d6_scores, d7_scores, d8_scores, d9_scores, d10_scores]
test_rows = d1_scores + d2_scores + d3_scores + d4_scores + d5_scores + d6_scores + d7_scores + d8_scores + d9_scores + d10_scores

In [31]:
test_aucs = [ [np.mean([score["auc"] for score in dataset[i*10:(i+1)*10]]) for i in range(3) ] for dataset in test_datasets]
test_aucs

[[0.7672000000000001, 0.76543, 0.73004],
 [0.7383200000000001, 0.74924, 0.7549300000000001],
 [0.75329, 0.7595699999999999, 0.7523899999999999],
 [0.76243, 0.76091, 0.74236],
 [0.7603099999999999, 0.76981, 0.7370599999999999],
 [0.74667, 0.7358, 0.71757],
 [0.7477199999999999, 0.74664, 0.7336499999999999],
 [0.76738, 0.78081, 0.75242],
 [0.7165999999999999, 0.7376, 0.7094699999999999],
 [0.75142, 0.74622, 0.74007]]

In [19]:
# train_limit = 1000
# datasets = [{
#     "name": "{}_{}:{}".format(DATASET_NAME, i*train_limit, (i+1)*train_limit),
#     "X": c10_X_train[i*train_limit:(i+1)*train_limit],
#     "y": c10_y_train[i*train_limit:(i+1)*train_limit],
#     "n_classes": c10_classes,
#     "base_lr": 0.001,
#     "max_lr": 0.02,
#     "lr_step": 0.001,
#     "size": train_limit,
#   } for i in range(10)]

# train_rows, train_aucs = evaluate_all(datasets)
# print(train_rows)
# print(train_aucs)

In [20]:
d11_scores = [{'dataset name': 'cifar10_0:1000', 'alg name': 'Baseline', 'fold': 1, 'lr': 0.002, 'accuracy': 0.43, 'tpr': 0.4631, 'fpr': 0.06, 'precision': 0.4392, 'auc': 0.7603, 'pr_curve': 0.3948, 'train_time': 18.6934, 'infer_time': 0.3656}, {'dataset name': 'cifar10_0:1000', 'alg name': 'Baseline', 'fold': 2, 'lr': 0.004, 'accuracy': 0.41, 'tpr': 0.4427, 'fpr': 0.0613, 'precision': 0.4842, 'auc': 0.7996, 'pr_curve': 0.435, 'train_time': 3.7056, 'infer_time': 0.0924}, {'dataset name': 'cifar10_0:1000', 'alg name': 'Baseline', 'fold': 3, 'lr': 0.003, 'accuracy': 0.33, 'tpr': 0.3703, 'fpr': 0.0697, 'precision': 0.3534, 'auc': 0.8059, 'pr_curve': 0.3744, 'train_time': 25.446, 'infer_time': 0.4876}, {'dataset name': 'cifar10_0:1000', 'alg name': 'Baseline', 'fold': 4, 'lr': 0.002, 'accuracy': 0.28, 'tpr': 0.2773, 'fpr': 0.0841, 'precision': 0.2896, 'auc': 0.7464, 'pr_curve': 0.3566, 'train_time': 18.2401, 'infer_time': 0.2948}, {'dataset name': 'cifar10_0:1000', 'alg name': 'Baseline', 'fold': 5, 'lr': 0.005, 'accuracy': 0.29, 'tpr': 0.2943, 'fpr': 0.079, 'precision': 0.3196, 'auc': 0.7541, 'pr_curve': 0.301, 'train_time': 17.0256, 'infer_time': 0.3042}, {'dataset name': 'cifar10_0:1000', 'alg name': 'Baseline', 'fold': 6, 'lr': 0.004, 'accuracy': 0.38, 'tpr': 0.3531, 'fpr': 0.0693, 'precision': 0.3498, 'auc': 0.7662, 'pr_curve': 0.3596, 'train_time': 19.7395, 'infer_time': 0.3146}, {'dataset name': 'cifar10_0:1000', 'alg name': 'Baseline', 'fold': 7, 'lr': 0.002, 'accuracy': 0.33, 'tpr': 0.3405, 'fpr': 0.075, 'precision': 0.3268, 'auc': 0.7569, 'pr_curve': 0.3303, 'train_time': 4.9261, 'infer_time': 0.086}, {'dataset name': 'cifar10_0:1000', 'alg name': 'Baseline', 'fold': 8, 'lr': 0.004, 'accuracy': 0.27, 'tpr': 0.2958, 'fpr': 0.0795, 'precision': 0.2949, 'auc': 0.7198, 'pr_curve': 0.2668, 'train_time': 21.2608, 'infer_time': 0.2886}, {'dataset name': 'cifar10_0:1000', 'alg name': 'Baseline', 'fold': 9, 'lr': 0.001, 'accuracy': 0.29, 'tpr': 0.3151, 'fpr': 0.0775, 'precision': 0.2958, 'auc': 0.7356, 'pr_curve': 0.3028, 'train_time': 29.6089, 'infer_time': 0.3128}, {'dataset name': 'cifar10_0:1000', 'alg name': 'Baseline', 'fold': 10, 'lr': 0.004, 'accuracy': 0.31, 'tpr': 0.3353, 'fpr': 0.0745, 'precision': 0.4272, 'auc': 0.7543, 'pr_curve': 0.3368, 'train_time': 3.8501, 'infer_time': 0.0842}, {'dataset name': 'cifar10_0:1000', 'alg name': 'Cyclic', 'fold': 1, 'lr': '0.001-0.02', 'accuracy': 0.27, 'tpr': 0.3, 'fpr': 0.0807, 'precision': 0.2648, 'auc': 0.7252, 'pr_curve': 0.2923, 'train_time': 3.8859, 'infer_time': 0.0942}, {'dataset name': 'cifar10_0:1000', 'alg name': 'Cyclic', 'fold': 2, 'lr': '0.001-0.02', 'accuracy': 0.29, 'tpr': 0.3443, 'fpr': 0.0726, 'precision': 0.3205, 'auc': 0.7743, 'pr_curve': 0.3244, 'train_time': 4.2829, 'infer_time': 0.0866}, {'dataset name': 'cifar10_0:1000', 'alg name': 'Cyclic', 'fold': 3, 'lr': '0.001-0.02', 'accuracy': 0.24, 'tpr': 0.2593, 'fpr': 0.0851, 'precision': 0.3028, 'auc': 0.7377, 'pr_curve': 0.3074, 'train_time': 7.4598, 'infer_time': 0.2805}, {'dataset name': 'cifar10_0:1000', 'alg name': 'Cyclic', 'fold': 4, 'lr': '0.001-0.02', 'accuracy': 0.27, 'tpr': 0.3577, 'fpr': 0.0729, 'precision': 0.2704, 'auc': 0.723, 'pr_curve': 0.2857, 'train_time': 13.7741, 'infer_time': 0.273}, {'dataset name': 'cifar10_0:1000', 'alg name': 'Cyclic', 'fold': 5, 'lr': '0.001-0.02', 'accuracy': 0.33, 'tpr': 0.3345, 'fpr': 0.0746, 'precision': 0.3545, 'auc': 0.7941, 'pr_curve': 0.3467, 'train_time': 14.0013, 'infer_time': 0.2856}, {'dataset name': 'cifar10_0:1000', 'alg name': 'Cyclic', 'fold': 6, 'lr': '0.001-0.02', 'accuracy': 0.35, 'tpr': 0.3931, 'fpr': 0.0667, 'precision': 0.402, 'auc': 0.7945, 'pr_curve': 0.4108, 'train_time': 13.6792, 'infer_time': 0.272}, {'dataset name': 'cifar10_0:1000', 'alg name': 'Cyclic', 'fold': 7, 'lr': '0.001-0.02', 'accuracy': 0.28, 'tpr': 0.3101, 'fpr': 0.0808, 'precision': 0.3029, 'auc': 0.7403, 'pr_curve': 0.2959, 'train_time': 17.5328, 'infer_time': 0.2819}, {'dataset name': 'cifar10_0:1000', 'alg name': 'Cyclic', 'fold': 8, 'lr': '0.001-0.02', 'accuracy': 0.26, 'tpr': 0.2634, 'fpr': 0.0809, 'precision': 0.278, 'auc': 0.7139, 'pr_curve': 0.2976, 'train_time': 14.9833, 'infer_time': 0.2714}, {'dataset name': 'cifar10_0:1000', 'alg name': 'Cyclic', 'fold': 9, 'lr': '0.001-0.02', 'accuracy': 0.28, 'tpr': 0.2984, 'fpr': 0.078, 'precision': 0.284, 'auc': 0.7486, 'pr_curve': 0.3144, 'train_time': 13.6587, 'infer_time': 0.3102}, {'dataset name': 'cifar10_0:1000', 'alg name': 'Cyclic', 'fold': 10, 'lr': '0.001-0.02', 'accuracy': 0.29, 'tpr': 0.2457, 'fpr': 0.0824, 'precision': 0.2128, 'auc': 0.7442, 'pr_curve': 0.325, 'train_time': 14.7085, 'infer_time': 0.2951}, {'dataset name': 'cifar10_0:1000', 'alg name': 'Ipmroved', 'fold': 1, 'lr': '0.001-0.012', 'accuracy': 0.18, 'tpr': 0.3222, 'fpr': 0.0806, 'precision': 0.2362, 'auc': 0.7191, 'pr_curve': 0.3206, 'train_time': 11.6189, 'infer_time': 0.2787}, {'dataset name': 'cifar10_0:1000', 'alg name': 'Ipmroved', 'fold': 2, 'lr': '0.001-0.012', 'accuracy': 0.32, 'tpr': 0.4149, 'fpr': 0.069, 'precision': 0.302, 'auc': 0.7909, 'pr_curve': 0.368, 'train_time': 13.5542, 'infer_time': 0.2766}, {'dataset name': 'cifar10_0:1000', 'alg name': 'Ipmroved', 'fold': 3, 'lr': '0.001-0.012', 'accuracy': 0.33, 'tpr': 0.2961, 'fpr': 0.0721, 'precision': 0.2532, 'auc': 0.8257, 'pr_curve': 0.4064, 'train_time': 3.2881, 'infer_time': 0.0847}, {'dataset name': 'cifar10_0:1000', 'alg name': 'Ipmroved', 'fold': 4, 'lr': '0.0-0.013000000000000001', 'accuracy': 0.28, 'tpr': 0.3334, 'fpr': 0.0788, 'precision': 0.3458, 'auc': 0.7234, 'pr_curve': 0.3145, 'train_time': 4.0998, 'infer_time': 0.0879}, {'dataset name': 'cifar10_0:1000', 'alg name': 'Ipmroved', 'fold': 5, 'lr': '0.0-0.015', 'accuracy': 0.35, 'tpr': 0.319, 'fpr': 0.0723, 'precision': 0.3058, 'auc': 0.7661, 'pr_curve': 0.3279, 'train_time': 4.2068, 'infer_time': 0.1047}, {'dataset name': 'cifar10_0:1000', 'alg name': 'Ipmroved', 'fold': 6, 'lr': '0.001-0.014', 'accuracy': 0.28, 'tpr': 0.2696, 'fpr': 0.0789, 'precision': 0.3126, 'auc': 0.7322, 'pr_curve': 0.3054, 'train_time': 19.9728, 'infer_time': 0.2826}, {'dataset name': 'cifar10_0:1000', 'alg name': 'Ipmroved', 'fold': 7, 'lr': '0.001-0.013000000000000001', 'accuracy': 0.29, 'tpr': 0.2862, 'fpr': 0.0794, 'precision': 0.2947, 'auc': 0.7364, 'pr_curve': 0.3181, 'train_time': 16.1414, 'infer_time': 0.3149}, {'dataset name': 'cifar10_0:1000', 'alg name': 'Ipmroved', 'fold': 8, 'lr': '0.001-0.013000000000000001', 'accuracy': 0.3, 'tpr': 0.3259, 'fpr': 0.0758, 'precision': 0.2831, 'auc': 0.7481, 'pr_curve': 0.3254, 'train_time': 15.3378, 'infer_time': 0.2877}, {'dataset name': 'cifar10_0:1000', 'alg name': 'Ipmroved', 'fold': 9, 'lr': '0.0-0.012', 'accuracy': 0.31, 'tpr': 0.2994, 'fpr': 0.0761, 'precision': 0.2581, 'auc': 0.761, 'pr_curve': 0.3192, 'train_time': 9.182, 'infer_time': 0.084}, {'dataset name': 'cifar10_0:1000', 'alg name': 'Ipmroved', 'fold': 10, 'lr': '0.001-0.013000000000000001', 'accuracy': 0.33, 'tpr': 0.3314, 'fpr': 0.0776, 'precision': 0.3748, 'auc': 0.7877, 'pr_curve': 0.419, 'train_time': 3.3156, 'infer_time': 0.0827}]
d12_scores = [{'dataset name': 'cifar10_1000:2000', 'alg name': 'Baseline', 'fold': 1, 'lr': 0.001, 'accuracy': 0.21, 'tpr': 0.1948, 'fpr': 0.0871, 'precision': 0.2026, 'auc': 0.7733, 'pr_curve': 0.3365, 'train_time': 21.5311, 'infer_time': 0.2965}, {'dataset name': 'cifar10_1000:2000', 'alg name': 'Baseline', 'fold': 2, 'lr': 0.001, 'accuracy': 0.33, 'tpr': 0.3554, 'fpr': 0.0759, 'precision': 0.3016, 'auc': 0.8061, 'pr_curve': 0.4051, 'train_time': 27.5683, 'infer_time': 0.2975}, {'dataset name': 'cifar10_1000:2000', 'alg name': 'Baseline', 'fold': 3, 'lr': 0.003, 'accuracy': 0.29, 'tpr': 0.3001, 'fpr': 0.0793, 'precision': 0.3265, 'auc': 0.7579, 'pr_curve': 0.3444, 'train_time': 20.8895, 'infer_time': 0.2975}, {'dataset name': 'cifar10_1000:2000', 'alg name': 'Baseline', 'fold': 4, 'lr': 0.003, 'accuracy': 0.3, 'tpr': 0.2867, 'fpr': 0.0792, 'precision': 0.3119, 'auc': 0.773, 'pr_curve': 0.3976, 'train_time': 4.7445, 'infer_time': 0.0885}, {'dataset name': 'cifar10_1000:2000', 'alg name': 'Baseline', 'fold': 5, 'lr': 0.002, 'accuracy': 0.4, 'tpr': 0.4385, 'fpr': 0.0616, 'precision': 0.5165, 'auc': 0.796, 'pr_curve': 0.436, 'train_time': 16.8495, 'infer_time': 0.2581}, {'dataset name': 'cifar10_1000:2000', 'alg name': 'Baseline', 'fold': 6, 'lr': 0.002, 'accuracy': 0.38, 'tpr': 0.3868, 'fpr': 0.0684, 'precision': 0.3863, 'auc': 0.7871, 'pr_curve': 0.3966, 'train_time': 5.5015, 'infer_time': 0.1161}, {'dataset name': 'cifar10_1000:2000', 'alg name': 'Baseline', 'fold': 7, 'lr': 0.002, 'accuracy': 0.33, 'tpr': 0.3222, 'fpr': 0.0745, 'precision': 0.3165, 'auc': 0.7551, 'pr_curve': 0.3877, 'train_time': 3.5976, 'infer_time': 0.0854}, {'dataset name': 'cifar10_1000:2000', 'alg name': 'Baseline', 'fold': 8, 'lr': 0.001, 'accuracy': 0.32, 'tpr': 0.373, 'fpr': 0.0718, 'precision': 0.3303, 'auc': 0.7784, 'pr_curve': 0.3689, 'train_time': 23.2941, 'infer_time': 0.2882}, {'dataset name': 'cifar10_1000:2000', 'alg name': 'Baseline', 'fold': 9, 'lr': 0.001, 'accuracy': 0.35, 'tpr': 0.4303, 'fpr': 0.0717, 'precision': 0.3815, 'auc': 0.7751, 'pr_curve': 0.3528, 'train_time': 27.1616, 'infer_time': 0.2896}, {'dataset name': 'cifar10_1000:2000', 'alg name': 'Baseline', 'fold': 10, 'lr': 0.005, 'accuracy': 0.36, 'tpr': 0.3941, 'fpr': 0.0698, 'precision': 0.3743, 'auc': 0.7836, 'pr_curve': 0.4125, 'train_time': 4.7568, 'infer_time': 0.0914}, {'dataset name': 'cifar10_1000:2000', 'alg name': 'Cyclic', 'fold': 1, 'lr': '0.001-0.02', 'accuracy': 0.32, 'tpr': 0.3548, 'fpr': 0.0715, 'precision': 0.3441, 'auc': 0.7703, 'pr_curve': 0.3893, 'train_time': 4.0702, 'infer_time': 0.0888}, {'dataset name': 'cifar10_1000:2000', 'alg name': 'Cyclic', 'fold': 2, 'lr': '0.001-0.02', 'accuracy': 0.35, 'tpr': 0.3536, 'fpr': 0.0669, 'precision': 0.4093, 'auc': 0.75, 'pr_curve': 0.3501, 'train_time': 5.2956, 'infer_time': 0.1116}, {'dataset name': 'cifar10_1000:2000', 'alg name': 'Cyclic', 'fold': 3, 'lr': '0.001-0.02', 'accuracy': 0.31, 'tpr': 0.2897, 'fpr': 0.0765, 'precision': 0.2403, 'auc': 0.7171, 'pr_curve': 0.3116, 'train_time': 15.2291, 'infer_time': 0.268}, {'dataset name': 'cifar10_1000:2000', 'alg name': 'Cyclic', 'fold': 4, 'lr': '0.001-0.02', 'accuracy': 0.4, 'tpr': 0.4356, 'fpr': 0.0638, 'precision': 0.3962, 'auc': 0.7951, 'pr_curve': 0.3974, 'train_time': 13.9636, 'infer_time': 0.2889}, {'dataset name': 'cifar10_1000:2000', 'alg name': 'Cyclic', 'fold': 5, 'lr': '0.001-0.02', 'accuracy': 0.38, 'tpr': 0.4136, 'fpr': 0.0654, 'precision': 0.3852, 'auc': 0.7941, 'pr_curve': 0.3987, 'train_time': 16.1857, 'infer_time': 0.2753}, {'dataset name': 'cifar10_1000:2000', 'alg name': 'Cyclic', 'fold': 6, 'lr': '0.001-0.02', 'accuracy': 0.42, 'tpr': 0.4043, 'fpr': 0.0653, 'precision': 0.4091, 'auc': 0.8065, 'pr_curve': 0.4562, 'train_time': 13.008, 'infer_time': 0.2997}, {'dataset name': 'cifar10_1000:2000', 'alg name': 'Cyclic', 'fold': 7, 'lr': '0.001-0.02', 'accuracy': 0.35, 'tpr': 0.3407, 'fpr': 0.0726, 'precision': 0.3473, 'auc': 0.7748, 'pr_curve': 0.4022, 'train_time': 17.7629, 'infer_time': 0.4037}, {'dataset name': 'cifar10_1000:2000', 'alg name': 'Cyclic', 'fold': 8, 'lr': '0.001-0.02', 'accuracy': 0.22, 'tpr': 0.2826, 'fpr': 0.0824, 'precision': 0.3685, 'auc': 0.7165, 'pr_curve': 0.3321, 'train_time': 12.6761, 'infer_time': 0.2808}, {'dataset name': 'cifar10_1000:2000', 'alg name': 'Cyclic', 'fold': 9, 'lr': '0.001-0.02', 'accuracy': 0.25, 'tpr': 0.2995, 'fpr': 0.0808, 'precision': 0.2821, 'auc': 0.7298, 'pr_curve': 0.2738, 'train_time': 17.6929, 'infer_time': 0.2615}, {'dataset name': 'cifar10_1000:2000', 'alg name': 'Cyclic', 'fold': 10, 'lr': '0.001-0.02', 'accuracy': 0.37, 'tpr': 0.4157, 'fpr': 0.0696, 'precision': 0.4418, 'auc': 0.7895, 'pr_curve': 0.4414, 'train_time': 13.9592, 'infer_time': 0.2919}, {'dataset name': 'cifar10_1000:2000', 'alg name': 'Ipmroved', 'fold': 1, 'lr': '0.001-0.012', 'accuracy': 0.19, 'tpr': 0.1963, 'fpr': 0.0891, 'precision': 0.2056, 'auc': 0.7454, 'pr_curve': 0.297, 'train_time': 16.7836, 'infer_time': 0.3541}, {'dataset name': 'cifar10_1000:2000', 'alg name': 'Ipmroved', 'fold': 2, 'lr': '0.001-0.015', 'accuracy': 0.3, 'tpr': 0.3678, 'fpr': 0.0719, 'precision': 0.332, 'auc': 0.7425, 'pr_curve': 0.3427, 'train_time': 5.027, 'infer_time': 0.0896}, {'dataset name': 'cifar10_1000:2000', 'alg name': 'Ipmroved', 'fold': 3, 'lr': '0.001-0.014', 'accuracy': 0.25, 'tpr': 0.2421, 'fpr': 0.0867, 'precision': 0.3675, 'auc': 0.6943, 'pr_curve': 0.3191, 'train_time': 3.288, 'infer_time': 0.0825}, {'dataset name': 'cifar10_1000:2000', 'alg name': 'Ipmroved', 'fold': 4, 'lr': '0.001-0.016', 'accuracy': 0.25, 'tpr': 0.2603, 'fpr': 0.0851, 'precision': 0.3267, 'auc': 0.7365, 'pr_curve': 0.3336, 'train_time': 3.5754, 'infer_time': 0.0849}, {'dataset name': 'cifar10_1000:2000', 'alg name': 'Ipmroved', 'fold': 5, 'lr': '0.001-0.015', 'accuracy': 0.36, 'tpr': 0.4442, 'fpr': 0.0622, 'precision': 0.5184, 'auc': 0.7776, 'pr_curve': 0.3849, 'train_time': 16.4566, 'infer_time': 0.2839}, {'dataset name': 'cifar10_1000:2000', 'alg name': 'Ipmroved', 'fold': 6, 'lr': '0.001-0.013000000000000001', 'accuracy': 0.41, 'tpr': 0.4094, 'fpr': 0.0643, 'precision': 0.5412, 'auc': 0.8198, 'pr_curve': 0.4668, 'train_time': 13.528, 'infer_time': 0.2703}, {'dataset name': 'cifar10_1000:2000', 'alg name': 'Ipmroved', 'fold': 7, 'lr': '0.0-0.015', 'accuracy': 0.41, 'tpr': 0.3934, 'fpr': 0.0661, 'precision': 0.3682, 'auc': 0.763, 'pr_curve': 0.3995, 'train_time': 16.0781, 'infer_time': 0.2602}, {'dataset name': 'cifar10_1000:2000', 'alg name': 'Ipmroved', 'fold': 8, 'lr': '0.001-0.011', 'accuracy': 0.38, 'tpr': 0.4225, 'fpr': 0.0655, 'precision': 0.4091, 'auc': 0.7863, 'pr_curve': 0.3832, 'train_time': 18.8203, 'infer_time': 0.2919}, {'dataset name': 'cifar10_1000:2000', 'alg name': 'Ipmroved', 'fold': 9, 'lr': '0.001-0.011', 'accuracy': 0.34, 'tpr': 0.3425, 'fpr': 0.0734, 'precision': 0.4339, 'auc': 0.7779, 'pr_curve': 0.392, 'train_time': 3.6174, 'infer_time': 0.0859}, {'dataset name': 'cifar10_1000:2000', 'alg name': 'Ipmroved', 'fold': 10, 'lr': '0.001-0.012', 'accuracy': 0.39, 'tpr': 0.4402, 'fpr': 0.0651, 'precision': 0.4472, 'auc': 0.8075, 'pr_curve': 0.4645, 'train_time': 3.8239, 'infer_time': 0.0823}]
d13_scores = [{'dataset name': 'cifar10_2000:3000', 'alg name': 'Baseline', 'fold': 1, 'lr': 0.002, 'accuracy': 0.35, 'tpr': 0.3827, 'fpr': 0.0724, 'precision': 0.339, 'auc': 0.786, 'pr_curve': 0.4015, 'train_time': 20.1794, 'infer_time': 0.2745}, {'dataset name': 'cifar10_2000:3000', 'alg name': 'Baseline', 'fold': 2, 'lr': 0.003, 'accuracy': 0.34, 'tpr': 0.4028, 'fpr': 0.0692, 'precision': 0.4313, 'auc': 0.7962, 'pr_curve': 0.4002, 'train_time': 5.0221, 'infer_time': 0.1169}, {'dataset name': 'cifar10_2000:3000', 'alg name': 'Baseline', 'fold': 3, 'lr': 0.001, 'accuracy': 0.33, 'tpr': 0.3521, 'fpr': 0.0714, 'precision': 0.3299, 'auc': 0.7883, 'pr_curve': 0.3283, 'train_time': 6.2492, 'infer_time': 0.082}, {'dataset name': 'cifar10_2000:3000', 'alg name': 'Baseline', 'fold': 4, 'lr': 0.002, 'accuracy': 0.35, 'tpr': 0.3559, 'fpr': 0.0698, 'precision': 0.3102, 'auc': 0.7709, 'pr_curve': 0.3469, 'train_time': 17.6478, 'infer_time': 0.2857}, {'dataset name': 'cifar10_2000:3000', 'alg name': 'Baseline', 'fold': 5, 'lr': 0.002, 'accuracy': 0.4, 'tpr': 0.4288, 'fpr': 0.0648, 'precision': 0.412, 'auc': 0.8097, 'pr_curve': 0.393, 'train_time': 4.3448, 'infer_time': 0.0848}, {'dataset name': 'cifar10_2000:3000', 'alg name': 'Baseline', 'fold': 6, 'lr': 0.003, 'accuracy': 0.28, 'tpr': 0.2966, 'fpr': 0.0801, 'precision': 0.3241, 'auc': 0.732, 'pr_curve': 0.2803, 'train_time': 17.6713, 'infer_time': 0.2971}, {'dataset name': 'cifar10_2000:3000', 'alg name': 'Baseline', 'fold': 7, 'lr': 0.003, 'accuracy': 0.27, 'tpr': 0.2974, 'fpr': 0.0801, 'precision': 0.2957, 'auc': 0.7598, 'pr_curve': 0.3489, 'train_time': 3.317, 'infer_time': 0.0886}, {'dataset name': 'cifar10_2000:3000', 'alg name': 'Baseline', 'fold': 8, 'lr': 0.001, 'accuracy': 0.28, 'tpr': 0.3121, 'fpr': 0.0776, 'precision': 0.3302, 'auc': 0.7534, 'pr_curve': 0.3138, 'train_time': 26.7857, 'infer_time': 0.2931}, {'dataset name': 'cifar10_2000:3000', 'alg name': 'Baseline', 'fold': 9, 'lr': 0.001, 'accuracy': 0.44, 'tpr': 0.4477, 'fpr': 0.0583, 'precision': 0.4626, 'auc': 0.8126, 'pr_curve': 0.4687, 'train_time': 33.3122, 'infer_time': 0.2791}, {'dataset name': 'cifar10_2000:3000', 'alg name': 'Baseline', 'fold': 10, 'lr': 0.001, 'accuracy': 0.34, 'tpr': 0.3739, 'fpr': 0.0708, 'precision': 0.3415, 'auc': 0.7502, 'pr_curve': 0.3188, 'train_time': 5.7148, 'infer_time': 0.0903}, {'dataset name': 'cifar10_2000:3000', 'alg name': 'Cyclic', 'fold': 1, 'lr': '0.001-0.02', 'accuracy': 0.2, 'tpr': 0.205, 'fpr': 0.0881, 'precision': 0.2159, 'auc': 0.6683, 'pr_curve': 0.2347, 'train_time': 3.4792, 'infer_time': 0.0933}, {'dataset name': 'cifar10_2000:3000', 'alg name': 'Cyclic', 'fold': 2, 'lr': '0.001-0.02', 'accuracy': 0.4, 'tpr': 0.4284, 'fpr': 0.0666, 'precision': 0.4265, 'auc': 0.8208, 'pr_curve': 0.458, 'train_time': 4.0391, 'infer_time': 0.1028}, {'dataset name': 'cifar10_2000:3000', 'alg name': 'Cyclic', 'fold': 3, 'lr': '0.001-0.02', 'accuracy': 0.25, 'tpr': 0.2763, 'fpr': 0.0787, 'precision': 0.3413, 'auc': 0.7101, 'pr_curve': 0.3121, 'train_time': 6.2812, 'infer_time': 0.2621}, {'dataset name': 'cifar10_2000:3000', 'alg name': 'Cyclic', 'fold': 4, 'lr': '0.001-0.02', 'accuracy': 0.39, 'tpr': 0.3656, 'fpr': 0.0697, 'precision': 0.3475, 'auc': 0.7881, 'pr_curve': 0.386, 'train_time': 15.0792, 'infer_time': 0.2834}, {'dataset name': 'cifar10_2000:3000', 'alg name': 'Cyclic', 'fold': 5, 'lr': '0.001-0.02', 'accuracy': 0.34, 'tpr': 0.2913, 'fpr': 0.0779, 'precision': 0.2596, 'auc': 0.7926, 'pr_curve': 0.3728, 'train_time': 13.5491, 'infer_time': 0.3019}, {'dataset name': 'cifar10_2000:3000', 'alg name': 'Cyclic', 'fold': 6, 'lr': '0.001-0.02', 'accuracy': 0.23, 'tpr': 0.265, 'fpr': 0.0839, 'precision': 0.2569, 'auc': 0.7511, 'pr_curve': 0.3402, 'train_time': 13.6047, 'infer_time': 0.2675}, {'dataset name': 'cifar10_2000:3000', 'alg name': 'Cyclic', 'fold': 7, 'lr': '0.001-0.02', 'accuracy': 0.31, 'tpr': 0.3148, 'fpr': 0.0756, 'precision': 0.3077, 'auc': 0.7821, 'pr_curve': 0.3423, 'train_time': 13.4509, 'infer_time': 0.2752}, {'dataset name': 'cifar10_2000:3000', 'alg name': 'Cyclic', 'fold': 8, 'lr': '0.001-0.02', 'accuracy': 0.26, 'tpr': 0.2919, 'fpr': 0.0823, 'precision': 0.3011, 'auc': 0.715, 'pr_curve': 0.3061, 'train_time': 12.472, 'infer_time': 0.2949}, {'dataset name': 'cifar10_2000:3000', 'alg name': 'Cyclic', 'fold': 9, 'lr': '0.001-0.02', 'accuracy': 0.31, 'tpr': 0.3336, 'fpr': 0.0727, 'precision': 0.3427, 'auc': 0.7427, 'pr_curve': 0.3331, 'train_time': 14.8608, 'infer_time': 0.2931}, {'dataset name': 'cifar10_2000:3000', 'alg name': 'Cyclic', 'fold': 10, 'lr': '0.001-0.02', 'accuracy': 0.28, 'tpr': 0.4683, 'fpr': 0.0655, 'precision': 0.4836, 'auc': 0.7105, 'pr_curve': 0.3204, 'train_time': 13.79, 'infer_time': 0.2648}, {'dataset name': 'cifar10_2000:3000', 'alg name': 'Ipmroved', 'fold': 1, 'lr': '0.001-0.015', 'accuracy': 0.29, 'tpr': 0.3146, 'fpr': 0.0784, 'precision': 0.3074, 'auc': 0.7321, 'pr_curve': 0.272, 'train_time': 13.447, 'infer_time': 0.2801}, {'dataset name': 'cifar10_2000:3000', 'alg name': 'Ipmroved', 'fold': 2, 'lr': '0.001-0.013000000000000001', 'accuracy': 0.33, 'tpr': 0.3865, 'fpr': 0.0736, 'precision': 0.445, 'auc': 0.7887, 'pr_curve': 0.3973, 'train_time': 10.7223, 'infer_time': 0.2731}, {'dataset name': 'cifar10_2000:3000', 'alg name': 'Ipmroved', 'fold': 3, 'lr': '0.001-0.014', 'accuracy': 0.37, 'tpr': 0.3793, 'fpr': 0.0683, 'precision': 0.3913, 'auc': 0.7893, 'pr_curve': 0.4011, 'train_time': 4.5932, 'infer_time': 0.0838}, {'dataset name': 'cifar10_2000:3000', 'alg name': 'Ipmroved', 'fold': 4, 'lr': '0.001-0.013000000000000001', 'accuracy': 0.28, 'tpr': 0.2567, 'fpr': 0.0827, 'precision': 0.2595, 'auc': 0.7702, 'pr_curve': 0.3347, 'train_time': 3.6104, 'infer_time': 0.0904}, {'dataset name': 'cifar10_2000:3000', 'alg name': 'Ipmroved', 'fold': 5, 'lr': '0.0-0.012', 'accuracy': 0.37, 'tpr': 0.4167, 'fpr': 0.0676, 'precision': 0.5204, 'auc': 0.8101, 'pr_curve': 0.418, 'train_time': 5.0242, 'infer_time': 0.1171}, {'dataset name': 'cifar10_2000:3000', 'alg name': 'Ipmroved', 'fold': 6, 'lr': '0.0-0.013000000000000001', 'accuracy': 0.23, 'tpr': 0.301, 'fpr': 0.0809, 'precision': 0.3986, 'auc': 0.711, 'pr_curve': 0.3025, 'train_time': 13.8564, 'infer_time': 0.2825}, {'dataset name': 'cifar10_2000:3000', 'alg name': 'Ipmroved', 'fold': 7, 'lr': '0.001-0.014', 'accuracy': 0.28, 'tpr': 0.3021, 'fpr': 0.0797, 'precision': 0.3575, 'auc': 0.7613, 'pr_curve': 0.3326, 'train_time': 16.7148, 'infer_time': 0.2792}, {'dataset name': 'cifar10_2000:3000', 'alg name': 'Ipmroved', 'fold': 8, 'lr': '0.001-0.013000000000000001', 'accuracy': 0.2, 'tpr': 0.2077, 'fpr': 0.0895, 'precision': 0.2302, 'auc': 0.7222, 'pr_curve': 0.3032, 'train_time': 13.8018, 'infer_time': 0.2892}, {'dataset name': 'cifar10_2000:3000', 'alg name': 'Ipmroved', 'fold': 9, 'lr': '0.001-0.013000000000000001', 'accuracy': 0.41, 'tpr': 0.3864, 'fpr': 0.0649, 'precision': 0.3852, 'auc': 0.7801, 'pr_curve': 0.4033, 'train_time': 12.7608, 'infer_time': 0.0891}, {'dataset name': 'cifar10_2000:3000', 'alg name': 'Ipmroved', 'fold': 10, 'lr': '0.001-0.014', 'accuracy': 0.21, 'tpr': 0.2945, 'fpr': 0.081, 'precision': 0.2652, 'auc': 0.6699, 'pr_curve': 0.2866, 'train_time': 3.6534, 'infer_time': 0.0868}]
d14_scores = [{'dataset name': 'cifar10_3000:4000', 'alg name': 'Baseline', 'fold': 1, 'lr': 0.002, 'accuracy': 0.31, 'tpr': 0.2746, 'fpr': 0.0832, 'precision': 0.2532, 'auc': 0.7569, 'pr_curve': 0.343, 'train_time': 21.352, 'infer_time': 0.2813}, {'dataset name': 'cifar10_3000:4000', 'alg name': 'Baseline', 'fold': 2, 'lr': 0.005, 'accuracy': 0.29, 'tpr': 0.3012, 'fpr': 0.078, 'precision': 0.2731, 'auc': 0.6786, 'pr_curve': 0.2509, 'train_time': 4.0808, 'infer_time': 0.0835}, {'dataset name': 'cifar10_3000:4000', 'alg name': 'Baseline', 'fold': 3, 'lr': 0.001, 'accuracy': 0.37, 'tpr': 0.4052, 'fpr': 0.0677, 'precision': 0.4324, 'auc': 0.7916, 'pr_curve': 0.3705, 'train_time': 23.222, 'infer_time': 0.3375}, {'dataset name': 'cifar10_3000:4000', 'alg name': 'Baseline', 'fold': 4, 'lr': 0.004, 'accuracy': 0.29, 'tpr': 0.3318, 'fpr': 0.0756, 'precision': 0.3049, 'auc': 0.7574, 'pr_curve': 0.325, 'train_time': 18.1502, 'infer_time': 0.3038}, {'dataset name': 'cifar10_3000:4000', 'alg name': 'Baseline', 'fold': 5, 'lr': 0.003, 'accuracy': 0.3, 'tpr': 0.323, 'fpr': 0.0776, 'precision': 0.3332, 'auc': 0.7231, 'pr_curve': 0.2975, 'train_time': 4.3784, 'infer_time': 0.0821}, {'dataset name': 'cifar10_3000:4000', 'alg name': 'Baseline', 'fold': 6, 'lr': 0.002, 'accuracy': 0.28, 'tpr': 0.2875, 'fpr': 0.0822, 'precision': 0.2682, 'auc': 0.7643, 'pr_curve': 0.2958, 'train_time': 17.6532, 'infer_time': 0.3442}, {'dataset name': 'cifar10_3000:4000', 'alg name': 'Baseline', 'fold': 7, 'lr': 0.004, 'accuracy': 0.3, 'tpr': 0.3237, 'fpr': 0.0773, 'precision': 0.3274, 'auc': 0.7201, 'pr_curve': 0.3283, 'train_time': 23.5125, 'infer_time': 0.2893}, {'dataset name': 'cifar10_3000:4000', 'alg name': 'Baseline', 'fold': 8, 'lr': 0.001, 'accuracy': 0.33, 'tpr': 0.3148, 'fpr': 0.0759, 'precision': 0.3021, 'auc': 0.7736, 'pr_curve': 0.3564, 'train_time': 6.5658, 'infer_time': 0.0873}, {'dataset name': 'cifar10_3000:4000', 'alg name': 'Baseline', 'fold': 9, 'lr': 0.003, 'accuracy': 0.37, 'tpr': 0.3883, 'fpr': 0.0675, 'precision': 0.425, 'auc': 0.7517, 'pr_curve': 0.3577, 'train_time': 19.0394, 'infer_time': 0.2971}, {'dataset name': 'cifar10_3000:4000', 'alg name': 'Baseline', 'fold': 10, 'lr': 0.002, 'accuracy': 0.31, 'tpr': 0.3611, 'fpr': 0.0729, 'precision': 0.3262, 'auc': 0.7607, 'pr_curve': 0.3712, 'train_time': 21.6659, 'infer_time': 0.2798}, {'dataset name': 'cifar10_3000:4000', 'alg name': 'Cyclic', 'fold': 1, 'lr': '0.001-0.02', 'accuracy': 0.21, 'tpr': 0.1578, 'fpr': 0.0941, 'precision': 0.2988, 'auc': 0.6433, 'pr_curve': 0.2088, 'train_time': 8.3585, 'infer_time': 0.2937}, {'dataset name': 'cifar10_3000:4000', 'alg name': 'Cyclic', 'fold': 2, 'lr': '0.001-0.02', 'accuracy': 0.3, 'tpr': 0.2912, 'fpr': 0.0813, 'precision': 0.279, 'auc': 0.7086, 'pr_curve': 0.2964, 'train_time': 17.5993, 'infer_time': 0.2777}, {'dataset name': 'cifar10_3000:4000', 'alg name': 'Cyclic', 'fold': 3, 'lr': '0.001-0.02', 'accuracy': 0.25, 'tpr': 0.2701, 'fpr': 0.0815, 'precision': 0.335, 'auc': 0.7161, 'pr_curve': 0.2984, 'train_time': 15.2148, 'infer_time': 0.2773}, {'dataset name': 'cifar10_3000:4000', 'alg name': 'Cyclic', 'fold': 4, 'lr': '0.001-0.02', 'accuracy': 0.18, 'tpr': 0.2263, 'fpr': 0.0891, 'precision': 0.2655, 'auc': 0.735, 'pr_curve': 0.3164, 'train_time': 13.9462, 'infer_time': 0.3053}, {'dataset name': 'cifar10_3000:4000', 'alg name': 'Cyclic', 'fold': 5, 'lr': '0.001-0.02', 'accuracy': 0.26, 'tpr': 0.2552, 'fpr': 0.0842, 'precision': 0.3895, 'auc': 0.7454, 'pr_curve': 0.3281, 'train_time': 15.3031, 'infer_time': 0.3108}, {'dataset name': 'cifar10_3000:4000', 'alg name': 'Cyclic', 'fold': 6, 'lr': '0.001-0.02', 'accuracy': 0.31, 'tpr': 0.2761, 'fpr': 0.0798, 'precision': 0.3596, 'auc': 0.7493, 'pr_curve': 0.3329, 'train_time': 15.5993, 'infer_time': 0.275}, {'dataset name': 'cifar10_3000:4000', 'alg name': 'Cyclic', 'fold': 7, 'lr': '0.001-0.02', 'accuracy': 0.26, 'tpr': 0.2423, 'fpr': 0.0831, 'precision': 0.3014, 'auc': 0.6996, 'pr_curve': 0.3193, 'train_time': 16.8242, 'infer_time': 0.3207}, {'dataset name': 'cifar10_3000:4000', 'alg name': 'Cyclic', 'fold': 8, 'lr': '0.001-0.02', 'accuracy': 0.32, 'tpr': 0.3087, 'fpr': 0.0764, 'precision': 0.3287, 'auc': 0.7707, 'pr_curve': 0.3382, 'train_time': 17.7736, 'infer_time': 0.2847}, {'dataset name': 'cifar10_3000:4000', 'alg name': 'Cyclic', 'fold': 9, 'lr': '0.001-0.02', 'accuracy': 0.36, 'tpr': 0.3788, 'fpr': 0.0696, 'precision': 0.4201, 'auc': 0.7624, 'pr_curve': 0.3813, 'train_time': 18.6361, 'infer_time': 0.2982}, {'dataset name': 'cifar10_3000:4000', 'alg name': 'Cyclic', 'fold': 10, 'lr': '0.001-0.02', 'accuracy': 0.29, 'tpr': 0.3508, 'fpr': 0.0724, 'precision': 0.4398, 'auc': 0.723, 'pr_curve': 0.3383, 'train_time': 19.296, 'infer_time': 0.3015}, {'dataset name': 'cifar10_3000:4000', 'alg name': 'Ipmroved', 'fold': 1, 'lr': '0.001-0.014', 'accuracy': 0.29, 'tpr': 0.3688, 'fpr': 0.0756, 'precision': 0.4449, 'auc': 0.7876, 'pr_curve': 0.3359, 'train_time': 10.621, 'infer_time': 0.0867}, {'dataset name': 'cifar10_3000:4000', 'alg name': 'Ipmroved', 'fold': 2, 'lr': '0.001-0.01', 'accuracy': 0.32, 'tpr': 0.3329, 'fpr': 0.0753, 'precision': 0.3289, 'auc': 0.7478, 'pr_curve': 0.3439, 'train_time': 3.9756, 'infer_time': 0.0897}, {'dataset name': 'cifar10_3000:4000', 'alg name': 'Ipmroved', 'fold': 3, 'lr': '0.001-0.014', 'accuracy': 0.36, 'tpr': 0.411, 'fpr': 0.0674, 'precision': 0.3848, 'auc': 0.7701, 'pr_curve': 0.3585, 'train_time': 3.5736, 'infer_time': 0.0942}, {'dataset name': 'cifar10_3000:4000', 'alg name': 'Ipmroved', 'fold': 4, 'lr': '0.001-0.012', 'accuracy': 0.25, 'tpr': 0.3205, 'fpr': 0.0776, 'precision': 0.3921, 'auc': 0.7313, 'pr_curve': 0.3444, 'train_time': 12.0671, 'infer_time': 0.2674}, {'dataset name': 'cifar10_3000:4000', 'alg name': 'Ipmroved', 'fold': 5, 'lr': '0.0-0.014', 'accuracy': 0.26, 'tpr': 0.2637, 'fpr': 0.0801, 'precision': 0.2055, 'auc': 0.7567, 'pr_curve': 0.3372, 'train_time': 17.7041, 'infer_time': 0.3054}, {'dataset name': 'cifar10_3000:4000', 'alg name': 'Ipmroved', 'fold': 6, 'lr': '0.001-0.014', 'accuracy': 0.33, 'tpr': 0.3662, 'fpr': 0.0752, 'precision': 0.4419, 'auc': 0.7399, 'pr_curve': 0.3434, 'train_time': 15.091, 'infer_time': 0.298}, {'dataset name': 'cifar10_3000:4000', 'alg name': 'Ipmroved', 'fold': 7, 'lr': '0.001-0.013000000000000001', 'accuracy': 0.26, 'tpr': 0.2835, 'fpr': 0.0817, 'precision': 0.3278, 'auc': 0.7079, 'pr_curve': 0.2953, 'train_time': 18.778, 'infer_time': 0.306}, {'dataset name': 'cifar10_3000:4000', 'alg name': 'Ipmroved', 'fold': 8, 'lr': '0.0-0.016', 'accuracy': 0.3, 'tpr': 0.2778, 'fpr': 0.0806, 'precision': 0.2853, 'auc': 0.7329, 'pr_curve': 0.3191, 'train_time': 3.864, 'infer_time': 0.0896}, {'dataset name': 'cifar10_3000:4000', 'alg name': 'Ipmroved', 'fold': 9, 'lr': '0.001-0.012', 'accuracy': 0.31, 'tpr': 0.3657, 'fpr': 0.0747, 'precision': 0.3226, 'auc': 0.7609, 'pr_curve': 0.3763, 'train_time': 3.6194, 'infer_time': 0.0845}, {'dataset name': 'cifar10_3000:4000', 'alg name': 'Ipmroved', 'fold': 10, 'lr': '0.001-0.015', 'accuracy': 0.36, 'tpr': 0.3934, 'fpr': 0.0671, 'precision': 0.3829, 'auc': 0.7305, 'pr_curve': 0.3177, 'train_time': 5.3525, 'infer_time': 0.0851}]
d15_scores = [{'dataset name': 'cifar10_4000:5000', 'alg name': 'Baseline', 'fold': 1, 'lr': 0.004, 'accuracy': 0.37, 'tpr': 0.357, 'fpr': 0.0736, 'precision': 0.3239, 'auc': 0.7735, 'pr_curve': 0.3461, 'train_time': 5.2831, 'infer_time': 0.0845}, {'dataset name': 'cifar10_4000:5000', 'alg name': 'Baseline', 'fold': 2, 'lr': 0.003, 'accuracy': 0.38, 'tpr': 0.4323, 'fpr': 0.0669, 'precision': 0.4348, 'auc': 0.8021, 'pr_curve': 0.4366, 'train_time': 18.8623, 'infer_time': 0.3348}, {'dataset name': 'cifar10_4000:5000', 'alg name': 'Baseline', 'fold': 3, 'lr': 0.002, 'accuracy': 0.36, 'tpr': 0.3343, 'fpr': 0.07, 'precision': 0.3707, 'auc': 0.7751, 'pr_curve': 0.3946, 'train_time': 4.6311, 'infer_time': 0.0959}, {'dataset name': 'cifar10_4000:5000', 'alg name': 'Baseline', 'fold': 4, 'lr': 0.006, 'accuracy': 0.38, 'tpr': 0.4078, 'fpr': 0.0685, 'precision': 0.3927, 'auc': 0.7542, 'pr_curve': 0.373, 'train_time': 5.4075, 'infer_time': 0.0934}, {'dataset name': 'cifar10_4000:5000', 'alg name': 'Baseline', 'fold': 5, 'lr': 0.001, 'accuracy': 0.31, 'tpr': 0.3732, 'fpr': 0.074, 'precision': 0.3377, 'auc': 0.7771, 'pr_curve': 0.3572, 'train_time': 18.975, 'infer_time': 0.2649}, {'dataset name': 'cifar10_4000:5000', 'alg name': 'Baseline', 'fold': 6, 'lr': 0.005, 'accuracy': 0.36, 'tpr': 0.3198, 'fpr': 0.0722, 'precision': 0.2984, 'auc': 0.7728, 'pr_curve': 0.3517, 'train_time': 15.4068, 'infer_time': 0.2833}, {'dataset name': 'cifar10_4000:5000', 'alg name': 'Baseline', 'fold': 7, 'lr': 0.012, 'accuracy': 0.2, 'tpr': 0.2131, 'fpr': 0.0873, 'precision': 0.2212, 'auc': 0.7242, 'pr_curve': 0.2783, 'train_time': 3.2489, 'infer_time': 0.0971}, {'dataset name': 'cifar10_4000:5000', 'alg name': 'Baseline', 'fold': 8, 'lr': 0.001, 'accuracy': 0.23, 'tpr': 0.3271, 'fpr': 0.0847, 'precision': 0.2332, 'auc': 0.7647, 'pr_curve': 0.2938, 'train_time': 11.8692, 'infer_time': 0.0856}, {'dataset name': 'cifar10_4000:5000', 'alg name': 'Baseline', 'fold': 9, 'lr': 0.006, 'accuracy': 0.34, 'tpr': 0.3247, 'fpr': 0.0713, 'precision': 0.314, 'auc': 0.7838, 'pr_curve': 0.3399, 'train_time': 19.9782, 'infer_time': 0.2899}, {'dataset name': 'cifar10_4000:5000', 'alg name': 'Baseline', 'fold': 10, 'lr': 0.002, 'accuracy': 0.32, 'tpr': 0.3014, 'fpr': 0.076, 'precision': 0.2756, 'auc': 0.7564, 'pr_curve': 0.3457, 'train_time': 17.4935, 'infer_time': 0.282}, {'dataset name': 'cifar10_4000:5000', 'alg name': 'Cyclic', 'fold': 1, 'lr': '0.001-0.02', 'accuracy': 0.29, 'tpr': 0.3042, 'fpr': 0.0796, 'precision': 0.3915, 'auc': 0.746, 'pr_curve': 0.3337, 'train_time': 13.4603, 'infer_time': 0.2731}, {'dataset name': 'cifar10_4000:5000', 'alg name': 'Cyclic', 'fold': 2, 'lr': '0.001-0.02', 'accuracy': 0.32, 'tpr': 0.3336, 'fpr': 0.0751, 'precision': 0.3505, 'auc': 0.773, 'pr_curve': 0.3844, 'train_time': 17.2793, 'infer_time': 0.2764}, {'dataset name': 'cifar10_4000:5000', 'alg name': 'Cyclic', 'fold': 3, 'lr': '0.001-0.02', 'accuracy': 0.34, 'tpr': 0.3354, 'fpr': 0.0706, 'precision': 0.415, 'auc': 0.7855, 'pr_curve': 0.3889, 'train_time': 17.4003, 'infer_time': 0.2746}, {'dataset name': 'cifar10_4000:5000', 'alg name': 'Cyclic', 'fold': 4, 'lr': '0.001-0.02', 'accuracy': 0.29, 'tpr': 0.3007, 'fpr': 0.077, 'precision': 0.3495, 'auc': 0.7615, 'pr_curve': 0.3457, 'train_time': 14.6937, 'infer_time': 0.2798}, {'dataset name': 'cifar10_4000:5000', 'alg name': 'Cyclic', 'fold': 5, 'lr': '0.001-0.02', 'accuracy': 0.32, 'tpr': 0.3754, 'fpr': 0.0746, 'precision': 0.3163, 'auc': 0.7573, 'pr_curve': 0.349, 'train_time': 13.3287, 'infer_time': 0.2725}, {'dataset name': 'cifar10_4000:5000', 'alg name': 'Cyclic', 'fold': 6, 'lr': '0.001-0.02', 'accuracy': 0.42, 'tpr': 0.4219, 'fpr': 0.062, 'precision': 0.4296, 'auc': 0.7976, 'pr_curve': 0.3951, 'train_time': 15.8696, 'infer_time': 0.2869}, {'dataset name': 'cifar10_4000:5000', 'alg name': 'Cyclic', 'fold': 7, 'lr': '0.001-0.02', 'accuracy': 0.36, 'tpr': 0.4037, 'fpr': 0.0681, 'precision': 0.424, 'auc': 0.782, 'pr_curve': 0.3718, 'train_time': 14.7598, 'infer_time': 0.2859}, {'dataset name': 'cifar10_4000:5000', 'alg name': 'Cyclic', 'fold': 8, 'lr': '0.001-0.02', 'accuracy': 0.32, 'tpr': 0.3265, 'fpr': 0.0723, 'precision': 0.3448, 'auc': 0.7184, 'pr_curve': 0.3204, 'train_time': 18.2653, 'infer_time': 0.2644}, {'dataset name': 'cifar10_4000:5000', 'alg name': 'Cyclic', 'fold': 9, 'lr': '0.001-0.02', 'accuracy': 0.39, 'tpr': 0.3773, 'fpr': 0.0632, 'precision': 0.4559, 'auc': 0.7589, 'pr_curve': 0.36, 'train_time': 13.3971, 'infer_time': 0.2817}, {'dataset name': 'cifar10_4000:5000', 'alg name': 'Cyclic', 'fold': 10, 'lr': '0.001-0.02', 'accuracy': 0.34, 'tpr': 0.319, 'fpr': 0.076, 'precision': 0.3816, 'auc': 0.7681, 'pr_curve': 0.3581, 'train_time': 12.104, 'infer_time': 0.2763}, {'dataset name': 'cifar10_4000:5000', 'alg name': 'Ipmroved', 'fold': 1, 'lr': '0.001-0.011', 'accuracy': 0.29, 'tpr': 0.3138, 'fpr': 0.082, 'precision': 0.2668, 'auc': 0.7541, 'pr_curve': 0.3577, 'train_time': 3.2946, 'infer_time': 0.0912}, {'dataset name': 'cifar10_4000:5000', 'alg name': 'Ipmroved', 'fold': 2, 'lr': '0.001-0.011', 'accuracy': 0.3, 'tpr': 0.293, 'fpr': 0.0808, 'precision': 0.3471, 'auc': 0.751, 'pr_curve': 0.3664, 'train_time': 4.3411, 'infer_time': 0.083}, {'dataset name': 'cifar10_4000:5000', 'alg name': 'Ipmroved', 'fold': 3, 'lr': '0.001-0.014', 'accuracy': 0.28, 'tpr': 0.3375, 'fpr': 0.0742, 'precision': 0.3253, 'auc': 0.7334, 'pr_curve': 0.3101, 'train_time': 5.439, 'infer_time': 0.1106}, {'dataset name': 'cifar10_4000:5000', 'alg name': 'Ipmroved', 'fold': 4, 'lr': '0.001-0.012', 'accuracy': 0.27, 'tpr': 0.2899, 'fpr': 0.0818, 'precision': 0.2824, 'auc': 0.7351, 'pr_curve': 0.3268, 'train_time': 17.2403, 'infer_time': 0.2716}, {'dataset name': 'cifar10_4000:5000', 'alg name': 'Ipmroved', 'fold': 5, 'lr': '0.001-0.013000000000000001', 'accuracy': 0.33, 'tpr': 0.4249, 'fpr': 0.0701, 'precision': 0.3971, 'auc': 0.8142, 'pr_curve': 0.4128, 'train_time': 14.8088, 'infer_time': 0.2921}, {'dataset name': 'cifar10_4000:5000', 'alg name': 'Ipmroved', 'fold': 6, 'lr': '0.001-0.013000000000000001', 'accuracy': 0.45, 'tpr': 0.4217, 'fpr': 0.0586, 'precision': 0.3784, 'auc': 0.7843, 'pr_curve': 0.3895, 'train_time': 16.1628, 'infer_time': 0.2835}, {'dataset name': 'cifar10_4000:5000', 'alg name': 'Ipmroved', 'fold': 7, 'lr': '0.001-0.012', 'accuracy': 0.31, 'tpr': 0.3466, 'fpr': 0.0745, 'precision': 0.3308, 'auc': 0.7759, 'pr_curve': 0.3446, 'train_time': 7.0893, 'infer_time': 0.0845}, {'dataset name': 'cifar10_4000:5000', 'alg name': 'Ipmroved', 'fold': 8, 'lr': '0.001-0.012', 'accuracy': 0.24, 'tpr': 0.319, 'fpr': 0.085, 'precision': 0.251, 'auc': 0.7211, 'pr_curve': 0.2799, 'train_time': 3.0234, 'infer_time': 0.0826}, {'dataset name': 'cifar10_4000:5000', 'alg name': 'Ipmroved', 'fold': 9, 'lr': '0.001-0.015', 'accuracy': 0.36, 'tpr': 0.3353, 'fpr': 0.0671, 'precision': 0.3619, 'auc': 0.7774, 'pr_curve': 0.3488, 'train_time': 3.3325, 'infer_time': 0.091}, {'dataset name': 'cifar10_4000:5000', 'alg name': 'Ipmroved', 'fold': 10, 'lr': '0.001-0.017', 'accuracy': 0.26, 'tpr': 0.2442, 'fpr': 0.0826, 'precision': 0.3439, 'auc': 0.725, 'pr_curve': 0.3154, 'train_time': 9.9538, 'infer_time': 0.2515}]
d16_scores = [{'dataset name': 'cifar10_5000:6000', 'alg name': 'Baseline', 'fold': 1, 'lr': 0.004, 'accuracy': 0.29, 'tpr': 0.3506, 'fpr': 0.076, 'precision': 0.3674, 'auc': 0.7777, 'pr_curve': 0.3576, 'train_time': 29.2108, 'infer_time': 0.285}, {'dataset name': 'cifar10_5000:6000', 'alg name': 'Baseline', 'fold': 2, 'lr': 0.003, 'accuracy': 0.26, 'tpr': 0.2552, 'fpr': 0.0808, 'precision': 0.224, 'auc': 0.7016, 'pr_curve': 0.2843, 'train_time': 3.7731, 'infer_time': 0.0879}, {'dataset name': 'cifar10_5000:6000', 'alg name': 'Baseline', 'fold': 3, 'lr': 0.004, 'accuracy': 0.29, 'tpr': 0.2685, 'fpr': 0.0808, 'precision': 0.2977, 'auc': 0.7361, 'pr_curve': 0.3304, 'train_time': 17.6668, 'infer_time': 0.2687}, {'dataset name': 'cifar10_5000:6000', 'alg name': 'Baseline', 'fold': 4, 'lr': 0.002, 'accuracy': 0.35, 'tpr': 0.4181, 'fpr': 0.0698, 'precision': 0.3419, 'auc': 0.8099, 'pr_curve': 0.4053, 'train_time': 5.1066, 'infer_time': 0.0827}, {'dataset name': 'cifar10_5000:6000', 'alg name': 'Baseline', 'fold': 5, 'lr': 0.002, 'accuracy': 0.28, 'tpr': 0.2771, 'fpr': 0.0813, 'precision': 0.2891, 'auc': 0.7649, 'pr_curve': 0.3222, 'train_time': 4.0825, 'infer_time': 0.085}, {'dataset name': 'cifar10_5000:6000', 'alg name': 'Baseline', 'fold': 6, 'lr': 0.005, 'accuracy': 0.28, 'tpr': 0.2903, 'fpr': 0.076, 'precision': 0.2995, 'auc': 0.7501, 'pr_curve': 0.2973, 'train_time': 15.8516, 'infer_time': 0.2713}, {'dataset name': 'cifar10_5000:6000', 'alg name': 'Baseline', 'fold': 7, 'lr': 0.001, 'accuracy': 0.35, 'tpr': 0.3583, 'fpr': 0.0704, 'precision': 0.3705, 'auc': 0.7951, 'pr_curve': 0.3741, 'train_time': 30.2611, 'infer_time': 0.3044}, {'dataset name': 'cifar10_5000:6000', 'alg name': 'Baseline', 'fold': 8, 'lr': 0.003, 'accuracy': 0.31, 'tpr': 0.3395, 'fpr': 0.0757, 'precision': 0.3233, 'auc': 0.7544, 'pr_curve': 0.3209, 'train_time': 19.9236, 'infer_time': 0.2723}, {'dataset name': 'cifar10_5000:6000', 'alg name': 'Baseline', 'fold': 9, 'lr': 0.001, 'accuracy': 0.36, 'tpr': 0.3906, 'fpr': 0.0687, 'precision': 0.3685, 'auc': 0.8095, 'pr_curve': 0.3622, 'train_time': 5.1394, 'infer_time': 0.0884}, {'dataset name': 'cifar10_5000:6000', 'alg name': 'Baseline', 'fold': 10, 'lr': 0.002, 'accuracy': 0.27, 'tpr': 0.2657, 'fpr': 0.0823, 'precision': 0.3151, 'auc': 0.7367, 'pr_curve': 0.3174, 'train_time': 20.2399, 'infer_time': 0.2727}, {'dataset name': 'cifar10_5000:6000', 'alg name': 'Cyclic', 'fold': 1, 'lr': '0.001-0.02', 'accuracy': 0.26, 'tpr': 0.203, 'fpr': 0.0854, 'precision': 0.1577, 'auc': 0.6971, 'pr_curve': 0.2807, 'train_time': 15.9732, 'infer_time': 0.2569}, {'dataset name': 'cifar10_5000:6000', 'alg name': 'Cyclic', 'fold': 2, 'lr': '0.001-0.02', 'accuracy': 0.24, 'tpr': 0.2491, 'fpr': 0.0838, 'precision': 0.2826, 'auc': 0.6895, 'pr_curve': 0.2621, 'train_time': 15.9498, 'infer_time': 0.2858}, {'dataset name': 'cifar10_5000:6000', 'alg name': 'Cyclic', 'fold': 3, 'lr': '0.001-0.02', 'accuracy': 0.29, 'tpr': 0.3182, 'fpr': 0.0745, 'precision': 0.3873, 'auc': 0.7479, 'pr_curve': 0.351, 'train_time': 15.3637, 'infer_time': 0.0871}, {'dataset name': 'cifar10_5000:6000', 'alg name': 'Cyclic', 'fold': 4, 'lr': '0.001-0.02', 'accuracy': 0.35, 'tpr': 0.3545, 'fpr': 0.0721, 'precision': 0.3177, 'auc': 0.7735, 'pr_curve': 0.3625, 'train_time': 3.8355, 'infer_time': 0.0844}, {'dataset name': 'cifar10_5000:6000', 'alg name': 'Cyclic', 'fold': 5, 'lr': '0.001-0.02', 'accuracy': 0.25, 'tpr': 0.2568, 'fpr': 0.0797, 'precision': 0.3096, 'auc': 0.7247, 'pr_curve': 0.3072, 'train_time': 2.7706, 'infer_time': 0.0868}, {'dataset name': 'cifar10_5000:6000', 'alg name': 'Cyclic', 'fold': 6, 'lr': '0.001-0.02', 'accuracy': 0.29, 'tpr': 0.2502, 'fpr': 0.0787, 'precision': 0.2626, 'auc': 0.742, 'pr_curve': 0.35, 'train_time': 3.0105, 'infer_time': 0.0916}, {'dataset name': 'cifar10_5000:6000', 'alg name': 'Cyclic', 'fold': 7, 'lr': '0.001-0.02', 'accuracy': 0.28, 'tpr': 0.3065, 'fpr': 0.0757, 'precision': 0.3888, 'auc': 0.7552, 'pr_curve': 0.3624, 'train_time': 3.2992, 'infer_time': 0.0903}, {'dataset name': 'cifar10_5000:6000', 'alg name': 'Cyclic', 'fold': 8, 'lr': '0.001-0.02', 'accuracy': 0.27, 'tpr': 0.2978, 'fpr': 0.0806, 'precision': 0.309, 'auc': 0.726, 'pr_curve': 0.3121, 'train_time': 4.1159, 'infer_time': 0.0874}, {'dataset name': 'cifar10_5000:6000', 'alg name': 'Cyclic', 'fold': 9, 'lr': '0.001-0.02', 'accuracy': 0.26, 'tpr': 0.2777, 'fpr': 0.0817, 'precision': 0.2452, 'auc': 0.7438, 'pr_curve': 0.2878, 'train_time': 3.3067, 'infer_time': 0.0824}, {'dataset name': 'cifar10_5000:6000', 'alg name': 'Cyclic', 'fold': 10, 'lr': '0.001-0.02', 'accuracy': 0.32, 'tpr': 0.3116, 'fpr': 0.0727, 'precision': 0.3167, 'auc': 0.7087, 'pr_curve': 0.3303, 'train_time': 4.3564, 'infer_time': 0.0909}, {'dataset name': 'cifar10_5000:6000', 'alg name': 'Ipmroved', 'fold': 1, 'lr': '0.001-0.014', 'accuracy': 0.3, 'tpr': 0.3168, 'fpr': 0.0784, 'precision': 0.3229, 'auc': 0.7267, 'pr_curve': 0.3005, 'train_time': 3.3618, 'infer_time': 0.0936}, {'dataset name': 'cifar10_5000:6000', 'alg name': 'Ipmroved', 'fold': 2, 'lr': '0.001-0.013000000000000001', 'accuracy': 0.31, 'tpr': 0.3231, 'fpr': 0.0757, 'precision': 0.2954, 'auc': 0.7112, 'pr_curve': 0.2889, 'train_time': 14.7737, 'infer_time': 0.285}, {'dataset name': 'cifar10_5000:6000', 'alg name': 'Ipmroved', 'fold': 3, 'lr': '0.001-0.013000000000000001', 'accuracy': 0.33, 'tpr': 0.3295, 'fpr': 0.0739, 'precision': 0.3662, 'auc': 0.7629, 'pr_curve': 0.3797, 'train_time': 14.5519, 'infer_time': 0.2705}, {'dataset name': 'cifar10_5000:6000', 'alg name': 'Ipmroved', 'fold': 4, 'lr': '0.001-0.014', 'accuracy': 0.32, 'tpr': 0.2659, 'fpr': 0.0778, 'precision': 0.3332, 'auc': 0.767, 'pr_curve': 0.3577, 'train_time': 13.2317, 'infer_time': 0.2738}, {'dataset name': 'cifar10_5000:6000', 'alg name': 'Ipmroved', 'fold': 5, 'lr': '0.0-0.012', 'accuracy': 0.35, 'tpr': 0.3769, 'fpr': 0.0701, 'precision': 0.3593, 'auc': 0.7433, 'pr_curve': 0.3623, 'train_time': 17.2665, 'infer_time': 0.2897}, {'dataset name': 'cifar10_5000:6000', 'alg name': 'Ipmroved', 'fold': 6, 'lr': '0.001-0.013000000000000001', 'accuracy': 0.33, 'tpr': 0.3688, 'fpr': 0.0707, 'precision': 0.3649, 'auc': 0.7912, 'pr_curve': 0.3693, 'train_time': 2.9901, 'infer_time': 0.0855}, {'dataset name': 'cifar10_5000:6000', 'alg name': 'Ipmroved', 'fold': 7, 'lr': '0.001-0.013000000000000001', 'accuracy': 0.29, 'tpr': 0.3311, 'fpr': 0.0749, 'precision': 0.347, 'auc': 0.7395, 'pr_curve': 0.3427, 'train_time': 3.8536, 'infer_time': 0.0843}, {'dataset name': 'cifar10_5000:6000', 'alg name': 'Ipmroved', 'fold': 8, 'lr': '0.001-0.013000000000000001', 'accuracy': 0.31, 'tpr': 0.3304, 'fpr': 0.077, 'precision': 0.2985, 'auc': 0.7395, 'pr_curve': 0.3128, 'train_time': 5.7072, 'infer_time': 0.1191}, {'dataset name': 'cifar10_5000:6000', 'alg name': 'Ipmroved', 'fold': 9, 'lr': '0.001-0.013000000000000001', 'accuracy': 0.32, 'tpr': 0.3182, 'fpr': 0.0729, 'precision': 0.2734, 'auc': 0.8142, 'pr_curve': 0.4169, 'train_time': 13.2952, 'infer_time': 0.2925}, {'dataset name': 'cifar10_5000:6000', 'alg name': 'Ipmroved', 'fold': 10, 'lr': '0.001-0.015', 'accuracy': 0.36, 'tpr': 0.3961, 'fpr': 0.0659, 'precision': 0.4183, 'auc': 0.7851, 'pr_curve': 0.4048, 'train_time': 19.8345, 'infer_time': 0.2739}]
d17_scores = [{'dataset name': 'cifar10_6000:7000', 'alg name': 'Baseline', 'fold': 1, 'lr': 0.003, 'accuracy': 0.7076, 'tpr': 0.3321, 'fpr': 0.0782, 'precision': 0.3173, 'auc': 0.7612, 'pr_curve': 0.3269, 'train_time': 6.8836, 'infer_time': 0.4479}, {'dataset name': 'cifar10_6000:7000', 'alg name': 'Baseline', 'fold': 2, 'lr': 0.006, 'accuracy': 0.6412, 'tpr': 0.2476, 'fpr': 0.0826, 'precision': 0.2465, 'auc': 0.7585, 'pr_curve': 0.3265, 'train_time': 5.8298, 'infer_time': 0.2527}, {'dataset name': 'cifar10_6000:7000', 'alg name': 'Baseline', 'fold': 3, 'lr': 0.007, 'accuracy': 0.7406, 'tpr': 0.2954, 'fpr': 0.076, 'precision': 0.3324, 'auc': 0.7293, 'pr_curve': 0.3535, 'train_time': 6.8884, 'infer_time': 4.4772}, {'dataset name': 'cifar10_6000:7000', 'alg name': 'Baseline', 'fold': 4, 'lr': 0.004, 'accuracy': 0.6105, 'tpr': 0.3582, 'fpr': 0.0722, 'precision': 0.4447, 'auc': 0.7761, 'pr_curve': 0.3879, 'train_time': 5.0722, 'infer_time': 0.2351}, {'dataset name': 'cifar10_6000:7000', 'alg name': 'Baseline', 'fold': 5, 'lr': 0.003, 'accuracy': 0.6068, 'tpr': 0.3559, 'fpr': 0.0726, 'precision': 0.3772, 'auc': 0.7832, 'pr_curve': 0.4096, 'train_time': 5.8561, 'infer_time': 0.3987}, {'dataset name': 'cifar10_6000:7000', 'alg name': 'Baseline', 'fold': 6, 'lr': 0.003, 'accuracy': 0.6062, 'tpr': 0.3329, 'fpr': 0.0735, 'precision': 0.3557, 'auc': 0.7579, 'pr_curve': 0.337, 'train_time': 5.3803, 'infer_time': 0.2627}, {'dataset name': 'cifar10_6000:7000', 'alg name': 'Baseline', 'fold': 7, 'lr': 0.006, 'accuracy': 0.6863, 'tpr': 0.3126, 'fpr': 0.077, 'precision': 0.297, 'auc': 0.7089, 'pr_curve': 0.3069, 'train_time': 7.1993, 'infer_time': 4.3313}, {'dataset name': 'cifar10_6000:7000', 'alg name': 'Baseline', 'fold': 8, 'lr': 0.004, 'accuracy': 0.7019, 'tpr': 0.3818, 'fpr': 0.0653, 'precision': 0.386, 'auc': 0.7825, 'pr_curve': 0.3637, 'train_time': 7.3455, 'infer_time': 0.2424}, {'dataset name': 'cifar10_6000:7000', 'alg name': 'Baseline', 'fold': 9, 'lr': 0.002, 'accuracy': 0.6175, 'tpr': 0.3081, 'fpr': 0.0764, 'precision': 0.3166, 'auc': 0.769, 'pr_curve': 0.3477, 'train_time': 6.5543, 'infer_time': 0.2687}, {'dataset name': 'cifar10_6000:7000', 'alg name': 'Baseline', 'fold': 10, 'lr': 0.007, 'accuracy': 0.7189, 'tpr': 0.266, 'fpr': 0.0835, 'precision': 0.3513, 'auc': 0.7028, 'pr_curve': 0.2935, 'train_time': 6.2592, 'infer_time': 0.4389}, {'dataset name': 'cifar10_6000:7000', 'alg name': 'Cyclic', 'fold': 1, 'lr': '0.001-0.02', 'accuracy': 0.6004, 'tpr': 0.2685, 'fpr': 0.0871, 'precision': 0.2258, 'auc': 0.7739, 'pr_curve': 0.3265, 'train_time': 5.7181, 'infer_time': 0.5325}, {'dataset name': 'cifar10_6000:7000', 'alg name': 'Cyclic', 'fold': 2, 'lr': '0.001-0.02', 'accuracy': 0.6012, 'tpr': 0.3213, 'fpr': 0.077, 'precision': 0.3442, 'auc': 0.759, 'pr_curve': 0.3437, 'train_time': 5.0401, 'infer_time': 0.2513}, {'dataset name': 'cifar10_6000:7000', 'alg name': 'Cyclic', 'fold': 3, 'lr': '0.001-0.02', 'accuracy': 0.7034, 'tpr': 0.2781, 'fpr': 0.081, 'precision': 0.3091, 'auc': 0.7476, 'pr_curve': 0.3063, 'train_time': 6.8913, 'infer_time': 4.4411}, {'dataset name': 'cifar10_6000:7000', 'alg name': 'Cyclic', 'fold': 4, 'lr': '0.001-0.02', 'accuracy': 0.4944, 'tpr': 0.3749, 'fpr': 0.0718, 'precision': 0.3877, 'auc': 0.7808, 'pr_curve': 0.3606, 'train_time': 4.6532, 'infer_time': 1.5793}, {'dataset name': 'cifar10_6000:7000', 'alg name': 'Cyclic', 'fold': 5, 'lr': '0.001-0.02', 'accuracy': 0.4926, 'tpr': 0.2852, 'fpr': 0.0833, 'precision': 0.264, 'auc': 0.7357, 'pr_curve': 0.2724, 'train_time': 4.2136, 'infer_time': 0.2396}, {'dataset name': 'cifar10_6000:7000', 'alg name': 'Cyclic', 'fold': 6, 'lr': '0.001-0.02', 'accuracy': 0.5938, 'tpr': 0.2596, 'fpr': 0.085, 'precision': 0.4173, 'auc': 0.7871, 'pr_curve': 0.43, 'train_time': 4.66, 'infer_time': 1.5715}, {'dataset name': 'cifar10_6000:7000', 'alg name': 'Cyclic', 'fold': 7, 'lr': '0.001-0.02', 'accuracy': 0.6302, 'tpr': 0.3164, 'fpr': 0.0768, 'precision': 0.2999, 'auc': 0.7408, 'pr_curve': 0.3245, 'train_time': 5.0497, 'infer_time': 0.638}, {'dataset name': 'cifar10_6000:7000', 'alg name': 'Cyclic', 'fold': 8, 'lr': '0.001-0.02', 'accuracy': 0.6246, 'tpr': 0.4061, 'fpr': 0.0595, 'precision': 0.5425, 'auc': 0.7664, 'pr_curve': 0.3813, 'train_time': 5.6097, 'infer_time': 0.6704}, {'dataset name': 'cifar10_6000:7000', 'alg name': 'Cyclic', 'fold': 9, 'lr': '0.001-0.02', 'accuracy': 0.547, 'tpr': 0.2567, 'fpr': 0.0831, 'precision': 0.3168, 'auc': 0.6946, 'pr_curve': 0.2499, 'train_time': 5.0605, 'infer_time': 0.2578}, {'dataset name': 'cifar10_6000:7000', 'alg name': 'Cyclic', 'fold': 10, 'lr': '0.001-0.02', 'accuracy': 0.5079, 'tpr': 0.2541, 'fpr': 0.0836, 'precision': 0.2512, 'auc': 0.7556, 'pr_curve': 0.3639, 'train_time': 4.3293, 'infer_time': 0.2292}, {'dataset name': 'cifar10_6000:7000', 'alg name': 'Ipmroved', 'fold': 1, 'lr': '0.002-0.05', 'accuracy': 0.665, 'tpr': 0.3732, 'fpr': 0.0767, 'precision': 0.377, 'auc': 0.7754, 'pr_curve': 0.3622, 'train_time': 5.4741, 'infer_time': 0.7553}, {'dataset name': 'cifar10_6000:7000', 'alg name': 'Ipmroved', 'fold': 2, 'lr': '0.001-0.05', 'accuracy': 0.5412, 'tpr': 0.378, 'fpr': 0.0761, 'precision': 0.385, 'auc': 0.7621, 'pr_curve': 0.3779, 'train_time': 4.259, 'infer_time': 0.2295}, {'dataset name': 'cifar10_6000:7000', 'alg name': 'Ipmroved', 'fold': 3, 'lr': '0.001-0.05', 'accuracy': 0.603, 'tpr': 0.234, 'fpr': 0.0839, 'precision': 0.2106, 'auc': 0.7358, 'pr_curve': 0.2979, 'train_time': 5.1429, 'infer_time': 0.2353}, {'dataset name': 'cifar10_6000:7000', 'alg name': 'Ipmroved', 'fold': 4, 'lr': '0.0-0.05', 'accuracy': 0.467, 'tpr': 0.3561, 'fpr': 0.0743, 'precision': 0.3203, 'auc': 0.7624, 'pr_curve': 0.3229, 'train_time': 3.9237, 'infer_time': 2.3046}, {'dataset name': 'cifar10_6000:7000', 'alg name': 'Ipmroved', 'fold': 5, 'lr': '0.0-0.05', 'accuracy': 0.4305, 'tpr': 0.275, 'fpr': 0.0776, 'precision': 0.3049, 'auc': 0.6948, 'pr_curve': 0.2718, 'train_time': 3.5124, 'infer_time': 2.7321}, {'dataset name': 'cifar10_6000:7000', 'alg name': 'Ipmroved', 'fold': 6, 'lr': '0.002-0.05', 'accuracy': 0.4986, 'tpr': 0.3744, 'fpr': 0.0758, 'precision': 0.3199, 'auc': 0.751, 'pr_curve': 0.3545, 'train_time': 3.8931, 'infer_time': 2.3259}, {'dataset name': 'cifar10_6000:7000', 'alg name': 'Ipmroved', 'fold': 7, 'lr': '0.001-0.05', 'accuracy': 0.5806, 'tpr': 0.2778, 'fpr': 0.0822, 'precision': 0.2779, 'auc': 0.7096, 'pr_curve': 0.2636, 'train_time': 5.0317, 'infer_time': 0.2591}, {'dataset name': 'cifar10_6000:7000', 'alg name': 'Ipmroved', 'fold': 8, 'lr': '0.002-0.05', 'accuracy': 0.5994, 'tpr': 0.3392, 'fpr': 0.0711, 'precision': 0.509, 'auc': 0.7433, 'pr_curve': 0.3408, 'train_time': 4.7055, 'infer_time': 1.5001}, {'dataset name': 'cifar10_6000:7000', 'alg name': 'Ipmroved', 'fold': 9, 'lr': '0.001-0.05', 'accuracy': 0.6174, 'tpr': 0.3025, 'fpr': 0.0779, 'precision': 0.2836, 'auc': 0.7117, 'pr_curve': 0.2688, 'train_time': 6.2136, 'infer_time': 5.148}, {'dataset name': 'cifar10_6000:7000', 'alg name': 'Ipmroved', 'fold': 10, 'lr': '0.001-0.05', 'accuracy': 0.5679, 'tpr': 0.309, 'fpr': 0.0819, 'precision': 0.5009, 'auc': 0.7199, 'pr_curve': 0.3352, 'train_time': 4.3101, 'infer_time': 1.9101}]
d18_scores = [{'dataset name': 'cifar10_7000:8000', 'alg name': 'Baseline', 'fold': 1, 'lr': 0.007, 'accuracy': 0.5963, 'tpr': 0.3195, 'fpr': 0.0773, 'precision': 0.3508, 'auc': 0.7593, 'pr_curve': 0.366, 'train_time': 1.3479, 'infer_time': 0.8932}, {'dataset name': 'cifar10_7000:8000', 'alg name': 'Baseline', 'fold': 2, 'lr': 0.008, 'accuracy': 0.7238, 'tpr': 0.2478, 'fpr': 0.0876, 'precision': 0.3297, 'auc': 0.7403, 'pr_curve': 0.2864, 'train_time': 1.7314, 'infer_time': 0.4184}, {'dataset name': 'cifar10_7000:8000', 'alg name': 'Baseline', 'fold': 3, 'lr': 0.006, 'accuracy': 0.594, 'tpr': 0.3574, 'fpr': 0.0731, 'precision': 0.3384, 'auc': 0.7746, 'pr_curve': 0.3736, 'train_time': 1.4566, 'infer_time': 0.6951}, {'dataset name': 'cifar10_7000:8000', 'alg name': 'Baseline', 'fold': 4, 'lr': 0.009000000000000001, 'accuracy': 0.6728, 'tpr': 0.3797, 'fpr': 0.0705, 'precision': 0.3673, 'auc': 0.7531, 'pr_curve': 0.3448, 'train_time': 1.4385, 'infer_time': 0.6353}, {'dataset name': 'cifar10_7000:8000', 'alg name': 'Baseline', 'fold': 5, 'lr': 0.013000000000000001, 'accuracy': 0.6429, 'tpr': 0.3062, 'fpr': 0.0782, 'precision': 0.3368, 'auc': 0.7189, 'pr_curve': 0.2715, 'train_time': 1.5852, 'infer_time': 0.5663}, {'dataset name': 'cifar10_7000:8000', 'alg name': 'Baseline', 'fold': 6, 'lr': 0.002, 'accuracy': 0.6106, 'tpr': 0.2938, 'fpr': 0.079, 'precision': 0.2776, 'auc': 0.7684, 'pr_curve': 0.3378, 'train_time': 1.5606, 'infer_time': 0.5273}, {'dataset name': 'cifar10_7000:8000', 'alg name': 'Baseline', 'fold': 7, 'lr': 0.005, 'accuracy': 0.622, 'tpr': 0.3291, 'fpr': 0.0758, 'precision': 0.3948, 'auc': 0.7302, 'pr_curve': 0.3326, 'train_time': 1.3146, 'infer_time': 0.8307}, {'dataset name': 'cifar10_7000:8000', 'alg name': 'Baseline', 'fold': 8, 'lr': 0.004, 'accuracy': 0.624, 'tpr': 0.2704, 'fpr': 0.0809, 'precision': 0.2768, 'auc': 0.739, 'pr_curve': 0.2841, 'train_time': 1.7178, 'infer_time': 0.1824}, {'dataset name': 'cifar10_7000:8000', 'alg name': 'Baseline', 'fold': 9, 'lr': 0.01, 'accuracy': 0.5209, 'tpr': 0.2781, 'fpr': 0.0815, 'precision': 0.272, 'auc': 0.6947, 'pr_curve': 0.2691, 'train_time': 1.4486, 'infer_time': 0.1859}, {'dataset name': 'cifar10_7000:8000', 'alg name': 'Baseline', 'fold': 10, 'lr': 0.01, 'accuracy': 0.6591, 'tpr': 0.2906, 'fpr': 0.0784, 'precision': 0.2461, 'auc': 0.7459, 'pr_curve': 0.3472, 'train_time': 1.3979, 'infer_time': 0.1665}, {'dataset name': 'cifar10_7000:8000', 'alg name': 'Cyclic', 'fold': 1, 'lr': '0.001-0.02', 'accuracy': 0.5821, 'tpr': 0.3652, 'fpr': 0.0694, 'precision': 0.3924, 'auc': 0.7673, 'pr_curve': 0.3779, 'train_time': 1.5133, 'infer_time': 0.583}, {'dataset name': 'cifar10_7000:8000', 'alg name': 'Cyclic', 'fold': 2, 'lr': '0.001-0.02', 'accuracy': 0.5949, 'tpr': 0.3773, 'fpr': 0.0761, 'precision': 0.3475, 'auc': 0.77, 'pr_curve': 0.3572, 'train_time': 1.4216, 'infer_time': 0.6647}, {'dataset name': 'cifar10_7000:8000', 'alg name': 'Cyclic', 'fold': 3, 'lr': '0.001-0.02', 'accuracy': 0.6147, 'tpr': 0.3, 'fpr': 0.0788, 'precision': 0.3354, 'auc': 0.7662, 'pr_curve': 0.3645, 'train_time': 1.4037, 'infer_time': 0.1617}, {'dataset name': 'cifar10_7000:8000', 'alg name': 'Cyclic', 'fold': 4, 'lr': '0.001-0.02', 'accuracy': 0.5611, 'tpr': 0.3134, 'fpr': 0.0771, 'precision': 0.3707, 'auc': 0.7231, 'pr_curve': 0.3056, 'train_time': 1.2883, 'infer_time': 0.7865}, {'dataset name': 'cifar10_7000:8000', 'alg name': 'Cyclic', 'fold': 5, 'lr': '0.001-0.02', 'accuracy': 0.4725, 'tpr': 0.3629, 'fpr': 0.0686, 'precision': 0.4381, 'auc': 0.826, 'pr_curve': 0.4023, 'train_time': 1.2787, 'infer_time': 0.8107}, {'dataset name': 'cifar10_7000:8000', 'alg name': 'Cyclic', 'fold': 6, 'lr': '0.001-0.02', 'accuracy': 0.6648, 'tpr': 0.3536, 'fpr': 0.0746, 'precision': 0.3248, 'auc': 0.761, 'pr_curve': 0.3486, 'train_time': 1.572, 'infer_time': 0.5006}, {'dataset name': 'cifar10_7000:8000', 'alg name': 'Cyclic', 'fold': 7, 'lr': '0.001-0.02', 'accuracy': 0.6531, 'tpr': 0.3318, 'fpr': 0.0797, 'precision': 0.3275, 'auc': 0.7507, 'pr_curve': 0.3277, 'train_time': 1.7548, 'infer_time': 0.6825}, {'dataset name': 'cifar10_7000:8000', 'alg name': 'Cyclic', 'fold': 8, 'lr': '0.001-0.02', 'accuracy': 0.5467, 'tpr': 0.2331, 'fpr': 0.0843, 'precision': 0.2853, 'auc': 0.7416, 'pr_curve': 0.2937, 'train_time': 1.3447, 'infer_time': 0.1789}, {'dataset name': 'cifar10_7000:8000', 'alg name': 'Cyclic', 'fold': 9, 'lr': '0.001-0.02', 'accuracy': 0.688, 'tpr': 0.3452, 'fpr': 0.0773, 'precision': 0.3167, 'auc': 0.7549, 'pr_curve': 0.3632, 'train_time': 1.5973, 'infer_time': 0.4993}, {'dataset name': 'cifar10_7000:8000', 'alg name': 'Cyclic', 'fold': 10, 'lr': '0.001-0.02', 'accuracy': 0.5681, 'tpr': 0.2787, 'fpr': 0.0795, 'precision': 0.2728, 'auc': 0.7738, 'pr_curve': 0.3387, 'train_time': 1.373, 'infer_time': 0.7049}, {'dataset name': 'cifar10_7000:8000', 'alg name': 'Ipmroved', 'fold': 1, 'lr': '0.002-0.05', 'accuracy': 0.6422, 'tpr': 0.3746, 'fpr': 0.0718, 'precision': 0.441, 'auc': 0.7868, 'pr_curve': 0.41, 'train_time': 1.3434, 'infer_time': 0.725}, {'dataset name': 'cifar10_7000:8000', 'alg name': 'Ipmroved', 'fold': 2, 'lr': '0.001-0.05', 'accuracy': 0.6379, 'tpr': 0.2961, 'fpr': 0.081, 'precision': 0.2929, 'auc': 0.6857, 'pr_curve': 0.2766, 'train_time': 1.3998, 'infer_time': 0.1752}, {'dataset name': 'cifar10_7000:8000', 'alg name': 'Ipmroved', 'fold': 3, 'lr': '0.002-0.05', 'accuracy': 0.6469, 'tpr': 0.3248, 'fpr': 0.0763, 'precision': 0.2868, 'auc': 0.7298, 'pr_curve': 0.3231, 'train_time': 1.6873, 'infer_time': 0.1751}, {'dataset name': 'cifar10_7000:8000', 'alg name': 'Ipmroved', 'fold': 4, 'lr': '0.001-0.05', 'accuracy': 0.5614, 'tpr': 0.3051, 'fpr': 0.0781, 'precision': 0.2947, 'auc': 0.7208, 'pr_curve': 0.3034, 'train_time': 1.4327, 'infer_time': 0.1897}, {'dataset name': 'cifar10_7000:8000', 'alg name': 'Ipmroved', 'fold': 5, 'lr': '0.001-0.05', 'accuracy': 0.5869, 'tpr': 0.3591, 'fpr': 0.0749, 'precision': 0.354, 'auc': 0.8211, 'pr_curve': 0.3723, 'train_time': 1.3331, 'infer_time': 0.77}, {'dataset name': 'cifar10_7000:8000', 'alg name': 'Ipmroved', 'fold': 6, 'lr': '0.0-0.05', 'accuracy': 0.5934, 'tpr': 0.2929, 'fpr': 0.081, 'precision': 0.306, 'auc': 0.739, 'pr_curve': 0.3182, 'train_time': 1.5804, 'infer_time': 0.5853}, {'dataset name': 'cifar10_7000:8000', 'alg name': 'Ipmroved', 'fold': 7, 'lr': '0.002-0.05', 'accuracy': 0.5749, 'tpr': 0.269, 'fpr': 0.0808, 'precision': 0.2866, 'auc': 0.7236, 'pr_curve': 0.2969, 'train_time': 1.3731, 'infer_time': 0.1811}, {'dataset name': 'cifar10_7000:8000', 'alg name': 'Ipmroved', 'fold': 8, 'lr': '0.002-0.05', 'accuracy': 0.5407, 'tpr': 0.218, 'fpr': 0.0866, 'precision': 0.3211, 'auc': 0.7622, 'pr_curve': 0.3405, 'train_time': 1.4612, 'infer_time': 0.1782}, {'dataset name': 'cifar10_7000:8000', 'alg name': 'Ipmroved', 'fold': 9, 'lr': '0.001-0.05', 'accuracy': 0.5744, 'tpr': 0.3792, 'fpr': 0.0769, 'precision': 0.3956, 'auc': 0.7471, 'pr_curve': 0.345, 'train_time': 1.4224, 'infer_time': 0.7332}, {'dataset name': 'cifar10_7000:8000', 'alg name': 'Ipmroved', 'fold': 10, 'lr': '0.001-0.05', 'accuracy': 0.6064, 'tpr': 0.211, 'fpr': 0.0881, 'precision': 0.3409, 'auc': 0.6958, 'pr_curve': 0.2576, 'train_time': 1.4374, 'infer_time': 0.7348}]
d19_scores = [{'dataset name': 'cifar10_8000:9000', 'alg name': 'Baseline', 'fold': 1, 'lr': 0.006, 'accuracy': 0.598, 'tpr': 0.2864, 'fpr': 0.0822, 'precision': 0.3773, 'auc': 0.721, 'pr_curve': 0.3028, 'train_time': 1.2537, 'infer_time': 0.1664}, {'dataset name': 'cifar10_8000:9000', 'alg name': 'Baseline', 'fold': 2, 'lr': 0.005, 'accuracy': 0.7273, 'tpr': 0.3481, 'fpr': 0.0742, 'precision': 0.3581, 'auc': 0.75, 'pr_curve': 0.351, 'train_time': 2.1548, 'infer_time': 0.3587}, {'dataset name': 'cifar10_8000:9000', 'alg name': 'Baseline', 'fold': 3, 'lr': 0.004, 'accuracy': 0.7211, 'tpr': 0.4189, 'fpr': 0.0657, 'precision': 0.4709, 'auc': 0.7978, 'pr_curve': 0.4376, 'train_time': 1.5639, 'infer_time': 0.5328}, {'dataset name': 'cifar10_8000:9000', 'alg name': 'Baseline', 'fold': 4, 'lr': 0.003, 'accuracy': 0.6461, 'tpr': 0.3948, 'fpr': 0.07, 'precision': 0.3641, 'auc': 0.7334, 'pr_curve': 0.3427, 'train_time': 1.669, 'infer_time': 0.1741}, {'dataset name': 'cifar10_8000:9000', 'alg name': 'Baseline', 'fold': 5, 'lr': 0.006, 'accuracy': 0.7319, 'tpr': 0.3139, 'fpr': 0.0773, 'precision': 0.2929, 'auc': 0.7428, 'pr_curve': 0.3428, 'train_time': 1.5615, 'infer_time': 0.1893}, {'dataset name': 'cifar10_8000:9000', 'alg name': 'Baseline', 'fold': 6, 'lr': 0.005, 'accuracy': 0.7156, 'tpr': 0.413, 'fpr': 0.0632, 'precision': 0.3702, 'auc': 0.7927, 'pr_curve': 0.441, 'train_time': 1.5062, 'infer_time': 0.6099}, {'dataset name': 'cifar10_8000:9000', 'alg name': 'Baseline', 'fold': 7, 'lr': 0.006, 'accuracy': 0.7677, 'tpr': 0.3601, 'fpr': 0.069, 'precision': 0.3884, 'auc': 0.7598, 'pr_curve': 0.371, 'train_time': 1.6759, 'infer_time': 0.4689}, {'dataset name': 'cifar10_8000:9000', 'alg name': 'Baseline', 'fold': 8, 'lr': 0.006, 'accuracy': 0.695, 'tpr': 0.2444, 'fpr': 0.0847, 'precision': 0.3102, 'auc': 0.7435, 'pr_curve': 0.32, 'train_time': 1.5987, 'infer_time': 0.1899}, {'dataset name': 'cifar10_8000:9000', 'alg name': 'Baseline', 'fold': 9, 'lr': 0.004, 'accuracy': 0.6701, 'tpr': 0.2452, 'fpr': 0.0838, 'precision': 0.2466, 'auc': 0.7549, 'pr_curve': 0.3018, 'train_time': 1.5617, 'infer_time': 0.566}, {'dataset name': 'cifar10_8000:9000', 'alg name': 'Baseline', 'fold': 10, 'lr': 0.004, 'accuracy': 0.7397, 'tpr': 0.3049, 'fpr': 0.0765, 'precision': 0.3037, 'auc': 0.7221, 'pr_curve': 0.3243, 'train_time': 2.0125, 'infer_time': 0.4437}, {'dataset name': 'cifar10_8000:9000', 'alg name': 'Cyclic', 'fold': 1, 'lr': '0.001-0.02', 'accuracy': 0.5183, 'tpr': 0.2293, 'fpr': 0.0852, 'precision': 0.3643, 'auc': 0.6972, 'pr_curve': 0.2883, 'train_time': 1.3572, 'infer_time': 0.1968}, {'dataset name': 'cifar10_8000:9000', 'alg name': 'Cyclic', 'fold': 2, 'lr': '0.001-0.02', 'accuracy': 0.594, 'tpr': 0.3688, 'fpr': 0.0682, 'precision': 0.5193, 'auc': 0.7547, 'pr_curve': 0.3931, 'train_time': 1.4602, 'infer_time': 0.6632}, {'dataset name': 'cifar10_8000:9000', 'alg name': 'Cyclic', 'fold': 3, 'lr': '0.001-0.02', 'accuracy': 0.5326, 'tpr': 0.3388, 'fpr': 0.0713, 'precision': 0.3227, 'auc': 0.7531, 'pr_curve': 0.3626, 'train_time': 1.4003, 'infer_time': 0.1948}, {'dataset name': 'cifar10_8000:9000', 'alg name': 'Cyclic', 'fold': 4, 'lr': '0.001-0.02', 'accuracy': 0.6163, 'tpr': 0.4229, 'fpr': 0.0649, 'precision': 0.3976, 'auc': 0.7701, 'pr_curve': 0.392, 'train_time': 1.5206, 'infer_time': 0.6}, {'dataset name': 'cifar10_8000:9000', 'alg name': 'Cyclic', 'fold': 5, 'lr': '0.001-0.02', 'accuracy': 0.6487, 'tpr': 0.3066, 'fpr': 0.077, 'precision': 0.3045, 'auc': 0.747, 'pr_curve': 0.3185, 'train_time': 1.5376, 'infer_time': 0.2015}, {'dataset name': 'cifar10_8000:9000', 'alg name': 'Cyclic', 'fold': 6, 'lr': '0.001-0.02', 'accuracy': 0.5717, 'tpr': 0.3457, 'fpr': 0.0723, 'precision': 0.3555, 'auc': 0.753, 'pr_curve': 0.3306, 'train_time': 1.516, 'infer_time': 0.17}, {'dataset name': 'cifar10_8000:9000', 'alg name': 'Cyclic', 'fold': 7, 'lr': '0.001-0.02', 'accuracy': 0.6356, 'tpr': 0.2418, 'fpr': 0.0817, 'precision': 0.2962, 'auc': 0.7239, 'pr_curve': 0.3118, 'train_time': 1.5774, 'infer_time': 0.1987}, {'dataset name': 'cifar10_8000:9000', 'alg name': 'Cyclic', 'fold': 8, 'lr': '0.001-0.02', 'accuracy': 0.6017, 'tpr': 0.4246, 'fpr': 0.0667, 'precision': 0.4695, 'auc': 0.7408, 'pr_curve': 0.3352, 'train_time': 1.684, 'infer_time': 0.1796}, {'dataset name': 'cifar10_8000:9000', 'alg name': 'Cyclic', 'fold': 9, 'lr': '0.001-0.02', 'accuracy': 0.6115, 'tpr': 0.3836, 'fpr': 0.0679, 'precision': 0.3896, 'auc': 0.7764, 'pr_curve': 0.3935, 'train_time': 1.5982, 'infer_time': 0.1826}, {'dataset name': 'cifar10_8000:9000', 'alg name': 'Cyclic', 'fold': 10, 'lr': '0.001-0.02', 'accuracy': 0.5927, 'tpr': 0.3473, 'fpr': 0.0739, 'precision': 0.3203, 'auc': 0.6975, 'pr_curve': 0.3285, 'train_time': 1.5667, 'infer_time': 0.6331}, {'dataset name': 'cifar10_8000:9000', 'alg name': 'Ipmroved', 'fold': 1, 'lr': '0.001-0.05', 'accuracy': 0.6367, 'tpr': 0.251, 'fpr': 0.0833, 'precision': 0.2152, 'auc': 0.7122, 'pr_curve': 0.2924, 'train_time': 1.5123, 'infer_time': 0.1937}, {'dataset name': 'cifar10_8000:9000', 'alg name': 'Ipmroved', 'fold': 2, 'lr': '0.001-0.05', 'accuracy': 0.5345, 'tpr': 0.3268, 'fpr': 0.0773, 'precision': 0.3949, 'auc': 0.76, 'pr_curve': 0.3194, 'train_time': 1.3728, 'infer_time': 0.7878}, {'dataset name': 'cifar10_8000:9000', 'alg name': 'Ipmroved', 'fold': 3, 'lr': '0.002-0.05', 'accuracy': 0.622, 'tpr': 0.3795, 'fpr': 0.0709, 'precision': 0.4878, 'auc': 0.7501, 'pr_curve': 0.393, 'train_time': 1.4578, 'infer_time': 0.5111}, {'dataset name': 'cifar10_8000:9000', 'alg name': 'Ipmroved', 'fold': 4, 'lr': '0.001-0.05', 'accuracy': 0.5731, 'tpr': 0.3, 'fpr': 0.0821, 'precision': 0.512, 'auc': 0.7304, 'pr_curve': 0.3774, 'train_time': 1.3827, 'infer_time': 0.2231}, {'dataset name': 'cifar10_8000:9000', 'alg name': 'Ipmroved', 'fold': 5, 'lr': '0.001-0.05', 'accuracy': 0.5892, 'tpr': 0.219, 'fpr': 0.0865, 'precision': 0.2265, 'auc': 0.7032, 'pr_curve': 0.2578, 'train_time': 1.4654, 'infer_time': 0.1806}, {'dataset name': 'cifar10_8000:9000', 'alg name': 'Ipmroved', 'fold': 6, 'lr': '0.001-0.05', 'accuracy': 0.5485, 'tpr': 0.3257, 'fpr': 0.0763, 'precision': 0.4481, 'auc': 0.7626, 'pr_curve': 0.3814, 'train_time': 1.4118, 'infer_time': 0.7401}, {'dataset name': 'cifar10_8000:9000', 'alg name': 'Ipmroved', 'fold': 7, 'lr': '0.002-0.05', 'accuracy': 0.6441, 'tpr': 0.3388, 'fpr': 0.0723, 'precision': 0.3492, 'auc': 0.7234, 'pr_curve': 0.2928, 'train_time': 1.4622, 'infer_time': 0.1767}, {'dataset name': 'cifar10_8000:9000', 'alg name': 'Ipmroved', 'fold': 8, 'lr': '0.001-0.05', 'accuracy': 0.566, 'tpr': 0.3991, 'fpr': 0.0688, 'precision': 0.3853, 'auc': 0.767, 'pr_curve': 0.3931, 'train_time': 1.8135, 'infer_time': 0.1931}, {'dataset name': 'cifar10_8000:9000', 'alg name': 'Ipmroved', 'fold': 9, 'lr': '0.001-0.043000000000000003', 'accuracy': 0.5578, 'tpr': 0.3362, 'fpr': 0.0734, 'precision': 0.3943, 'auc': 0.7388, 'pr_curve': 0.3188, 'train_time': 1.3735, 'infer_time': 0.7723}, {'dataset name': 'cifar10_8000:9000', 'alg name': 'Ipmroved', 'fold': 10, 'lr': '0.001-0.05', 'accuracy': 0.5542, 'tpr': 0.3177, 'fpr': 0.0749, 'precision': 0.4293, 'auc': 0.6639, 'pr_curve': 0.2584, 'train_time': 1.3965, 'infer_time': 0.1949}]
d20_scores = [{'dataset name': 'cifar10_9000:10000', 'alg name': 'Baseline', 'fold': 1, 'lr': 0.004, 'accuracy': 0.6295, 'tpr': 0.2863, 'fpr': 0.079, 'precision': 0.3385, 'auc': 0.7065, 'pr_curve': 0.3125, 'train_time': 6.6249, 'infer_time': 5.0199}, {'dataset name': 'cifar10_9000:10000', 'alg name': 'Baseline', 'fold': 2, 'lr': 0.005, 'accuracy': 0.7283, 'tpr': 0.2953, 'fpr': 0.0805, 'precision': 0.276, 'auc': 0.752, 'pr_curve': 0.3344, 'train_time': 5.6258, 'infer_time': 0.5625}, {'dataset name': 'cifar10_9000:10000', 'alg name': 'Baseline', 'fold': 3, 'lr': 0.005, 'accuracy': 0.7335, 'tpr': 0.3028, 'fpr': 0.0778, 'precision': 0.2702, 'auc': 0.7741, 'pr_curve': 0.3782, 'train_time': 7.1564, 'infer_time': 0.2186}, {'dataset name': 'cifar10_9000:10000', 'alg name': 'Baseline', 'fold': 4, 'lr': 0.004, 'accuracy': 0.6975, 'tpr': 0.3844, 'fpr': 0.0739, 'precision': 0.4115, 'auc': 0.7419, 'pr_curve': 0.297, 'train_time': 4.5186, 'infer_time': 0.2336}, {'dataset name': 'cifar10_9000:10000', 'alg name': 'Baseline', 'fold': 5, 'lr': 0.006, 'accuracy': 0.7482, 'tpr': 0.3757, 'fpr': 0.0697, 'precision': 0.3484, 'auc': 0.7621, 'pr_curve': 0.4, 'train_time': 5.8032, 'infer_time': 0.4838}, {'dataset name': 'cifar10_9000:10000', 'alg name': 'Baseline', 'fold': 6, 'lr': 0.007, 'accuracy': 0.6939, 'tpr': 0.4673, 'fpr': 0.0678, 'precision': 0.415, 'auc': 0.7958, 'pr_curve': 0.3735, 'train_time': 4.9155, 'infer_time': 1.2896}, {'dataset name': 'cifar10_9000:10000', 'alg name': 'Baseline', 'fold': 7, 'lr': 0.006, 'accuracy': 0.7065, 'tpr': 0.3019, 'fpr': 0.0795, 'precision': 0.2791, 'auc': 0.769, 'pr_curve': 0.3349, 'train_time': 5.7111, 'infer_time': 0.4947}, {'dataset name': 'cifar10_9000:10000', 'alg name': 'Baseline', 'fold': 8, 'lr': 0.007, 'accuracy': 0.6156, 'tpr': 0.2281, 'fpr': 0.0859, 'precision': 0.2393, 'auc': 0.7305, 'pr_curve': 0.2866, 'train_time': 4.5922, 'infer_time': 1.6031}, {'dataset name': 'cifar10_9000:10000', 'alg name': 'Baseline', 'fold': 9, 'lr': 0.005, 'accuracy': 0.704, 'tpr': 0.3821, 'fpr': 0.0691, 'precision': 0.3793, 'auc': 0.8128, 'pr_curve': 0.3915, 'train_time': 4.9367, 'infer_time': 1.2822}, {'dataset name': 'cifar10_9000:10000', 'alg name': 'Baseline', 'fold': 10, 'lr': 0.003, 'accuracy': 0.6452, 'tpr': 0.3252, 'fpr': 0.078, 'precision': 0.3845, 'auc': 0.7505, 'pr_curve': 0.3624, 'train_time': 4.9453, 'infer_time': 0.2336}, {'dataset name': 'cifar10_9000:10000', 'alg name': 'Cyclic', 'fold': 1, 'lr': '0.001-0.02', 'accuracy': 0.6216, 'tpr': 0.2279, 'fpr': 0.0839, 'precision': 0.241, 'auc': 0.728, 'pr_curve': 0.2793, 'train_time': 4.9829, 'infer_time': 1.2282}, {'dataset name': 'cifar10_9000:10000', 'alg name': 'Cyclic', 'fold': 2, 'lr': '0.001-0.02', 'accuracy': 0.6545, 'tpr': 0.3087, 'fpr': 0.0824, 'precision': 0.311, 'auc': 0.7376, 'pr_curve': 0.37, 'train_time': 5.7767, 'infer_time': 0.2394}, {'dataset name': 'cifar10_9000:10000', 'alg name': 'Cyclic', 'fold': 3, 'lr': '0.001-0.02', 'accuracy': 0.6026, 'tpr': 0.278, 'fpr': 0.082, 'precision': 0.3108, 'auc': 0.7584, 'pr_curve': 0.3544, 'train_time': 5.7231, 'infer_time': 0.2246}, {'dataset name': 'cifar10_9000:10000', 'alg name': 'Cyclic', 'fold': 4, 'lr': '0.001-0.02', 'accuracy': 0.6028, 'tpr': 0.3755, 'fpr': 0.0737, 'precision': 0.4296, 'auc': 0.7721, 'pr_curve': 0.391, 'train_time': 5.3497, 'infer_time': 0.876}, {'dataset name': 'cifar10_9000:10000', 'alg name': 'Cyclic', 'fold': 5, 'lr': '0.001-0.02', 'accuracy': 0.6357, 'tpr': 0.414, 'fpr': 0.0661, 'precision': 0.3697, 'auc': 0.7541, 'pr_curve': 0.3294, 'train_time': 4.975, 'infer_time': 1.2087}, {'dataset name': 'cifar10_9000:10000', 'alg name': 'Cyclic', 'fold': 6, 'lr': '0.001-0.02', 'accuracy': 0.6307, 'tpr': 0.3678, 'fpr': 0.071, 'precision': 0.3418, 'auc': 0.786, 'pr_curve': 0.3829, 'train_time': 5.7356, 'infer_time': 0.4918}, {'dataset name': 'cifar10_9000:10000', 'alg name': 'Cyclic', 'fold': 7, 'lr': '0.001-0.02', 'accuracy': 0.5471, 'tpr': 0.2622, 'fpr': 0.0851, 'precision': 0.2583, 'auc': 0.7709, 'pr_curve': 0.3422, 'train_time': 4.96, 'infer_time': 1.2414}, {'dataset name': 'cifar10_9000:10000', 'alg name': 'Cyclic', 'fold': 8, 'lr': '0.001-0.02', 'accuracy': 0.6793, 'tpr': 0.3745, 'fpr': 0.0772, 'precision': 0.2688, 'auc': 0.7537, 'pr_curve': 0.3182, 'train_time': 5.7477, 'infer_time': 0.4635}, {'dataset name': 'cifar10_9000:10000', 'alg name': 'Cyclic', 'fold': 9, 'lr': '0.001-0.02', 'accuracy': 0.5929, 'tpr': 0.3409, 'fpr': 0.0749, 'precision': 0.4079, 'auc': 0.7312, 'pr_curve': 0.323, 'train_time': 4.9829, 'infer_time': 0.2457}, {'dataset name': 'cifar10_9000:10000', 'alg name': 'Cyclic', 'fold': 10, 'lr': '0.001-0.02', 'accuracy': 0.5206, 'tpr': 0.2553, 'fpr': 0.0844, 'precision': 0.2878, 'auc': 0.7222, 'pr_curve': 0.2965, 'train_time': 4.2293, 'infer_time': 1.9829}, {'dataset name': 'cifar10_9000:10000', 'alg name': 'Ipmroved', 'fold': 1, 'lr': '0.002-0.05', 'accuracy': 0.64, 'tpr': 0.2889, 'fpr': 0.0788, 'precision': 0.3167, 'auc': 0.733, 'pr_curve': 0.3231, 'train_time': 4.6253, 'infer_time': 1.5935}, {'dataset name': 'cifar10_9000:10000', 'alg name': 'Ipmroved', 'fold': 2, 'lr': '0.001-0.05', 'accuracy': 0.637, 'tpr': 0.3163, 'fpr': 0.0809, 'precision': 0.3623, 'auc': 0.7001, 'pr_curve': 0.3138, 'train_time': 5.1067, 'infer_time': 1.1341}, {'dataset name': 'cifar10_9000:10000', 'alg name': 'Ipmroved', 'fold': 3, 'lr': '0.001-0.05', 'accuracy': 0.6508, 'tpr': 0.2598, 'fpr': 0.082, 'precision': 0.2976, 'auc': 0.7505, 'pr_curve': 0.3765, 'train_time': 6.2083, 'infer_time': 0.2607}, {'dataset name': 'cifar10_9000:10000', 'alg name': 'Ipmroved', 'fold': 4, 'lr': '0.001-0.05', 'accuracy': 0.6129, 'tpr': 0.3002, 'fpr': 0.0818, 'precision': 0.3887, 'auc': 0.7132, 'pr_curve': 0.2815, 'train_time': 4.6676, 'infer_time': 0.252}, {'dataset name': 'cifar10_9000:10000', 'alg name': 'Ipmroved', 'fold': 5, 'lr': '0.002-0.05', 'accuracy': 0.5375, 'tpr': 0.2905, 'fpr': 0.0791, 'precision': 0.3924, 'auc': 0.7219, 'pr_curve': 0.3161, 'train_time': 4.2364, 'infer_time': 1.9611}, {'dataset name': 'cifar10_9000:10000', 'alg name': 'Ipmroved', 'fold': 6, 'lr': '0.001-0.05', 'accuracy': 0.5715, 'tpr': 0.3088, 'fpr': 0.0784, 'precision': 0.3244, 'auc': 0.7868, 'pr_curve': 0.3981, 'train_time': 4.3465, 'infer_time': 0.2493}, {'dataset name': 'cifar10_9000:10000', 'alg name': 'Ipmroved', 'fold': 7, 'lr': '0.001-0.05', 'accuracy': 0.603, 'tpr': 0.3926, 'fpr': 0.0707, 'precision': 0.4184, 'auc': 0.7994, 'pr_curve': 0.3781, 'train_time': 4.5992, 'infer_time': 1.577}, {'dataset name': 'cifar10_9000:10000', 'alg name': 'Ipmroved', 'fold': 8, 'lr': '0.002-0.05', 'accuracy': 0.5883, 'tpr': 0.2116, 'fpr': 0.0916, 'precision': 0.1511, 'auc': 0.6635, 'pr_curve': 0.2603, 'train_time': 4.3059, 'infer_time': 0.2436}, {'dataset name': 'cifar10_9000:10000', 'alg name': 'Ipmroved', 'fold': 9, 'lr': '0.002-0.038', 'accuracy': 0.5344, 'tpr': 0.3179, 'fpr': 0.081, 'precision': 0.3729, 'auc': 0.7512, 'pr_curve': 0.3277, 'train_time': 4.387, 'infer_time': 0.2348}, {'dataset name': 'cifar10_9000:10000', 'alg name': 'Ipmroved', 'fold': 10, 'lr': '0.002-0.05', 'accuracy': 0.6215, 'tpr': 0.2198, 'fpr': 0.0836, 'precision': 0.2283, 'auc': 0.7255, 'pr_curve': 0.2969, 'train_time': 3.905, 'infer_time': 0.235}]

train_datasets = [d11_scores, d12_scores, d13_scores, d14_scores, d15_scores, d16_scores, d17_scores, d18_scores, d19_scores, d20_scores]
train_rows = d11_scores + d12_scores + d13_scores + d14_scores + d15_scores + d16_scores + d17_scores + d18_scores + d19_scores + d20_scores

In [32]:
train_aucs = [ [np.mean([score["auc"] 
                         for score in dataset[i*10:(i+1)*10]]) 
                for i in range(3) ] 
              for dataset in train_datasets]
train_aucs

[[0.7599099999999999, 0.74958, 0.7590600000000001],
 [0.77856, 0.76437, 0.76508],
 [0.77591, 0.74813, 0.75349],
 [0.7478, 0.72534, 0.74656],
 [0.7683900000000001, 0.7648299999999999, 0.75715],
 [0.7636000000000001, 0.73084, 0.7580600000000001],
 [0.7529399999999999, 0.7541500000000001, 0.7366],
 [0.74244, 0.76346, 0.74119],
 [0.7518, 0.74137, 0.73116],
 [0.75952, 0.7514200000000001, 0.73451]]

In [22]:
df = pd.DataFrame(test_rows + train_rows)
df.to_csv("example20.csv")

In [40]:
all_aucs = test_aucs + train_aucs
all_aucs_t = np.transpose(all_aucs)
print(all_aucs_t)
_, p_value = friedmanchisquare(*all_aucs_t)

[[0.7672  0.73832 0.75329 0.76243 0.76031 0.74667 0.74772 0.76738 0.7166
  0.75142 0.75991 0.77856 0.77591 0.7478  0.76839 0.7636  0.75294 0.74244
  0.7518  0.75952]
 [0.76543 0.74924 0.75957 0.76091 0.76981 0.7358  0.74664 0.78081 0.7376
  0.74622 0.74958 0.76437 0.74813 0.72534 0.76483 0.73084 0.75415 0.76346
  0.74137 0.75142]
 [0.73004 0.75493 0.75239 0.74236 0.73706 0.71757 0.73365 0.75242 0.70947
  0.74007 0.75906 0.76508 0.75349 0.74656 0.75715 0.75806 0.7366  0.74119
  0.73116 0.73451]]


In [41]:
p_value

0.00038975196825275134

In [42]:
posthoc_nemenyi_friedman(all_aucs)

,0,1,2
0,1.000000,0.190777,0.001000
1,0.190777,1.000000,0.068941
2,0.001000,0.068941,1.000000
